time to use usgs api much more straightforward

In [16]:
import requests
import pandas as pd
import numpy as np
import geopandas as gpd
import shapely
from shapely.geometry import Polygon, MultiPolygon
import json
from pathlib import Path
import sys

bloom_df = gpd.read_file("WildflowerBlooms_AreaOfInterest.geojson")
#testing if the api works
serviceUrl = "https://m2m.cr.usgs.gov/api/api/json/stable/"
url = serviceUrl + "login-token"
# login-token
payload = {"username" : "nasef", "token" : "ukeM_NMUg8SUa_IL_4qxDXR3NZycquwQjSp8441Gcd38OCOQMVHysFoj@C05zsHS"}
json_data = json.dumps(payload)
response = requests.post(url, json_data)
try:
    httpStatusCode = response.status_code
    if response == None:
        print("No output from service")
        sys.exit()

    output = json.loads(response.text)
    if output['errorCode'] != None:    
        print(output)
        sys.exit()
    if httpStatusCode == 404:
        print("404 Not Found")
        sys.exit()
    elif httpStatusCode == 403:
        print("401 Unauthorized")
        sys.exit()
    elif httpStatusCode == 500:
        print("Error Code", httpStatusCode)
        sys.exit()
except Exception as e:
    response.close()
    print(e)

response.close()
apiKey = output['data']

In [21]:
selected_site = bloom_df.iloc[1]  # testing for Carrizo Plain National Monument
site_geometry = selected_site.geometry

def geometry_to_geojson_dict(geom):
    """Converts a Shapely geometry to a GeoJSON dictionary using the official Shapely function."""
    geojson_string = shapely.to_geojson(geom)
    geojson_dict = json.loads(geojson_string)
    return geojson_dict

# Convert the site_geometry to GeoJSON
geometry_geojson = geometry_to_geojson_dict(site_geometry)

# Define the search period for the bloom season (use consistent YYYY-MM-DD format)
search_payload = {
    "datasetName": "landsat_tm_c2_l2",
    "spatialFilter": {
        "filterType": "geojson", 
        "geoJson": geometry_geojson
    },
    "temporalFilter": {
        "startDate": "2023-03-01",
        "endDate": "2023-05-31"
    },
    "sceneFilter": {
        "acquisitionFilter": {
            "include": ["Landsat_9"]  # Filter for Landsat 9 only
        }
    },
    "maxResults": 5
}

# Make the search request with authentication
API_URL = "https://m2m.cr.usgs.gov/api/api/json/stable/"
search = requests.post(API_URL + "scene-search", json=search_payload, headers={"X-Auth-Token": apiKey}).json()
print(search)

{'requestId': 2095298281, 'version': 'stable', 'sessionId': 323083841, 'data': {'results': [{'browse': [{'id': '5fb4b9d0e5094d61', 'browseRotationEnabled': None, 'browseName': 'Level-1 Reflective Browse', 'browsePath': 'https://landsatlook.usgs.gov/gen-browse?size=rrb&type=refl&product_id=LT05_L1TP_038037_20120505_20200820_02_T1', 'overlayPath': 'https://landsatlook.usgs.gov/dynamic-tiler/scenes/LT05_L1TP_038037_20120505_20200820_02_T1/tiles/{z}/{x}/{y}.png?layer=natural_color', 'overlayType': 'ls_chs', 'thumbnailPath': 'https://landsatlook.usgs.gov/gen-browse?size=thumb&type=refl&product_id=LT05_L1TP_038037_20120505_20200820_02_T1'}], 'cloudCover': 3, 'entityId': 'LT50380372012126EDC00', 'displayId': 'LT05_L2SP_038037_20120505_20200820_02_T1', 'orderingId': None, 'metadata': [{'id': '5e83d1199c3a007a', 'fieldName': 'ID', 'dictionaryLink': '', 'value': 'LT05_L2SP_038037_20120505_20200820_02_T1'}, {'id': '5e83d1197bb3b471', 'fieldName': 'Landsat Product Identifier L2', 'dictionaryLink':

In [23]:
file = {
   "requestId":2095298281,
   "version":"stable",
   "sessionId":323083841,
   "data":{
      "results":[
         {
            "browse":[
               {
                  "id":"5fb4b9d0e5094d61",
                  "browseRotationEnabled":"None",
                  "browseName":"Level-1 Reflective Browse",
                  "browsePath":"https://landsatlook.usgs.gov/gen-browse?size=rrb&type=refl&product_id=LT05_L1TP_038037_20120505_20200820_02_T1",
                  "overlayPath":"https://landsatlook.usgs.gov/dynamic-tiler/scenes/LT05_L1TP_038037_20120505_20200820_02_T1/tiles/{z}/{x}/{y}.png?layer=natural_color",
                  "overlayType":"ls_chs",
                  "thumbnailPath":"https://landsatlook.usgs.gov/gen-browse?size=thumb&type=refl&product_id=LT05_L1TP_038037_20120505_20200820_02_T1"
               }
            ],
            "cloudCover":3,
            "entityId":"LT50380372012126EDC00",
            "displayId":"LT05_L2SP_038037_20120505_20200820_02_T1",
            "orderingId":"None",
            "metadata":[
               {
                  "id":"5e83d1199c3a007a",
                  "fieldName":"ID",
                  "dictionaryLink":"",
                  "value":"LT05_L2SP_038037_20120505_20200820_02_T1"
               },
               {
                  "id":"5e83d1197bb3b471",
                  "fieldName":"Landsat Product Identifier L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#landsat_product_id",
                  "value":"LT05_L2SP_038037_20120505_20200820_02_T1"
               },
               {
                  "id":"5e83d1192913040c",
                  "fieldName":"Landsat Product Identifier L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#landsat_product_id",
                  "value":"LT05_L2SP_038037_20120505_20200820_02_T1"
               },
               {
                  "id":"5e83d11959960e26",
                  "fieldName":"Landsat Product Identifier L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#landsat_product_id",
                  "value":"LT05_L1TP_038037_20120505_20200820_02_T1"
               },
               {
                  "id":"5e83d119d77c851b",
                  "fieldName":"Landsat Product Identifier L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#landsat_product_id",
                  "value":"LT05_L1TP_038037_20120505_20200820_02_T1"
               },
               {
                  "id":"5e83d119e1b717c2",
                  "fieldName":"ACQ_DATE",
                  "dictionaryLink":"",
                  "value":"2012/05/05"
               },
               {
                  "id":"5e83d11991cd15e9",
                  "fieldName":"Date Acquired",
                  "dictionaryLink":"",
                  "value":"2012/05/05"
               },
               {
                  "id":"5e83d119cd314e81",
                  "fieldName":"WRS_PATH",
                  "dictionaryLink":"",
                  "value":38
               },
               {
                  "id":"5e83d11953cebdfe",
                  "fieldName":"Landsat Scene Identifier",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#landsat_scene_id",
                  "value":"LT50380372012126EDC00"
               },
               {
                  "id":"5e83d1194a136c3d",
                  "fieldName":"Landsat Scene Identifier",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#landsat_scene_id",
                  "value":"LT50380372012126EDC00"
               },
               {
                  "id":"5e83d1192f6378c5",
                  "fieldName":"Path",
                  "dictionaryLink":"",
                  "value":" 038"
               },
               {
                  "id":"5e83d119f944a59c",
                  "fieldName":"Date Acquired",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#acquisition_date",
                  "value":"2012/05/05"
               },
               {
                  "id":"5e83d1194a5390ef",
                  "fieldName":"Date Acquired",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#acquisition_date",
                  "value":"2012/05/05"
               },
               {
                  "id":"5e83d11918829eab",
                  "fieldName":"Row",
                  "dictionaryLink":"",
                  "value":" 037"
               },
               {
                  "id":"5e83d1193fd26358",
                  "fieldName":"WRS_ROW",
                  "dictionaryLink":"",
                  "value":37
               },
               {
                  "id":"5e83d119c421c04",
                  "fieldName":"UL_LAT",
                  "dictionaryLink":"",
                  "value":"34.12700"
               },
               {
                  "id":"5e83d1197bb52c7b",
                  "fieldName":"Collection Category",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#collection_category",
                  "value":"T1"
               },
               {
                  "id":"5e83d119d3605acf",
                  "fieldName":"Collection Category",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#collection_category",
                  "value":"T1"
               },
               {
                  "id":"5e83d1193fcff364",
                  "fieldName":"Collection Number",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#collection_number",
                  "value":2
               },
               {
                  "id":"5e83d119d14a7b88",
                  "fieldName":"Collection Number",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#collection_number",
                  "value":2
               },
               {
                  "id":"5e83d11938eb6b8b",
                  "fieldName":"UL_LON",
                  "dictionaryLink":"",
                  "value":"-114.90113"
               },
               {
                  "id":"5e83d11959773042",
                  "fieldName":"UR_LAT",
                  "dictionaryLink":"",
                  "value":"32.26169"
               },
               {
                  "id":"5e83d1191e09dd34",
                  "fieldName":"Sensor Mode",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sensor_mode",
                  "value":"BUMPER"
               },
               {
                  "id":"5e83d119f3f29818",
                  "fieldName":"Sensor Mode",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sensor_mode",
                  "value":"BUMPER"
               },
               {
                  "id":"5e83d11967e7be31",
                  "fieldName":"WRS Path",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#wrs_path",
                  "value":" 038"
               },
               {
                  "id":"5e83d11934daa06d",
                  "fieldName":"WRS Path",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#wrs_path",
                  "value":" 038"
               },
               {
                  "id":"5e83d119dfa54533",
                  "fieldName":"UR_LON",
                  "dictionaryLink":"",
                  "value":"-112.92895"
               },
               {
                  "id":"5e83d119d19269d9",
                  "fieldName":"LR_LAT",
                  "dictionaryLink":"",
                  "value":"32.26169"
               },
               {
                  "id":"5e83d119d93462c",
                  "fieldName":"WRS Row",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#wrs_row",
                  "value":" 037"
               },
               {
                  "id":"5e83d119175a9f85",
                  "fieldName":"WRS Row",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#wrs_row",
                  "value":" 037"
               },
               {
                  "id":"5e83d119a2f770bb",
                  "fieldName":"LR_LON",
                  "dictionaryLink":"",
                  "value":"-113.38145"
               },
               {
                  "id":"5e83d1193eb1eda1",
                  "fieldName":"Full Partial Scene",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#full_partial_scene",
                  "value":"FULL"
               },
               {
                  "id":"5e83d119b5031133",
                  "fieldName":"Full Partial Scene",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#full_partial_scene",
                  "value":"FULL"
               },
               {
                  "id":"5e83d1199d010929",
                  "fieldName":"Date Product Generated L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#date_l2_generated",
                  "value":"2020/08/20"
               },
               {
                  "id":"5e83d11996b0840e",
                  "fieldName":"Date Product Generated L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#date_l2_generated",
                  "value":"2020/08/20"
               },
               {
                  "id":"5e83d119524c32e",
                  "fieldName":"LL_LAT",
                  "dictionaryLink":"",
                  "value":"32.54365"
               },
               {
                  "id":"5e83d119aa12a17b",
                  "fieldName":"LL_LON",
                  "dictionaryLink":"",
                  "value":"-115.31971"
               },
               {
                  "id":"5e83d119c4c93633",
                  "fieldName":"Date Product Generated L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#date_l1_generated",
                  "value":"2020/08/20"
               },
               {
                  "id":"5e83d11925f4d7b1",
                  "fieldName":"Date Product Generated L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#date_l1_generated",
                  "value":"2020/08/20"
               },
               {
                  "id":"5e83d1198198937",
                  "fieldName":"Start Time",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#start_time",
                  "value":"2012-05-05 17:53:53.51056"
               },
               {
                  "id":"5e83d119799cfc69",
                  "fieldName":"Start Time",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#start_time",
                  "value":"2012-05-05 17:53:53.51056"
               },
               {
                  "id":"5e83d119c231dfdb",
                  "fieldName":"Stop Time",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#stop_time",
                  "value":"2012-05-05 17:54:20.12344"
               },
               {
                  "id":"5e83d1197f85f30",
                  "fieldName":"Stop Time",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#stop_time",
                  "value":"2012-05-05 17:54:20.12344"
               },
               {
                  "id":"5e83d1192d8d752e",
                  "fieldName":"Station Identifier",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#distribution_site",
                  "value":"EDC"
               },
               {
                  "id":"5e83d119e814b3d5",
                  "fieldName":"Station Identifier",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#distribution_site",
                  "value":"EDC"
               },
               {
                  "id":"5e83d119a6b422f1",
                  "fieldName":"Day/Night Indicator",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#day_or_night",
                  "value":"DAY"
               },
               {
                  "id":"5e83d119d580c676",
                  "fieldName":"Day/Night Indicator",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#day_or_night",
                  "value":"DAY"
               },
               {
                  "id":"5e83d1191dbc4bb2",
                  "fieldName":"Land Cloud Cover",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#cloud_cover_land",
                  "value":"   3.00"
               },
               {
                  "id":"5e83d119241ce3ad",
                  "fieldName":"Land Cloud Cover",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#cloud_cover_land",
                  "value":"   3.00"
               },
               {
                  "id":"5e83d1192b06440a",
                  "fieldName":"Scene Cloud Cover L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#cloud_cover",
                  "value":"   3.00"
               },
               {
                  "id":"5e83d119c9649ed5",
                  "fieldName":"Scene Cloud Cover L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#cloud_cover",
                  "value":"   3.00"
               },
               {
                  "id":"5e83d1193835ad13",
                  "fieldName":"Ground Control Points Model",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ground_control_points_model",
                  "value":859
               },
               {
                  "id":"5e83d119f145b805",
                  "fieldName":"Ground Control Points Model",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ground_control_points_model",
                  "value":859
               },
               {
                  "id":"5e83d119c72d835f",
                  "fieldName":"Ground Control Points Version",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ground_control_points_version",
                  "value":5
               },
               {
                  "id":"5e83d1195d9e6973",
                  "fieldName":"Ground Control Points Version",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ground_control_points_version",
                  "value":5
               },
               {
                  "id":"5e83d119369526c2",
                  "fieldName":"Geometric RMSE Model",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#geometric_rmse_model",
                  "value":"    3.830"
               },
               {
                  "id":"5e83d1199b894cc6",
                  "fieldName":"Geometric RMSE Model",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#geometric_rmse_model",
                  "value":"    3.830"
               },
               {
                  "id":"5e83d119eb4a52b2",
                  "fieldName":"Geometric RMSE Model X",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#geometric_rmse_model_x",
                  "value":"    2.429"
               },
               {
                  "id":"5e83d119e8b2a364",
                  "fieldName":"Geometric RMSE Model X",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#geometric_rmse_model_x",
                  "value":"    2.429"
               },
               {
                  "id":"5e83d119b3959f54",
                  "fieldName":"Geometric RMSE Model Y",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#geometric_rmse_model_y   ",
                  "value":"    2.961"
               },
               {
                  "id":"5e83d119545197e5",
                  "fieldName":"Geometric RMSE Model Y",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#geometric_rmse_model_y",
                  "value":"    2.961"
               },
               {
                  "id":"5e83d119c6488aea",
                  "fieldName":"Sensor Anomalies",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sensor_anomalies",
                  "value":"NONE"
               },
               {
                  "id":"5e83d11915d08447",
                  "fieldName":"Sensor Anomalies",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sensor_anomalies",
                  "value":"NONE"
               },
               {
                  "id":"5e83d119a837407a",
                  "fieldName":"Data Anomaly",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#data_anomaly",
                  "value":"None"
               },
               {
                  "id":"5e83d119907f9709",
                  "fieldName":"Data Anomaly",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#data_anomaly",
                  "value":"None"
               },
               {
                  "id":"5e83d119da3199a5",
                  "fieldName":"Image Quality",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#image_quality_f1",
                  "value":9
               },
               {
                  "id":"5e83d119c9703e76",
                  "fieldName":"Image Quality",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#image_quality_f1",
                  "value":9
               },
               {
                  "id":"5e83d119ed82d34a",
                  "fieldName":"Gain Change",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#gain_change_band_1",
                  "value":"N"
               },
               {
                  "id":"5e83d119160507a9",
                  "fieldName":"Gain Change",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#gain_change_band_1",
                  "value":"N"
               },
               {
                  "id":"5e83d11961e2e91f",
                  "fieldName":"Processing Software Version",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#processing_software_version",
                  "value":"LPGS_15.3.1c"
               },
               {
                  "id":"5e83d119e682a749",
                  "fieldName":"Processing Software Version",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#processing_software_version",
                  "value":"LPGS_15.3.1c"
               },
               {
                  "id":"5e83d119d68c3c56",
                  "fieldName":"Sun Elevation L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sun_elevation",
                  "value":" 62.14245250"
               },
               {
                  "id":"5e83d119b4f2499a",
                  "fieldName":"Sun Elevation L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sun_elevation",
                  "value":" 62.14245250"
               },
               {
                  "id":"5e83d11974719513",
                  "fieldName":"Sun Azimuth L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sun_azimuth",
                  "value":" 120.71516953"
               },
               {
                  "id":"5e83d1196bd1a17e",
                  "fieldName":"Sun Azimuth L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sun_azimuth",
                  "value":" 120.71516953"
               },
               {
                  "id":"5e83d119bd6f280e",
                  "fieldName":"Data Type L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#data_type_21",
                  "value":"TM_L2SP"
               },
               {
                  "id":"5e83d119e43f7db9",
                  "fieldName":"Data Type L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#data_type_21",
                  "value":"TM_L2SP"
               },
               {
                  "id":"5e83d119709b05d5",
                  "fieldName":"Sensor Identifier",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sensor_id",
                  "value":"TM"
               },
               {
                  "id":"5e83d11922c49edf",
                  "fieldName":"Satellite",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#satellite",
                  "value":"LANDSAT_5"
               },
               {
                  "id":"5e83d11943388776",
                  "fieldName":"Sensor Identifier",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sensor_id",
                  "value":"TM"
               },
               {
                  "id":"5e83d1194ce8c25",
                  "fieldName":"Ephemeris Type",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ephemeris_type",
                  "value":"DEFINITIVE"
               },
               {
                  "id":"5e83d119620279fa",
                  "fieldName":"Satellite",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#satellite",
                  "value":5
               },
               {
                  "id":"5e83d119c93da2ae",
                  "fieldName":"Product Map Projection L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#map_projection_l2",
                  "value":"UTM"
               },
               {
                  "id":"5e83d11994227806",
                  "fieldName":"Ephemeris Type",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ephemeris_type",
                  "value":"DEFINITIVE"
               },
               {
                  "id":"5e83d11920d22995",
                  "fieldName":"UTM Zone",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#utm_zone",
                  "value":11
               },
               {
                  "id":"5e83d119dfc32fd1",
                  "fieldName":"Datum",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#datum",
                  "value":"WGS84"
               },
               {
                  "id":"5e83d119889cd954",
                  "fieldName":"Ellipsoid",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ellipsoid",
                  "value":"WGS84"
               },
               {
                  "id":"5e83d119629796c8",
                  "fieldName":"Product Map Projection L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#map_projection_l2",
                  "value":"UTM"
               },
               {
                  "id":"5e83d119fadd1071",
                  "fieldName":"UTM Zone",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#utm_zone",
                  "value":11
               },
               {
                  "id":"5e83d119388164c",
                  "fieldName":"Datum",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#datum",
                  "value":"WGS84"
               },
               {
                  "id":"5e83d119e4ff9290",
                  "fieldName":"Ellipsoid",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ellipsoid",
                  "value":"WGS84"
               },
               {
                  "id":"5e83d1193b2598ef",
                  "fieldName":"Scene Center Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"33&deg;11\\'49.52\"N"
               },
               {
                  "id":"5e83d119cf9cdcd0",
                  "fieldName":"Scene Center Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"114&deg;07\\'57.68\"W"
               },
               {
                  "id":"5e83d1197f20b27e",
                  "fieldName":"Corner Upper Left Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"34&deg;07\\'37.20\"N"
               },
               {
                  "id":"5e83d11936b6b01f",
                  "fieldName":"Corner Upper Left Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"114&deg;54\\'04.07\"W"
               },
               {
                  "id":"5e83d11961cccb4e",
                  "fieldName":"Corner Upper Right Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"33&deg;50\\'24.14\"N"
               },
               {
                  "id":"5e83d119914affb9",
                  "fieldName":"Corner Upper Right Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"112&deg;55\\'44.22\"W"
               },
               {
                  "id":"5e83d11933e075b3",
                  "fieldName":"Scene Center Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"33&deg;11\\'49.52\"N"
               },
               {
                  "id":"5e83d11929b7c342",
                  "fieldName":"Corner Lower Left Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"32&deg;32\\'37.14\"N"
               },
               {
                  "id":"5e83d11915ccea1b",
                  "fieldName":"Originator",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d119d03bef11",
                  "fieldName":"Scene Center Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"114&deg;07\\'57.68\"W"
               },
               {
                  "id":"5e83d119b27f36cd",
                  "fieldName":"Corner Upper Left Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"34&deg;07\\'37.20\"N"
               },
               {
                  "id":"5e83d119e5a5bc5a",
                  "fieldName":"Corner Lower Left Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"115&deg;19\\'10.96\"W"
               },
               {
                  "id":"5e83d119ae3d36ca",
                  "fieldName":"Corner Lower Right Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"32&deg;15\\'42.08\"N"
               },
               {
                  "id":"5e83d1197695fcdf",
                  "fieldName":"Corner Upper Left Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"114&deg;54\\'04.07\"W"
               },
               {
                  "id":"5e83d119ea4acbd4",
                  "fieldName":"Corner Upper Right Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"33&deg;50\\'24.14\"N"
               },
               {
                  "id":"5e83d119adad6c55",
                  "fieldName":"Corner Lower Right Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"113&deg;22\\'53.22\"W"
               },
               {
                  "id":"5e83d119f6d182f7",
                  "fieldName":"Corner Upper Right Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"112&deg;55\\'44.22\"W"
               },
               {
                  "id":"5e83d1197cfbeb7a",
                  "fieldName":"Scene Center Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 33.19709"
               },
               {
                  "id":"5e83d119dc6d299d",
                  "fieldName":"Scene Center Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-114.13269"
               },
               {
                  "id":"5e83d11932520ee0",
                  "fieldName":"Corner Lower Left Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"32&deg;32\\'37.14\"N"
               },
               {
                  "id":"5e83d1199113b057",
                  "fieldName":"Corner Lower Left Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"115&deg;19\\'10.96\"W"
               },
               {
                  "id":"5e83d11960e9d77a",
                  "fieldName":"Corner Upper Left Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 34.12700"
               },
               {
                  "id":"5e83d119595283a3",
                  "fieldName":"Corner Upper Left Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-114.90113"
               },
               {
                  "id":"5e83d1198025a16",
                  "fieldName":"Corner Lower Right Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"32&deg;15\\'42.08\"N"
               },
               {
                  "id":"5e83d119bac5c53b",
                  "fieldName":"Corner Lower Right Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"113&deg;22\\'53.22\"W"
               },
               {
                  "id":"5e83d119539410e8",
                  "fieldName":"Corner Upper Right Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 33.84004"
               },
               {
                  "id":"5e83d119189a44a8",
                  "fieldName":"Corner Upper Right Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-112.92895"
               },
               {
                  "id":"5e83d119e58ff41",
                  "fieldName":"Scene Center Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 33.19709"
               },
               {
                  "id":"5e83d119ac572a41",
                  "fieldName":"Publication_Date",
                  "dictionaryLink":"",
                  "value":"20050831"
               },
               {
                  "id":"5e83d119b97778e3",
                  "fieldName":"Corner Lower Left Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 32.54365"
               },
               {
                  "id":"5e83d119fc691855",
                  "fieldName":"Scene Center Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-114.13269"
               },
               {
                  "id":"5e83d119d033de1",
                  "fieldName":"Corner Lower Left Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-115.31971"
               },
               {
                  "id":"5e83d119b84d4d52",
                  "fieldName":"Corner Upper Left Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 34.12700"
               },
               {
                  "id":"5e83d119aef3d62e",
                  "fieldName":"Corner Lower Right Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 32.26169"
               },
               {
                  "id":"5e83d1196f42a17a",
                  "fieldName":"Corner Upper Left Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-114.90113"
               },
               {
                  "id":"5e83d119a7e81d55",
                  "fieldName":"Corner Upper Right Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 33.84004"
               },
               {
                  "id":"5e83d11967541ce6",
                  "fieldName":"Corner Lower Right Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-113.38145"
               },
               {
                  "id":"5e83d119a9ebd21b",
                  "fieldName":"Corner Upper Right Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-112.92895"
               },
               {
                  "id":"5e83d1199d5244f6",
                  "fieldName":"Corner Lower Left Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 32.54365"
               },
               {
                  "id":"5e83d119799824a8",
                  "fieldName":"Corner Lower Left Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-115.31971"
               },
               {
                  "id":"5e83d11917097daf",
                  "fieldName":"Corner Lower Right Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 32.26169"
               },
               {
                  "id":"5e83d119d6ce5eaf",
                  "fieldName":"Corner Lower Right Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-113.38145"
               },
               {
                  "id":"5e83d11913a82746",
                  "fieldName":"Title",
                  "dictionaryLink":"",
                  "value":"LANDSAT TM Collection 2 - Path:  38 Row:  37 for Scene:  LT05_L2SP_038037_20120505_20200820_02_T1"
               },
               {
                  "id":"5e83d11972d6e9f7",
                  "fieldName":"Geospatial_Data_Presentation_Form",
                  "dictionaryLink":"",
                  "value":"Remote-Sensing Image"
               },
               {
                  "id":"5e83d119920813bc",
                  "fieldName":"Publication_Place",
                  "dictionaryLink":"",
                  "value":"Sioux Falls, SD USA"
               },
               {
                  "id":"5e83d119df3fe2e7",
                  "fieldName":"Publisher",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d119d2a6e1c7",
                  "fieldName":"Online_Linkage",
                  "dictionaryLink":"",
                  "value":"https://doi.org/10.5066/P9IAXOVV"
               },
               {
                  "id":"5e83d1196aefe139",
                  "fieldName":"Abstract",
                  "dictionaryLink":"",
                  "value":"The USGS Earth Resources Observation and Science (EROS) Center archive holds data collected by the Landsat suite of satellites, beginning with Landsat 1 in 1972 through the latest Landsat Mission."
               },
               {
                  "id":"5e83d119684bd37c",
                  "fieldName":"Purpose",
                  "dictionaryLink":"",
                  "value":"The Landsat archive provides a rich collection of information about the Earth's land surface.  Major characteristics of changes to the surface of the planet can be detected, measured, and analyzed using Landsat data.  The information obtainable from the historical and current Landsat data play a key role in studying surface changes through time.  Collection 2 provides a quality-controlled selection of radiometrically calibrated and consistently geolocated data; particular geodetic accuracy and error tolerances are specified to align pixels for time series analysis through the full Landsat record."
               },
               {
                  "id":"5e83d11945f920c",
                  "fieldName":"Supplemental_Information",
                  "dictionaryLink":"",
                  "value":"Landsat represents the world's longest continuously acquired collection of space-based moderate-resolution land remote sensing data.  As a joint initiative between the U.S. Geological Survey (USGS) and the National Aeronautical Space Administration (NASA), the Landsat Project and the data it collects support government, commercial, industrial, civilian, military, and educational communities throughout the United States and world.  Landsat 8 Operational Land Imager (OLI)/Thermal Infrared Sensor (TIRS) Collection 2 provides a quality-controlled selection of radiometrically calibrated and consistently geolocated data; particular geodetic accuracy and error tolerances are specified to align pixels for time series analysis through the full Landsat record. (https://www.usgs.gov/core-science-systems/nli/landsat/landsat-collections)"
               },
               {
                  "id":"5e83d1197cc754b4",
                  "fieldName":"Calendar_Date",
                  "dictionaryLink":"",
                  "value":"19920704"
               },
               {
                  "id":"5e83d11966963850",
                  "fieldName":"Currentness_Reference",
                  "dictionaryLink":"",
                  "value":"Ground Condition"
               },
               {
                  "id":"5e83d119fc177449",
                  "fieldName":"Progress",
                  "dictionaryLink":"",
                  "value":"In Work"
               },
               {
                  "id":"5e83d11960ba536d",
                  "fieldName":"Maintenance_and_Update_Frequency",
                  "dictionaryLink":"",
                  "value":"Continually"
               },
               {
                  "id":"5e83d119586f0830",
                  "fieldName":"G-Ring_Latitude",
                  "dictionaryLink":"",
                  "value":"32.54365"
               },
               {
                  "id":"5e83d1191be10c7c",
                  "fieldName":"G-Ring_Longitude",
                  "dictionaryLink":"",
                  "value":"-115.31971"
               },
               {
                  "id":"5e83d119281579e6",
                  "fieldName":"Theme_Keyword_Thesaurus",
                  "dictionaryLink":"",
                  "value":"ISO 19115 Topic Category"
               },
               {
                  "id":"5e83d1194abc14dd",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"ImageryBaseMapsEarthCover"
               },
               {
                  "id":"60470441398df5b",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Remote Sensing"
               },
               {
                  "id":"604704413623c358",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Collection 2"
               },
               {
                  "id":"60470441c22543ba",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Infrared Imagery"
               },
               {
                  "id":"604704413ae1d393",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Imagery"
               },
               {
                  "id":"60470441a5f6eec9",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Visible Imagery"
               },
               {
                  "id":"6047044176fd6e3d",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Visible Wavelengths"
               },
               {
                  "id":"604704417339664e",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Radiance"
               },
               {
                  "id":"60470441fb324cd2",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Landsat"
               },
               {
                  "id":"60470441ef8940aa",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Satellite"
               },
               {
                  "id":"60470441fd132ec0",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) "
               },
               {
                  "id":"6047044187bc92ee",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Thematic Mapper (TM)"
               },
               {
                  "id":"6047044159f1e540",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"60470441627f211c",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Infrared Wavelengths"
               },
               {
                  "id":"5e83d119796d51f5",
                  "fieldName":"Place_Keyword_Thesaurus",
                  "dictionaryLink":"",
                  "value":"U.S. Department of Commerce, 1995, Countries, dependencies, areas of special sovereignty, and their principal administrative divisions, Federal Information Processing Standard 10-4, Washington, D.C., National Institute of Standards and Technology"
               },
               {
                  "id":"5e83d119abd5525a",
                  "fieldName":"Place_Keyword",
                  "dictionaryLink":"",
                  "value":"Land Surface"
               },
               {
                  "id":"5e83d119ba008a5",
                  "fieldName":"Access_Constraints",
                  "dictionaryLink":"",
                  "value":"Downloading and using these data signifies that the user comprehends and will comply with the USGS Privacy Policy.  Reading and understanding all portions of the metadata will protect the interests of both the user and the USGS. Please refer to https://www.usgs.gov/privacy.html."
               },
               {
                  "id":"5e83d11a5fe32ed8",
                  "fieldName":"Use_Constraints",
                  "dictionaryLink":"",
                  "value":"There is no guarantee of warranty concerning the accuracy of these data. Users should not use these data for critical applications without a full awareness of their limitations. Temporal changes may have occurred since the data were collected and some parts of these data may no longer represent actual surface conditions.  There are no restrictions regarding the use of Landsat data distributed by the U.S. Geological Survey (USGS), unless expressly identified prior to or at the time of receipt. For Landsat, we request the following statement be used: Data available from the U.S. Geological Survey. Please refer to https://www.usgs.gov/centers/eros/data-citation for more information on data citation."
               },
               {
                  "id":"5e83d11ab9c2ebac",
                  "fieldName":"Contact_Person",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11ab32a4f35",
                  "fieldName":"Contact_Organization",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d11ab61696bc",
                  "fieldName":"Contact_Position",
                  "dictionaryLink":"",
                  "value":"Customer Service Representative"
               },
               {
                  "id":"5e83d11ad3056599",
                  "fieldName":"Address_Type",
                  "dictionaryLink":"",
                  "value":"Mailing and Physical Address"
               },
               {
                  "id":"5e83d11a257fe85d",
                  "fieldName":"Address",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center 47914 252nd Street"
               },
               {
                  "id":"5e83d11ab3eacc95",
                  "fieldName":"City",
                  "dictionaryLink":"",
                  "value":"Sioux Falls"
               },
               {
                  "id":"5e83d11aa1694de",
                  "fieldName":"State_or_Province",
                  "dictionaryLink":"",
                  "value":"SD"
               },
               {
                  "id":"5e83d11ad2c2fb60",
                  "fieldName":"Postal_Code",
                  "dictionaryLink":"",
                  "value":"57198-0001"
               },
               {
                  "id":"5e83d11ae22dc5e3",
                  "fieldName":"Country",
                  "dictionaryLink":"",
                  "value":"USA"
               },
               {
                  "id":"5e83d11aa9f27cf6",
                  "fieldName":"Contact_Electronic_Mail_Address",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a540dcfe9",
                  "fieldName":"Hours_of_Service",
                  "dictionaryLink":"",
                  "value":"0800 - 1600 CT, M-F, 1400 - 2200 h GMT"
               },
               {
                  "id":"5e83d11ac32f724c",
                  "fieldName":"Contact_Instructions",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a5c48c7bc",
                  "fieldName":"Browse_Graphic_File_Name",
                  "dictionaryLink":"",
                  "value":"Not Available"
               },
               {
                  "id":"5e83d11ad8675c1d",
                  "fieldName":"Browse_Graphic_File_Description",
                  "dictionaryLink":"",
                  "value":"This is a compressed and resampled version of the Landsat image to help validate area of interest, data quality, and the impact of cloud cover, if it exists."
               },
               {
                  "id":"5e83d11ae71a3055",
                  "fieldName":"Browse_Graphic_File_Type",
                  "dictionaryLink":"",
                  "value":"JPEG"
               },
               {
                  "id":"5e83d11a9000b8ec",
                  "fieldName":"Security_Classification_System",
                  "dictionaryLink":"",
                  "value":"None"
               },
               {
                  "id":"5e83d11aa39ac7de",
                  "fieldName":"Security_Classification",
                  "dictionaryLink":"",
                  "value":"Unclassified"
               },
               {
                  "id":"5e83d11a1883ab2b",
                  "fieldName":"Security_Handling_Description",
                  "dictionaryLink":"",
                  "value":"None"
               },
               {
                  "id":"5e83d11a7a5e7378",
                  "fieldName":"Native_Data_Set_Environment",
                  "dictionaryLink":"",
                  "value":"Oracle"
               },
               {
                  "id":"5e83d11a53641db4",
                  "fieldName":"Originator",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d11a23ff726c",
                  "fieldName":"Publication_Date",
                  "dictionaryLink":"",
                  "value":"19720723"
               },
               {
                  "id":"5e83d11a3ae40d19",
                  "fieldName":"Title",
                  "dictionaryLink":"",
                  "value":"Landsat Thematic Mapper Collection 2"
               },
               {
                  "id":"5e83d11aa866af3f",
                  "fieldName":"Geospatial_Data_Presentation_Form",
                  "dictionaryLink":"",
                  "value":"Remote-Sensing Image"
               },
               {
                  "id":"5e83d11a3ff08366",
                  "fieldName":"Publication_Place",
                  "dictionaryLink":"",
                  "value":"Sioux Falls,  SD USA"
               },
               {
                  "id":"5e83d11ac764e8b1",
                  "fieldName":"Publisher",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d11af27a7036",
                  "fieldName":"Online_Linkage",
                  "dictionaryLink":"",
                  "value":"https://doi.org/10.5066/P9IAXOVV"
               },
               {
                  "id":"5e83d11a393fe841",
                  "fieldName":"Attribute_Accuracy_Report",
                  "dictionaryLink":"",
                  "value":"The identification of features is provided by the spectral signature which is the distinct electromagnetic energy it emits, reflects, or otherwise transmits.  Other signatures such as tone (lightness or darkness), texture (surface roughness or smoothness), pattern, shadow, shape, and size are just as important.  Thus, through the use of Landsat data, such elements as water features, soils, and vegetation can be identified and distinguished from each other."
               },
               {
                  "id":"5e83d11abfffd008",
                  "fieldName":"Logical_Consistency_Report",
                  "dictionaryLink":"",
                  "value":"N/A"
               },
               {
                  "id":"5e83d11a3a7b431a",
                  "fieldName":"Completeness_Report",
                  "dictionaryLink":"",
                  "value":"The Landsat satellites circle the Earth on a sun-synchronous, near-polar orbit imaging the same 185 km (115 miles) ground swath every 16 days."
               },
               {
                  "id":"5e83d11adfb52c64",
                  "fieldName":"Process_Description",
                  "dictionaryLink":"",
                  "value":"Landsat Collection 2 Level-2 surface reflectance and surface temperature science products are generated from Collection 2 Level-1 inputs that meet the less than 76 degrees Solar Zenith Angle constraint and include the required auxiliary data inputs to generate a scientifically viable product. For more information visit https://www.usgs.gov/core-science-systems/nli/landsat/landsat-collection-2-level-2-science-products."
               },
               {
                  "id":"5e83d11a886ef0cd",
                  "fieldName":"Process_Date",
                  "dictionaryLink":"",
                  "value":"19920704"
               },
               {
                  "id":"5e83d11a11da2ce6",
                  "fieldName":"Contact_Person",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11ab064168",
                  "fieldName":"Contact_Organization",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d11a2304f959",
                  "fieldName":"Contact_Organization",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d11a3bbcd0f2",
                  "fieldName":"Contact_Person",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a9e6d5dd6",
                  "fieldName":"Contact_Position",
                  "dictionaryLink":"",
                  "value":"Customer Service Representative"
               },
               {
                  "id":"5e83d11adff5e984",
                  "fieldName":"Address_Type",
                  "dictionaryLink":"",
                  "value":"Mailing and Physical Address"
               },
               {
                  "id":"5e83d11af63fd80e",
                  "fieldName":"Address",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center 47914 252nd Street"
               },
               {
                  "id":"5e83d11afb799f4a",
                  "fieldName":"City",
                  "dictionaryLink":"",
                  "value":"Sioux Falls"
               },
               {
                  "id":"5e83d11a44b6f7fa",
                  "fieldName":"State_or_Province",
                  "dictionaryLink":"",
                  "value":"SD"
               },
               {
                  "id":"5e83d11a7fc5e25c",
                  "fieldName":"Postal_Code",
                  "dictionaryLink":"",
                  "value":"57198-0001"
               },
               {
                  "id":"5e83d11a85d3d795",
                  "fieldName":"Country",
                  "dictionaryLink":"",
                  "value":"USA"
               },
               {
                  "id":"5e83d11ae7dc9222",
                  "fieldName":"Contact_Electronic_Mail_Address",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11aeb5a94ea",
                  "fieldName":"Hours_of_Service",
                  "dictionaryLink":"",
                  "value":"0800 - 1600 CT, M-F, 1400 - 2200 h GMT"
               },
               {
                  "id":"5e83d11ac055787a",
                  "fieldName":"Contact_Instructions",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a858097e6",
                  "fieldName":"Cloud_Cover",
                  "dictionaryLink":"",
                  "value":"3.000 percent"
               },
               {
                  "id":"5e83d11af1f2843b",
                  "fieldName":"Indirect_Spatial_Reference",
                  "dictionaryLink":"",
                  "value":"Landsats 4 and 5 collected data at 30-meter ground resolution. Band 6 was acquired at 120-meter resolution, but products are resampled to 30-meter pixels."
               },
               {
                  "id":"5e83d11a6b630fc9",
                  "fieldName":"Direct_Spatial_Reference_Method",
                  "dictionaryLink":"",
                  "value":"Raster"
               },
               {
                  "id":"5e83d11a19dda88e",
                  "fieldName":"Raster_Object_Type",
                  "dictionaryLink":"",
                  "value":"Pixel"
               },
               {
                  "id":"5e83d11ac4295b28",
                  "fieldName":"Contact_Person",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a883a82be",
                  "fieldName":"Contact_Organization",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d11aa774b26b",
                  "fieldName":"Contact_Position",
                  "dictionaryLink":"",
                  "value":"Customer Service Representative"
               },
               {
                  "id":"5e83d11ad2958365",
                  "fieldName":"Address_Type",
                  "dictionaryLink":"",
                  "value":"Mailing and Physical Address"
               },
               {
                  "id":"5e83d11af5fa6ab8",
                  "fieldName":"Address",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center 47914 252nd Street"
               },
               {
                  "id":"5e83d11a60757afb",
                  "fieldName":"City",
                  "dictionaryLink":"",
                  "value":"Sioux Falls"
               },
               {
                  "id":"5e83d11a7d7342b1",
                  "fieldName":"State_or_Province",
                  "dictionaryLink":"",
                  "value":"SD"
               },
               {
                  "id":"5e83d11a7c7a009c",
                  "fieldName":"Postal_Code",
                  "dictionaryLink":"",
                  "value":"57198-0001"
               },
               {
                  "id":"5e83d11a6085f9c4",
                  "fieldName":"Country",
                  "dictionaryLink":"",
                  "value":"USA"
               },
               {
                  "id":"5e83d11a8414a2f7",
                  "fieldName":"Contact_Electronic_Mail_Address",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11afd59eb5f",
                  "fieldName":"Hours_of_Service",
                  "dictionaryLink":"",
                  "value":"0800 - 1600 CT, M-F, 1400 - 2200 h GMT"
               },
               {
                  "id":"5e83d11a95f467a9",
                  "fieldName":"Contact_Instructions",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11abaff6573",
                  "fieldName":"Resource_Description",
                  "dictionaryLink":"",
                  "value":"LANDSAT TM Collection 2 Entity ID: LT50380372012126EDC00"
               },
               {
                  "id":"5e83d11a92c457e3",
                  "fieldName":"Distribution_Liability",
                  "dictionaryLink":"",
                  "value":"Although these data have been processed successfully on a computer system at the USGS, no warranty expressed or implied is made by the USGS regarding the use of the data on any other system, nor does the act of distribution constitute any such warranty."
               },
               {
                  "id":"5e83d11af5c0878d",
                  "fieldName":"Format_Name",
                  "dictionaryLink":"",
                  "value":"Geographcal-Tagged Image File Format (GeoTIFF), or Joint Photographic Experts Group (JPEG)"
               },
               {
                  "id":"5e83d11ae0bb7b8f",
                  "fieldName":"Network_Resource_Name",
                  "dictionaryLink":"",
                  "value":"https://earthexplorer.usgs.gov"
               },
               {
                  "id":"5e83d11afb83c4cd",
                  "fieldName":"Fees",
                  "dictionaryLink":"",
                  "value":"Fees:  All Landsat data held in the USGS EROS archive are downloadable at no cost to all users."
               },
               {
                  "id":"5e83d11abe18accc",
                  "fieldName":"Technical_Prerequisites",
                  "dictionaryLink":"",
                  "value":"Adequate computer capability is the only technical prerequisite for viewing data in digital form.\r\n\r\n"
               },
               {
                  "id":"5e83d11a1a5caead",
                  "fieldName":"Beginning_Date",
                  "dictionaryLink":"",
                  "value":"19821113"
               },
               {
                  "id":"5e83d11a7ab01984",
                  "fieldName":"Ending_Date",
                  "dictionaryLink":"",
                  "value":"20011001"
               },
               {
                  "id":"5e83d11a5fe31dbe",
                  "fieldName":"Metadata_Date",
                  "dictionaryLink":"",
                  "value":"20160422"
               },
               {
                  "id":"5e83d11ace8c4be6",
                  "fieldName":"Metadata_Review_Date",
                  "dictionaryLink":"",
                  "value":"As Needed"
               },
               {
                  "id":"5e83d11aaf7b1a8",
                  "fieldName":"Contact_Person",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a6fe629a0",
                  "fieldName":"Contact_Organization",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d11a612f9dca",
                  "fieldName":"Contact_Position",
                  "dictionaryLink":"",
                  "value":"Customer Service Representative"
               },
               {
                  "id":"5e83d11ac64398d4",
                  "fieldName":"Address_Type",
                  "dictionaryLink":"",
                  "value":"Mailing and Physical Address"
               },
               {
                  "id":"5e83d11a2ec3653a",
                  "fieldName":"Address",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center 47914 252nd Street"
               },
               {
                  "id":"5e83d11a9dc347a2",
                  "fieldName":"City",
                  "dictionaryLink":"",
                  "value":"Sioux Falls"
               },
               {
                  "id":"5e83d11a557d59a2",
                  "fieldName":"State_or_Province",
                  "dictionaryLink":"",
                  "value":"SD"
               },
               {
                  "id":"5e83d11ab5408135",
                  "fieldName":"Postal_Code",
                  "dictionaryLink":"",
                  "value":"57198-0001"
               },
               {
                  "id":"5e83d11a12675aa1",
                  "fieldName":"Country",
                  "dictionaryLink":"",
                  "value":"USA"
               },
               {
                  "id":"5e83d11a44f7d71b",
                  "fieldName":"Contact_Electronic_Mail_Address",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a72d65a0f",
                  "fieldName":"Hours_of_Service",
                  "dictionaryLink":"",
                  "value":"0800 - 1600 CT, M-F, 1400 - 2200 h GMT"
               },
               {
                  "id":"5e83d11ac62ff739",
                  "fieldName":"Contact_Instructions",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a2b75b6e1",
                  "fieldName":"Metadata_Standard_Name",
                  "dictionaryLink":"",
                  "value":"Content Standard for Digital Geospatial Metadata"
               },
               {
                  "id":"5e83d11a7c7a0451",
                  "fieldName":"Metadata_Standard_Version",
                  "dictionaryLink":"",
                  "value":"FGDC-STD-001-1998, Version 2"
               },
               {
                  "id":"5e83d11a1ab1600f",
                  "fieldName":"Metadata_Time_Convention",
                  "dictionaryLink":"",
                  "value":"Local Time"
               },
               {
                  "id":"5e83d11a6418b4e",
                  "fieldName":"Metadata_Access_Constraints",
                  "dictionaryLink":"",
                  "value":"None"
               },
               {
                  "id":"5e83d11a37751741",
                  "fieldName":"Metadata_Use_Constraints",
                  "dictionaryLink":"",
                  "value":"None"
               },
               {
                  "id":"5e83d11a6948e4a6",
                  "fieldName":"Metadata_Security_Classification_System",
                  "dictionaryLink":"",
                  "value":"None"
               },
               {
                  "id":"5e83d11a96bf92d9",
                  "fieldName":"Metadata_Security_Classification",
                  "dictionaryLink":"",
                  "value":"Unclassified"
               },
               {
                  "id":"5e83d11ae14acc9d",
                  "fieldName":"Metadata_Security_Handling_Description",
                  "dictionaryLink":"",
                  "value":"None"
               }
            ],
            "hasCustomizedMetadata":"None",
            "options":{
               "bulk":True,
               "download":True,
               "order":False,
               "secondary":False
            },
            "selected":{
               "bulk":False,
               "compare":False,
               "order":False
            },
            "spatialBounds":{
               "type":"Polygon",
               "coordinates":[
                  [
                     [
                        -115.31971,
                        32.26169
                     ],
                     [
                        -115.31971,
                        34.127
                     ],
                     [
                        -112.92895,
                        34.127
                     ],
                     [
                        -112.92895,
                        32.26169
                     ],
                     [
                        -115.31971,
                        32.26169
                     ]
                  ]
               ]
            },
            "spatialCoverage":{
               "type":"Polygon",
               "coordinates":[
                  [
                     [
                        -115.31971,
                        32.54365
                     ],
                     [
                        -113.38145,
                        32.26169
                     ],
                     [
                        -112.92895,
                        33.84004
                     ],
                     [
                        -114.90113,
                        34.127
                     ],
                     [
                        -115.31971,
                        32.54365
                     ]
                  ]
               ]
            },
            "temporalCoverage":{
               "endDate":"2012-05-05 00:00:00",
               "startDate":"2012-05-05 00:00:00"
            },
            "publishDate":"2022-06-22 19:23:13-05"
         },
         {
            "browse":[
               {
                  "id":"5fb4b9d0e5094d61",
                  "browseRotationEnabled":"None",
                  "browseName":"Level-1 Reflective Browse",
                  "browsePath":"https://landsatlook.usgs.gov/gen-browse?size=rrb&type=refl&product_id=LT05_L1TP_031033_20120504_20200820_02_T1",
                  "overlayPath":"https://landsatlook.usgs.gov/dynamic-tiler/scenes/LT05_L1TP_031033_20120504_20200820_02_T1/tiles/{z}/{x}/{y}.png?layer=natural_color",
                  "overlayType":"ls_chs",
                  "thumbnailPath":"https://landsatlook.usgs.gov/gen-browse?size=thumb&type=refl&product_id=LT05_L1TP_031033_20120504_20200820_02_T1"
               }
            ],
            "cloudCover":0,
            "entityId":"LT50310332012125EDC00",
            "displayId":"LT05_L2SP_031033_20120504_20200820_02_T1",
            "orderingId":"None",
            "metadata":[
               {
                  "id":"5e83d1199c3a007a",
                  "fieldName":"ID",
                  "dictionaryLink":"",
                  "value":"LT05_L2SP_031033_20120504_20200820_02_T1"
               },
               {
                  "id":"5e83d1197bb3b471",
                  "fieldName":"Landsat Product Identifier L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#landsat_product_id",
                  "value":"LT05_L2SP_031033_20120504_20200820_02_T1"
               },
               {
                  "id":"5e83d1192913040c",
                  "fieldName":"Landsat Product Identifier L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#landsat_product_id",
                  "value":"LT05_L2SP_031033_20120504_20200820_02_T1"
               },
               {
                  "id":"5e83d11959960e26",
                  "fieldName":"Landsat Product Identifier L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#landsat_product_id",
                  "value":"LT05_L1TP_031033_20120504_20200820_02_T1"
               },
               {
                  "id":"5e83d119d77c851b",
                  "fieldName":"Landsat Product Identifier L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#landsat_product_id",
                  "value":"LT05_L1TP_031033_20120504_20200820_02_T1"
               },
               {
                  "id":"5e83d119e1b717c2",
                  "fieldName":"ACQ_DATE",
                  "dictionaryLink":"",
                  "value":"2012/05/04"
               },
               {
                  "id":"5e83d11991cd15e9",
                  "fieldName":"Date Acquired",
                  "dictionaryLink":"",
                  "value":"2012/05/04"
               },
               {
                  "id":"5e83d119cd314e81",
                  "fieldName":"WRS_PATH",
                  "dictionaryLink":"",
                  "value":31
               },
               {
                  "id":"5e83d11953cebdfe",
                  "fieldName":"Landsat Scene Identifier",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#landsat_scene_id",
                  "value":"LT50310332012125EDC00"
               },
               {
                  "id":"5e83d1194a136c3d",
                  "fieldName":"Landsat Scene Identifier",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#landsat_scene_id",
                  "value":"LT50310332012125EDC00"
               },
               {
                  "id":"5e83d1192f6378c5",
                  "fieldName":"Path",
                  "dictionaryLink":"",
                  "value":" 031"
               },
               {
                  "id":"5e83d119f944a59c",
                  "fieldName":"Date Acquired",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#acquisition_date",
                  "value":"2012/05/04"
               },
               {
                  "id":"5e83d1194a5390ef",
                  "fieldName":"Date Acquired",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#acquisition_date",
                  "value":"2012/05/04"
               },
               {
                  "id":"5e83d11918829eab",
                  "fieldName":"Row",
                  "dictionaryLink":"",
                  "value":" 033"
               },
               {
                  "id":"5e83d1193fd26358",
                  "fieldName":"WRS_ROW",
                  "dictionaryLink":"",
                  "value":33
               },
               {
                  "id":"5e83d119c421c04",
                  "fieldName":"UL_LAT",
                  "dictionaryLink":"",
                  "value":"39.85299"
               },
               {
                  "id":"5e83d1197bb52c7b",
                  "fieldName":"Collection Category",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#collection_category",
                  "value":"T1"
               },
               {
                  "id":"5e83d119d3605acf",
                  "fieldName":"Collection Category",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#collection_category",
                  "value":"T1"
               },
               {
                  "id":"5e83d1193fcff364",
                  "fieldName":"Collection Number",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#collection_number",
                  "value":2
               },
               {
                  "id":"5e83d119d14a7b88",
                  "fieldName":"Collection Number",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#collection_number",
                  "value":2
               },
               {
                  "id":"5e83d11938eb6b8b",
                  "fieldName":"UL_LON",
                  "dictionaryLink":"",
                  "value":"-102.48663"
               },
               {
                  "id":"5e83d11959773042",
                  "fieldName":"UR_LAT",
                  "dictionaryLink":"",
                  "value":"37.97157"
               },
               {
                  "id":"5e83d1191e09dd34",
                  "fieldName":"Sensor Mode",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sensor_mode",
                  "value":"BUMPER"
               },
               {
                  "id":"5e83d119f3f29818",
                  "fieldName":"Sensor Mode",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sensor_mode",
                  "value":"BUMPER"
               },
               {
                  "id":"5e83d11967e7be31",
                  "fieldName":"WRS Path",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#wrs_path",
                  "value":" 031"
               },
               {
                  "id":"5e83d11934daa06d",
                  "fieldName":"WRS Path",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#wrs_path",
                  "value":" 031"
               },
               {
                  "id":"5e83d119dfa54533",
                  "fieldName":"UR_LON",
                  "dictionaryLink":"",
                  "value":"-100.36707"
               },
               {
                  "id":"5e83d119d19269d9",
                  "fieldName":"LR_LAT",
                  "dictionaryLink":"",
                  "value":"37.97157"
               },
               {
                  "id":"5e83d119d93462c",
                  "fieldName":"WRS Row",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#wrs_row",
                  "value":" 033"
               },
               {
                  "id":"5e83d119175a9f85",
                  "fieldName":"WRS Row",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#wrs_row",
                  "value":" 033"
               },
               {
                  "id":"5e83d119a2f770bb",
                  "fieldName":"LR_LON",
                  "dictionaryLink":"",
                  "value":"-100.87561"
               },
               {
                  "id":"5e83d1193eb1eda1",
                  "fieldName":"Full Partial Scene",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#full_partial_scene",
                  "value":"FULL"
               },
               {
                  "id":"5e83d119b5031133",
                  "fieldName":"Full Partial Scene",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#full_partial_scene",
                  "value":"FULL"
               },
               {
                  "id":"5e83d1199d010929",
                  "fieldName":"Date Product Generated L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#date_l2_generated",
                  "value":"2020/08/20"
               },
               {
                  "id":"5e83d11996b0840e",
                  "fieldName":"Date Product Generated L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#date_l2_generated",
                  "value":"2020/08/20"
               },
               {
                  "id":"5e83d119524c32e",
                  "fieldName":"LL_LAT",
                  "dictionaryLink":"",
                  "value":"38.27370"
               },
               {
                  "id":"5e83d119aa12a17b",
                  "fieldName":"LL_LON",
                  "dictionaryLink":"",
                  "value":"-102.95058"
               },
               {
                  "id":"5e83d119c4c93633",
                  "fieldName":"Date Product Generated L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#date_l1_generated",
                  "value":"2020/08/20"
               },
               {
                  "id":"5e83d11925f4d7b1",
                  "fieldName":"Date Product Generated L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#date_l1_generated",
                  "value":"2020/08/20"
               },
               {
                  "id":"5e83d1198198937",
                  "fieldName":"Start Time",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#start_time",
                  "value":"2012-05-04 17:09:05.19431"
               },
               {
                  "id":"5e83d119799cfc69",
                  "fieldName":"Start Time",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#start_time",
                  "value":"2012-05-04 17:09:05.19431"
               },
               {
                  "id":"5e83d119c231dfdb",
                  "fieldName":"Stop Time",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#stop_time",
                  "value":"2012-05-04 17:09:31.80713"
               },
               {
                  "id":"5e83d1197f85f30",
                  "fieldName":"Stop Time",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#stop_time",
                  "value":"2012-05-04 17:09:31.80713"
               },
               {
                  "id":"5e83d1192d8d752e",
                  "fieldName":"Station Identifier",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#distribution_site",
                  "value":"EDC"
               },
               {
                  "id":"5e83d119e814b3d5",
                  "fieldName":"Station Identifier",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#distribution_site",
                  "value":"EDC"
               },
               {
                  "id":"5e83d119a6b422f1",
                  "fieldName":"Day/Night Indicator",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#day_or_night",
                  "value":"DAY"
               },
               {
                  "id":"5e83d119d580c676",
                  "fieldName":"Day/Night Indicator",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#day_or_night",
                  "value":"DAY"
               },
               {
                  "id":"5e83d1191dbc4bb2",
                  "fieldName":"Land Cloud Cover",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#cloud_cover_land",
                  "value":"   0.00"
               },
               {
                  "id":"5e83d119241ce3ad",
                  "fieldName":"Land Cloud Cover",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#cloud_cover_land",
                  "value":"   0.00"
               },
               {
                  "id":"5e83d1192b06440a",
                  "fieldName":"Scene Cloud Cover L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#cloud_cover",
                  "value":"   0.00"
               },
               {
                  "id":"5e83d119c9649ed5",
                  "fieldName":"Scene Cloud Cover L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#cloud_cover",
                  "value":"   0.00"
               },
               {
                  "id":"5e83d1193835ad13",
                  "fieldName":"Ground Control Points Model",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ground_control_points_model",
                  "value":147
               },
               {
                  "id":"5e83d119f145b805",
                  "fieldName":"Ground Control Points Model",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ground_control_points_model",
                  "value":147
               },
               {
                  "id":"5e83d119c72d835f",
                  "fieldName":"Ground Control Points Version",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ground_control_points_version",
                  "value":5
               },
               {
                  "id":"5e83d1195d9e6973",
                  "fieldName":"Ground Control Points Version",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ground_control_points_version",
                  "value":5
               },
               {
                  "id":"5e83d119369526c2",
                  "fieldName":"Geometric RMSE Model",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#geometric_rmse_model",
                  "value":"    5.018"
               },
               {
                  "id":"5e83d1199b894cc6",
                  "fieldName":"Geometric RMSE Model",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#geometric_rmse_model",
                  "value":"    5.018"
               },
               {
                  "id":"5e83d119eb4a52b2",
                  "fieldName":"Geometric RMSE Model X",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#geometric_rmse_model_x",
                  "value":"    3.208"
               },
               {
                  "id":"5e83d119e8b2a364",
                  "fieldName":"Geometric RMSE Model X",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#geometric_rmse_model_x",
                  "value":"    3.208"
               },
               {
                  "id":"5e83d119b3959f54",
                  "fieldName":"Geometric RMSE Model Y",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#geometric_rmse_model_y   ",
                  "value":"    3.858"
               },
               {
                  "id":"5e83d119545197e5",
                  "fieldName":"Geometric RMSE Model Y",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#geometric_rmse_model_y",
                  "value":"    3.858"
               },
               {
                  "id":"5e83d119c6488aea",
                  "fieldName":"Sensor Anomalies",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sensor_anomalies",
                  "value":"NONE"
               },
               {
                  "id":"5e83d11915d08447",
                  "fieldName":"Sensor Anomalies",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sensor_anomalies",
                  "value":"NONE"
               },
               {
                  "id":"5e83d119a837407a",
                  "fieldName":"Data Anomaly",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#data_anomaly",
                  "value":"None"
               },
               {
                  "id":"5e83d119907f9709",
                  "fieldName":"Data Anomaly",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#data_anomaly",
                  "value":"None"
               },
               {
                  "id":"5e83d119da3199a5",
                  "fieldName":"Image Quality",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#image_quality_f1",
                  "value":9
               },
               {
                  "id":"5e83d119c9703e76",
                  "fieldName":"Image Quality",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#image_quality_f1",
                  "value":9
               },
               {
                  "id":"5e83d119ed82d34a",
                  "fieldName":"Gain Change",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#gain_change_band_1",
                  "value":"N"
               },
               {
                  "id":"5e83d119160507a9",
                  "fieldName":"Gain Change",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#gain_change_band_1",
                  "value":"N"
               },
               {
                  "id":"5e83d11961e2e91f",
                  "fieldName":"Processing Software Version",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#processing_software_version",
                  "value":"LPGS_15.3.1c"
               },
               {
                  "id":"5e83d119e682a749",
                  "fieldName":"Processing Software Version",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#processing_software_version",
                  "value":"LPGS_15.3.1c"
               },
               {
                  "id":"5e83d119d68c3c56",
                  "fieldName":"Sun Elevation L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sun_elevation",
                  "value":" 59.37310352"
               },
               {
                  "id":"5e83d119b4f2499a",
                  "fieldName":"Sun Elevation L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sun_elevation",
                  "value":" 59.37310352"
               },
               {
                  "id":"5e83d11974719513",
                  "fieldName":"Sun Azimuth L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sun_azimuth",
                  "value":" 131.19738443"
               },
               {
                  "id":"5e83d1196bd1a17e",
                  "fieldName":"Sun Azimuth L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sun_azimuth",
                  "value":" 131.19738443"
               },
               {
                  "id":"5e83d119bd6f280e",
                  "fieldName":"Data Type L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#data_type_21",
                  "value":"TM_L2SP"
               },
               {
                  "id":"5e83d119e43f7db9",
                  "fieldName":"Data Type L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#data_type_21",
                  "value":"TM_L2SP"
               },
               {
                  "id":"5e83d119709b05d5",
                  "fieldName":"Sensor Identifier",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sensor_id",
                  "value":"TM"
               },
               {
                  "id":"5e83d11922c49edf",
                  "fieldName":"Satellite",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#satellite",
                  "value":"LANDSAT_5"
               },
               {
                  "id":"5e83d11943388776",
                  "fieldName":"Sensor Identifier",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sensor_id",
                  "value":"TM"
               },
               {
                  "id":"5e83d1194ce8c25",
                  "fieldName":"Ephemeris Type",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ephemeris_type",
                  "value":"DEFINITIVE"
               },
               {
                  "id":"5e83d119620279fa",
                  "fieldName":"Satellite",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#satellite",
                  "value":5
               },
               {
                  "id":"5e83d119c93da2ae",
                  "fieldName":"Product Map Projection L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#map_projection_l2",
                  "value":"UTM"
               },
               {
                  "id":"5e83d11994227806",
                  "fieldName":"Ephemeris Type",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ephemeris_type",
                  "value":"DEFINITIVE"
               },
               {
                  "id":"5e83d11920d22995",
                  "fieldName":"UTM Zone",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#utm_zone",
                  "value":14
               },
               {
                  "id":"5e83d119dfc32fd1",
                  "fieldName":"Datum",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#datum",
                  "value":"WGS84"
               },
               {
                  "id":"5e83d119889cd954",
                  "fieldName":"Ellipsoid",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ellipsoid",
                  "value":"WGS84"
               },
               {
                  "id":"5e83d119629796c8",
                  "fieldName":"Product Map Projection L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#map_projection_l2",
                  "value":"UTM"
               },
               {
                  "id":"5e83d119fadd1071",
                  "fieldName":"UTM Zone",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#utm_zone",
                  "value":14
               },
               {
                  "id":"5e83d119388164c",
                  "fieldName":"Datum",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#datum",
                  "value":"WGS84"
               },
               {
                  "id":"5e83d119e4ff9290",
                  "fieldName":"Ellipsoid",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ellipsoid",
                  "value":"WGS84"
               },
               {
                  "id":"5e83d1193b2598ef",
                  "fieldName":"Scene Center Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"38&deg;54\\'55.94\"N"
               },
               {
                  "id":"5e83d119cf9cdcd0",
                  "fieldName":"Scene Center Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"101&deg;40\\'11.28\"W"
               },
               {
                  "id":"5e83d1197f20b27e",
                  "fieldName":"Corner Upper Left Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"39&deg;51\\'10.76\"N"
               },
               {
                  "id":"5e83d11936b6b01f",
                  "fieldName":"Corner Upper Left Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"102&deg;29\\'11.87\"W"
               },
               {
                  "id":"5e83d11961cccb4e",
                  "fieldName":"Corner Upper Right Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"39&deg;32\\'39.26\"N"
               },
               {
                  "id":"5e83d119914affb9",
                  "fieldName":"Corner Upper Right Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"100&deg;22\\'01.45\"W"
               },
               {
                  "id":"5e83d11933e075b3",
                  "fieldName":"Scene Center Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"38&deg;54\\'55.94\"N"
               },
               {
                  "id":"5e83d11929b7c342",
                  "fieldName":"Corner Lower Left Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"38&deg;16\\'25.32\"N"
               },
               {
                  "id":"5e83d11915ccea1b",
                  "fieldName":"Originator",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d119d03bef11",
                  "fieldName":"Scene Center Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"101&deg;40\\'11.28\"W"
               },
               {
                  "id":"5e83d119b27f36cd",
                  "fieldName":"Corner Upper Left Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"39&deg;51\\'10.76\"N"
               },
               {
                  "id":"5e83d119e5a5bc5a",
                  "fieldName":"Corner Lower Left Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"102&deg;57\\'02.09\"W"
               },
               {
                  "id":"5e83d119ae3d36ca",
                  "fieldName":"Corner Lower Right Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"37&deg;58\\'17.65\"N"
               },
               {
                  "id":"5e83d1197695fcdf",
                  "fieldName":"Corner Upper Left Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"102&deg;29\\'11.87\"W"
               },
               {
                  "id":"5e83d119ea4acbd4",
                  "fieldName":"Corner Upper Right Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"39&deg;32\\'39.26\"N"
               },
               {
                  "id":"5e83d119adad6c55",
                  "fieldName":"Corner Lower Right Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"100&deg;52\\'32.20\"W"
               },
               {
                  "id":"5e83d119f6d182f7",
                  "fieldName":"Corner Upper Right Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"100&deg;22\\'01.45\"W"
               },
               {
                  "id":"5e83d1197cfbeb7a",
                  "fieldName":"Scene Center Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 38.91554"
               },
               {
                  "id":"5e83d119dc6d299d",
                  "fieldName":"Scene Center Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-101.66980"
               },
               {
                  "id":"5e83d11932520ee0",
                  "fieldName":"Corner Lower Left Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"38&deg;16\\'25.32\"N"
               },
               {
                  "id":"5e83d1199113b057",
                  "fieldName":"Corner Lower Left Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"102&deg;57\\'02.09\"W"
               },
               {
                  "id":"5e83d11960e9d77a",
                  "fieldName":"Corner Upper Left Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 39.85299"
               },
               {
                  "id":"5e83d119595283a3",
                  "fieldName":"Corner Upper Left Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-102.48663"
               },
               {
                  "id":"5e83d1198025a16",
                  "fieldName":"Corner Lower Right Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"37&deg;58\\'17.65\"N"
               },
               {
                  "id":"5e83d119bac5c53b",
                  "fieldName":"Corner Lower Right Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"100&deg;52\\'32.20\"W"
               },
               {
                  "id":"5e83d119539410e8",
                  "fieldName":"Corner Upper Right Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 39.54424"
               },
               {
                  "id":"5e83d119189a44a8",
                  "fieldName":"Corner Upper Right Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-100.36707"
               },
               {
                  "id":"5e83d119e58ff41",
                  "fieldName":"Scene Center Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 38.91554"
               },
               {
                  "id":"5e83d119ac572a41",
                  "fieldName":"Publication_Date",
                  "dictionaryLink":"",
                  "value":"20050831"
               },
               {
                  "id":"5e83d119b97778e3",
                  "fieldName":"Corner Lower Left Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 38.27370"
               },
               {
                  "id":"5e83d119fc691855",
                  "fieldName":"Scene Center Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-101.66980"
               },
               {
                  "id":"5e83d119d033de1",
                  "fieldName":"Corner Lower Left Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-102.95058"
               },
               {
                  "id":"5e83d119b84d4d52",
                  "fieldName":"Corner Upper Left Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 39.85299"
               },
               {
                  "id":"5e83d119aef3d62e",
                  "fieldName":"Corner Lower Right Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 37.97157"
               },
               {
                  "id":"5e83d1196f42a17a",
                  "fieldName":"Corner Upper Left Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-102.48663"
               },
               {
                  "id":"5e83d119a7e81d55",
                  "fieldName":"Corner Upper Right Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 39.54424"
               },
               {
                  "id":"5e83d11967541ce6",
                  "fieldName":"Corner Lower Right Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-100.87561"
               },
               {
                  "id":"5e83d119a9ebd21b",
                  "fieldName":"Corner Upper Right Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-100.36707"
               },
               {
                  "id":"5e83d1199d5244f6",
                  "fieldName":"Corner Lower Left Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 38.27370"
               },
               {
                  "id":"5e83d119799824a8",
                  "fieldName":"Corner Lower Left Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-102.95058"
               },
               {
                  "id":"5e83d11917097daf",
                  "fieldName":"Corner Lower Right Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 37.97157"
               },
               {
                  "id":"5e83d119d6ce5eaf",
                  "fieldName":"Corner Lower Right Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-100.87561"
               },
               {
                  "id":"5e83d11913a82746",
                  "fieldName":"Title",
                  "dictionaryLink":"",
                  "value":"LANDSAT TM Collection 2 - Path:  31 Row:  33 for Scene:  LT05_L2SP_031033_20120504_20200820_02_T1"
               },
               {
                  "id":"5e83d11972d6e9f7",
                  "fieldName":"Geospatial_Data_Presentation_Form",
                  "dictionaryLink":"",
                  "value":"Remote-Sensing Image"
               },
               {
                  "id":"5e83d119920813bc",
                  "fieldName":"Publication_Place",
                  "dictionaryLink":"",
                  "value":"Sioux Falls, SD USA"
               },
               {
                  "id":"5e83d119df3fe2e7",
                  "fieldName":"Publisher",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d119d2a6e1c7",
                  "fieldName":"Online_Linkage",
                  "dictionaryLink":"",
                  "value":"https://doi.org/10.5066/P9IAXOVV"
               },
               {
                  "id":"5e83d1196aefe139",
                  "fieldName":"Abstract",
                  "dictionaryLink":"",
                  "value":"The USGS Earth Resources Observation and Science (EROS) Center archive holds data collected by the Landsat suite of satellites, beginning with Landsat 1 in 1972 through the latest Landsat Mission."
               },
               {
                  "id":"5e83d119684bd37c",
                  "fieldName":"Purpose",
                  "dictionaryLink":"",
                  "value":"The Landsat archive provides a rich collection of information about the Earth's land surface.  Major characteristics of changes to the surface of the planet can be detected, measured, and analyzed using Landsat data.  The information obtainable from the historical and current Landsat data play a key role in studying surface changes through time.  Collection 2 provides a quality-controlled selection of radiometrically calibrated and consistently geolocated data; particular geodetic accuracy and error tolerances are specified to align pixels for time series analysis through the full Landsat record."
               },
               {
                  "id":"5e83d11945f920c",
                  "fieldName":"Supplemental_Information",
                  "dictionaryLink":"",
                  "value":"Landsat represents the world's longest continuously acquired collection of space-based moderate-resolution land remote sensing data.  As a joint initiative between the U.S. Geological Survey (USGS) and the National Aeronautical Space Administration (NASA), the Landsat Project and the data it collects support government, commercial, industrial, civilian, military, and educational communities throughout the United States and world.  Landsat 8 Operational Land Imager (OLI)/Thermal Infrared Sensor (TIRS) Collection 2 provides a quality-controlled selection of radiometrically calibrated and consistently geolocated data; particular geodetic accuracy and error tolerances are specified to align pixels for time series analysis through the full Landsat record. (https://www.usgs.gov/core-science-systems/nli/landsat/landsat-collections)"
               },
               {
                  "id":"5e83d1197cc754b4",
                  "fieldName":"Calendar_Date",
                  "dictionaryLink":"",
                  "value":"19920704"
               },
               {
                  "id":"5e83d11966963850",
                  "fieldName":"Currentness_Reference",
                  "dictionaryLink":"",
                  "value":"Ground Condition"
               },
               {
                  "id":"5e83d119fc177449",
                  "fieldName":"Progress",
                  "dictionaryLink":"",
                  "value":"In Work"
               },
               {
                  "id":"5e83d11960ba536d",
                  "fieldName":"Maintenance_and_Update_Frequency",
                  "dictionaryLink":"",
                  "value":"Continually"
               },
               {
                  "id":"5e83d119586f0830",
                  "fieldName":"G-Ring_Latitude",
                  "dictionaryLink":"",
                  "value":"38.27370"
               },
               {
                  "id":"5e83d1191be10c7c",
                  "fieldName":"G-Ring_Longitude",
                  "dictionaryLink":"",
                  "value":"-102.95058"
               },
               {
                  "id":"5e83d119281579e6",
                  "fieldName":"Theme_Keyword_Thesaurus",
                  "dictionaryLink":"",
                  "value":"ISO 19115 Topic Category"
               },
               {
                  "id":"5e83d1194abc14dd",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"ImageryBaseMapsEarthCover"
               },
               {
                  "id":"60470441398df5b",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Remote Sensing"
               },
               {
                  "id":"604704413623c358",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Collection 2"
               },
               {
                  "id":"60470441c22543ba",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Infrared Imagery"
               },
               {
                  "id":"604704413ae1d393",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Imagery"
               },
               {
                  "id":"60470441a5f6eec9",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Visible Imagery"
               },
               {
                  "id":"6047044176fd6e3d",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Visible Wavelengths"
               },
               {
                  "id":"604704417339664e",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Radiance"
               },
               {
                  "id":"60470441fb324cd2",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Landsat"
               },
               {
                  "id":"60470441ef8940aa",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Satellite"
               },
               {
                  "id":"60470441fd132ec0",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) "
               },
               {
                  "id":"6047044187bc92ee",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Thematic Mapper (TM)"
               },
               {
                  "id":"6047044159f1e540",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"60470441627f211c",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Infrared Wavelengths"
               },
               {
                  "id":"5e83d119796d51f5",
                  "fieldName":"Place_Keyword_Thesaurus",
                  "dictionaryLink":"",
                  "value":"U.S. Department of Commerce, 1995, Countries, dependencies, areas of special sovereignty, and their principal administrative divisions, Federal Information Processing Standard 10-4, Washington, D.C., National Institute of Standards and Technology"
               },
               {
                  "id":"5e83d119abd5525a",
                  "fieldName":"Place_Keyword",
                  "dictionaryLink":"",
                  "value":"Land Surface"
               },
               {
                  "id":"5e83d119ba008a5",
                  "fieldName":"Access_Constraints",
                  "dictionaryLink":"",
                  "value":"Downloading and using these data signifies that the user comprehends and will comply with the USGS Privacy Policy.  Reading and understanding all portions of the metadata will protect the interests of both the user and the USGS. Please refer to https://www.usgs.gov/privacy.html."
               },
               {
                  "id":"5e83d11a5fe32ed8",
                  "fieldName":"Use_Constraints",
                  "dictionaryLink":"",
                  "value":"There is no guarantee of warranty concerning the accuracy of these data. Users should not use these data for critical applications without a full awareness of their limitations. Temporal changes may have occurred since the data were collected and some parts of these data may no longer represent actual surface conditions.  There are no restrictions regarding the use of Landsat data distributed by the U.S. Geological Survey (USGS), unless expressly identified prior to or at the time of receipt. For Landsat, we request the following statement be used: Data available from the U.S. Geological Survey. Please refer to https://www.usgs.gov/centers/eros/data-citation for more information on data citation."
               },
               {
                  "id":"5e83d11ab9c2ebac",
                  "fieldName":"Contact_Person",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11ab32a4f35",
                  "fieldName":"Contact_Organization",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d11ab61696bc",
                  "fieldName":"Contact_Position",
                  "dictionaryLink":"",
                  "value":"Customer Service Representative"
               },
               {
                  "id":"5e83d11ad3056599",
                  "fieldName":"Address_Type",
                  "dictionaryLink":"",
                  "value":"Mailing and Physical Address"
               },
               {
                  "id":"5e83d11a257fe85d",
                  "fieldName":"Address",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center 47914 252nd Street"
               },
               {
                  "id":"5e83d11ab3eacc95",
                  "fieldName":"City",
                  "dictionaryLink":"",
                  "value":"Sioux Falls"
               },
               {
                  "id":"5e83d11aa1694de",
                  "fieldName":"State_or_Province",
                  "dictionaryLink":"",
                  "value":"SD"
               },
               {
                  "id":"5e83d11ad2c2fb60",
                  "fieldName":"Postal_Code",
                  "dictionaryLink":"",
                  "value":"57198-0001"
               },
               {
                  "id":"5e83d11ae22dc5e3",
                  "fieldName":"Country",
                  "dictionaryLink":"",
                  "value":"USA"
               },
               {
                  "id":"5e83d11aa9f27cf6",
                  "fieldName":"Contact_Electronic_Mail_Address",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a540dcfe9",
                  "fieldName":"Hours_of_Service",
                  "dictionaryLink":"",
                  "value":"0800 - 1600 CT, M-F, 1400 - 2200 h GMT"
               },
               {
                  "id":"5e83d11ac32f724c",
                  "fieldName":"Contact_Instructions",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a5c48c7bc",
                  "fieldName":"Browse_Graphic_File_Name",
                  "dictionaryLink":"",
                  "value":"Not Available"
               },
               {
                  "id":"5e83d11ad8675c1d",
                  "fieldName":"Browse_Graphic_File_Description",
                  "dictionaryLink":"",
                  "value":"This is a compressed and resampled version of the Landsat image to help validate area of interest, data quality, and the impact of cloud cover, if it exists."
               },
               {
                  "id":"5e83d11ae71a3055",
                  "fieldName":"Browse_Graphic_File_Type",
                  "dictionaryLink":"",
                  "value":"JPEG"
               },
               {
                  "id":"5e83d11a9000b8ec",
                  "fieldName":"Security_Classification_System",
                  "dictionaryLink":"",
                  "value":"None"
               },
               {
                  "id":"5e83d11aa39ac7de",
                  "fieldName":"Security_Classification",
                  "dictionaryLink":"",
                  "value":"Unclassified"
               },
               {
                  "id":"5e83d11a1883ab2b",
                  "fieldName":"Security_Handling_Description",
                  "dictionaryLink":"",
                  "value":"None"
               },
               {
                  "id":"5e83d11a7a5e7378",
                  "fieldName":"Native_Data_Set_Environment",
                  "dictionaryLink":"",
                  "value":"Oracle"
               },
               {
                  "id":"5e83d11a53641db4",
                  "fieldName":"Originator",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d11a23ff726c",
                  "fieldName":"Publication_Date",
                  "dictionaryLink":"",
                  "value":"19720723"
               },
               {
                  "id":"5e83d11a3ae40d19",
                  "fieldName":"Title",
                  "dictionaryLink":"",
                  "value":"Landsat Thematic Mapper Collection 2"
               },
               {
                  "id":"5e83d11aa866af3f",
                  "fieldName":"Geospatial_Data_Presentation_Form",
                  "dictionaryLink":"",
                  "value":"Remote-Sensing Image"
               },
               {
                  "id":"5e83d11a3ff08366",
                  "fieldName":"Publication_Place",
                  "dictionaryLink":"",
                  "value":"Sioux Falls,  SD USA"
               },
               {
                  "id":"5e83d11ac764e8b1",
                  "fieldName":"Publisher",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d11af27a7036",
                  "fieldName":"Online_Linkage",
                  "dictionaryLink":"",
                  "value":"https://doi.org/10.5066/P9IAXOVV"
               },
               {
                  "id":"5e83d11a393fe841",
                  "fieldName":"Attribute_Accuracy_Report",
                  "dictionaryLink":"",
                  "value":"The identification of features is provided by the spectral signature which is the distinct electromagnetic energy it emits, reflects, or otherwise transmits.  Other signatures such as tone (lightness or darkness), texture (surface roughness or smoothness), pattern, shadow, shape, and size are just as important.  Thus, through the use of Landsat data, such elements as water features, soils, and vegetation can be identified and distinguished from each other."
               },
               {
                  "id":"5e83d11abfffd008",
                  "fieldName":"Logical_Consistency_Report",
                  "dictionaryLink":"",
                  "value":"N/A"
               },
               {
                  "id":"5e83d11a3a7b431a",
                  "fieldName":"Completeness_Report",
                  "dictionaryLink":"",
                  "value":"The Landsat satellites circle the Earth on a sun-synchronous, near-polar orbit imaging the same 185 km (115 miles) ground swath every 16 days."
               },
               {
                  "id":"5e83d11adfb52c64",
                  "fieldName":"Process_Description",
                  "dictionaryLink":"",
                  "value":"Landsat Collection 2 Level-2 surface reflectance and surface temperature science products are generated from Collection 2 Level-1 inputs that meet the less than 76 degrees Solar Zenith Angle constraint and include the required auxiliary data inputs to generate a scientifically viable product. For more information visit https://www.usgs.gov/core-science-systems/nli/landsat/landsat-collection-2-level-2-science-products."
               },
               {
                  "id":"5e83d11a886ef0cd",
                  "fieldName":"Process_Date",
                  "dictionaryLink":"",
                  "value":"19920704"
               },
               {
                  "id":"5e83d11a11da2ce6",
                  "fieldName":"Contact_Person",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11ab064168",
                  "fieldName":"Contact_Organization",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d11a2304f959",
                  "fieldName":"Contact_Organization",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d11a3bbcd0f2",
                  "fieldName":"Contact_Person",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a9e6d5dd6",
                  "fieldName":"Contact_Position",
                  "dictionaryLink":"",
                  "value":"Customer Service Representative"
               },
               {
                  "id":"5e83d11adff5e984",
                  "fieldName":"Address_Type",
                  "dictionaryLink":"",
                  "value":"Mailing and Physical Address"
               },
               {
                  "id":"5e83d11af63fd80e",
                  "fieldName":"Address",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center 47914 252nd Street"
               },
               {
                  "id":"5e83d11afb799f4a",
                  "fieldName":"City",
                  "dictionaryLink":"",
                  "value":"Sioux Falls"
               },
               {
                  "id":"5e83d11a44b6f7fa",
                  "fieldName":"State_or_Province",
                  "dictionaryLink":"",
                  "value":"SD"
               },
               {
                  "id":"5e83d11a7fc5e25c",
                  "fieldName":"Postal_Code",
                  "dictionaryLink":"",
                  "value":"57198-0001"
               },
               {
                  "id":"5e83d11a85d3d795",
                  "fieldName":"Country",
                  "dictionaryLink":"",
                  "value":"USA"
               },
               {
                  "id":"5e83d11ae7dc9222",
                  "fieldName":"Contact_Electronic_Mail_Address",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11aeb5a94ea",
                  "fieldName":"Hours_of_Service",
                  "dictionaryLink":"",
                  "value":"0800 - 1600 CT, M-F, 1400 - 2200 h GMT"
               },
               {
                  "id":"5e83d11ac055787a",
                  "fieldName":"Contact_Instructions",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a858097e6",
                  "fieldName":"Cloud_Cover",
                  "dictionaryLink":"",
                  "value":"0.000 percent"
               },
               {
                  "id":"5e83d11af1f2843b",
                  "fieldName":"Indirect_Spatial_Reference",
                  "dictionaryLink":"",
                  "value":"Landsats 4 and 5 collected data at 30-meter ground resolution. Band 6 was acquired at 120-meter resolution, but products are resampled to 30-meter pixels."
               },
               {
                  "id":"5e83d11a6b630fc9",
                  "fieldName":"Direct_Spatial_Reference_Method",
                  "dictionaryLink":"",
                  "value":"Raster"
               },
               {
                  "id":"5e83d11a19dda88e",
                  "fieldName":"Raster_Object_Type",
                  "dictionaryLink":"",
                  "value":"Pixel"
               },
               {
                  "id":"5e83d11ac4295b28",
                  "fieldName":"Contact_Person",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a883a82be",
                  "fieldName":"Contact_Organization",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d11aa774b26b",
                  "fieldName":"Contact_Position",
                  "dictionaryLink":"",
                  "value":"Customer Service Representative"
               },
               {
                  "id":"5e83d11ad2958365",
                  "fieldName":"Address_Type",
                  "dictionaryLink":"",
                  "value":"Mailing and Physical Address"
               },
               {
                  "id":"5e83d11af5fa6ab8",
                  "fieldName":"Address",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center 47914 252nd Street"
               },
               {
                  "id":"5e83d11a60757afb",
                  "fieldName":"City",
                  "dictionaryLink":"",
                  "value":"Sioux Falls"
               },
               {
                  "id":"5e83d11a7d7342b1",
                  "fieldName":"State_or_Province",
                  "dictionaryLink":"",
                  "value":"SD"
               },
               {
                  "id":"5e83d11a7c7a009c",
                  "fieldName":"Postal_Code",
                  "dictionaryLink":"",
                  "value":"57198-0001"
               },
               {
                  "id":"5e83d11a6085f9c4",
                  "fieldName":"Country",
                  "dictionaryLink":"",
                  "value":"USA"
               },
               {
                  "id":"5e83d11a8414a2f7",
                  "fieldName":"Contact_Electronic_Mail_Address",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11afd59eb5f",
                  "fieldName":"Hours_of_Service",
                  "dictionaryLink":"",
                  "value":"0800 - 1600 CT, M-F, 1400 - 2200 h GMT"
               },
               {
                  "id":"5e83d11a95f467a9",
                  "fieldName":"Contact_Instructions",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11abaff6573",
                  "fieldName":"Resource_Description",
                  "dictionaryLink":"",
                  "value":"LANDSAT TM Collection 2 Entity ID: LT50310332012125EDC00"
               },
               {
                  "id":"5e83d11a92c457e3",
                  "fieldName":"Distribution_Liability",
                  "dictionaryLink":"",
                  "value":"Although these data have been processed successfully on a computer system at the USGS, no warranty expressed or implied is made by the USGS regarding the use of the data on any other system, nor does the act of distribution constitute any such warranty."
               },
               {
                  "id":"5e83d11af5c0878d",
                  "fieldName":"Format_Name",
                  "dictionaryLink":"",
                  "value":"Geographcal-Tagged Image File Format (GeoTIFF), or Joint Photographic Experts Group (JPEG)"
               },
               {
                  "id":"5e83d11ae0bb7b8f",
                  "fieldName":"Network_Resource_Name",
                  "dictionaryLink":"",
                  "value":"https://earthexplorer.usgs.gov"
               },
               {
                  "id":"5e83d11afb83c4cd",
                  "fieldName":"Fees",
                  "dictionaryLink":"",
                  "value":"Fees:  All Landsat data held in the USGS EROS archive are downloadable at no cost to all users."
               },
               {
                  "id":"5e83d11abe18accc",
                  "fieldName":"Technical_Prerequisites",
                  "dictionaryLink":"",
                  "value":"Adequate computer capability is the only technical prerequisite for viewing data in digital form.\r\n\r\n"
               },
               {
                  "id":"5e83d11a1a5caead",
                  "fieldName":"Beginning_Date",
                  "dictionaryLink":"",
                  "value":"19821113"
               },
               {
                  "id":"5e83d11a7ab01984",
                  "fieldName":"Ending_Date",
                  "dictionaryLink":"",
                  "value":"20011001"
               },
               {
                  "id":"5e83d11a5fe31dbe",
                  "fieldName":"Metadata_Date",
                  "dictionaryLink":"",
                  "value":"20160422"
               },
               {
                  "id":"5e83d11ace8c4be6",
                  "fieldName":"Metadata_Review_Date",
                  "dictionaryLink":"",
                  "value":"As Needed"
               },
               {
                  "id":"5e83d11aaf7b1a8",
                  "fieldName":"Contact_Person",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a6fe629a0",
                  "fieldName":"Contact_Organization",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d11a612f9dca",
                  "fieldName":"Contact_Position",
                  "dictionaryLink":"",
                  "value":"Customer Service Representative"
               },
               {
                  "id":"5e83d11ac64398d4",
                  "fieldName":"Address_Type",
                  "dictionaryLink":"",
                  "value":"Mailing and Physical Address"
               },
               {
                  "id":"5e83d11a2ec3653a",
                  "fieldName":"Address",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center 47914 252nd Street"
               },
               {
                  "id":"5e83d11a9dc347a2",
                  "fieldName":"City",
                  "dictionaryLink":"",
                  "value":"Sioux Falls"
               },
               {
                  "id":"5e83d11a557d59a2",
                  "fieldName":"State_or_Province",
                  "dictionaryLink":"",
                  "value":"SD"
               },
               {
                  "id":"5e83d11ab5408135",
                  "fieldName":"Postal_Code",
                  "dictionaryLink":"",
                  "value":"57198-0001"
               },
               {
                  "id":"5e83d11a12675aa1",
                  "fieldName":"Country",
                  "dictionaryLink":"",
                  "value":"USA"
               },
               {
                  "id":"5e83d11a44f7d71b",
                  "fieldName":"Contact_Electronic_Mail_Address",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a72d65a0f",
                  "fieldName":"Hours_of_Service",
                  "dictionaryLink":"",
                  "value":"0800 - 1600 CT, M-F, 1400 - 2200 h GMT"
               },
               {
                  "id":"5e83d11ac62ff739",
                  "fieldName":"Contact_Instructions",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a2b75b6e1",
                  "fieldName":"Metadata_Standard_Name",
                  "dictionaryLink":"",
                  "value":"Content Standard for Digital Geospatial Metadata"
               },
               {
                  "id":"5e83d11a7c7a0451",
                  "fieldName":"Metadata_Standard_Version",
                  "dictionaryLink":"",
                  "value":"FGDC-STD-001-1998, Version 2"
               },
               {
                  "id":"5e83d11a1ab1600f",
                  "fieldName":"Metadata_Time_Convention",
                  "dictionaryLink":"",
                  "value":"Local Time"
               },
               {
                  "id":"5e83d11a6418b4e",
                  "fieldName":"Metadata_Access_Constraints",
                  "dictionaryLink":"",
                  "value":"None"
               },
               {
                  "id":"5e83d11a37751741",
                  "fieldName":"Metadata_Use_Constraints",
                  "dictionaryLink":"",
                  "value":"None"
               },
               {
                  "id":"5e83d11a6948e4a6",
                  "fieldName":"Metadata_Security_Classification_System",
                  "dictionaryLink":"",
                  "value":"None"
               },
               {
                  "id":"5e83d11a96bf92d9",
                  "fieldName":"Metadata_Security_Classification",
                  "dictionaryLink":"",
                  "value":"Unclassified"
               },
               {
                  "id":"5e83d11ae14acc9d",
                  "fieldName":"Metadata_Security_Handling_Description",
                  "dictionaryLink":"",
                  "value":"None"
               }
            ],
            "hasCustomizedMetadata":"None",
            "options":{
               "bulk":True,
               "download":True,
               "order":False,
               "secondary":False
            },
            "selected":{
               "bulk":False,
               "compare":False,
               "order":False
            },
            "spatialBounds":{
               "type":"Polygon",
               "coordinates":[
                  [
                     [
                        -102.95058,
                        37.97157
                     ],
                     [
                        -102.95058,
                        39.85299
                     ],
                     [
                        -100.36707,
                        39.85299
                     ],
                     [
                        -100.36707,
                        37.97157
                     ],
                     [
                        -102.95058,
                        37.97157
                     ]
                  ]
               ]
            },
            "spatialCoverage":{
               "type":"Polygon",
               "coordinates":[
                  [
                     [
                        -102.95058,
                        38.2737
                     ],
                     [
                        -100.87561,
                        37.97157
                     ],
                     [
                        -100.36707,
                        39.54424
                     ],
                     [
                        -102.48663,
                        39.85299
                     ],
                     [
                        -102.95058,
                        38.2737
                     ]
                  ]
               ]
            },
            "temporalCoverage":{
               "endDate":"2012-05-04 00:00:00",
               "startDate":"2012-05-04 00:00:00"
            },
            "publishDate":"2022-06-22 19:23:13-05"
         },
         {
            "browse":[
               {
                  "id":"5fb4b9d0e5094d61",
                  "browseRotationEnabled":"None",
                  "browseName":"Level-1 Reflective Browse",
                  "browsePath":"https://landsatlook.usgs.gov/gen-browse?size=rrb&type=refl&product_id=LT05_L1TP_031034_20120504_20200820_02_T1",
                  "overlayPath":"https://landsatlook.usgs.gov/dynamic-tiler/scenes/LT05_L1TP_031034_20120504_20200820_02_T1/tiles/{z}/{x}/{y}.png?layer=natural_color",
                  "overlayType":"ls_chs",
                  "thumbnailPath":"https://landsatlook.usgs.gov/gen-browse?size=thumb&type=refl&product_id=LT05_L1TP_031034_20120504_20200820_02_T1"
               }
            ],
            "cloudCover":0,
            "entityId":"LT50310342012125EDC00",
            "displayId":"LT05_L2SP_031034_20120504_20200820_02_T1",
            "orderingId":"None",
            "metadata":[
               {
                  "id":"5e83d1199c3a007a",
                  "fieldName":"ID",
                  "dictionaryLink":"",
                  "value":"LT05_L2SP_031034_20120504_20200820_02_T1"
               },
               {
                  "id":"5e83d1197bb3b471",
                  "fieldName":"Landsat Product Identifier L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#landsat_product_id",
                  "value":"LT05_L2SP_031034_20120504_20200820_02_T1"
               },
               {
                  "id":"5e83d1192913040c",
                  "fieldName":"Landsat Product Identifier L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#landsat_product_id",
                  "value":"LT05_L2SP_031034_20120504_20200820_02_T1"
               },
               {
                  "id":"5e83d11959960e26",
                  "fieldName":"Landsat Product Identifier L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#landsat_product_id",
                  "value":"LT05_L1TP_031034_20120504_20200820_02_T1"
               },
               {
                  "id":"5e83d119d77c851b",
                  "fieldName":"Landsat Product Identifier L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#landsat_product_id",
                  "value":"LT05_L1TP_031034_20120504_20200820_02_T1"
               },
               {
                  "id":"5e83d119e1b717c2",
                  "fieldName":"ACQ_DATE",
                  "dictionaryLink":"",
                  "value":"2012/05/04"
               },
               {
                  "id":"5e83d11991cd15e9",
                  "fieldName":"Date Acquired",
                  "dictionaryLink":"",
                  "value":"2012/05/04"
               },
               {
                  "id":"5e83d119cd314e81",
                  "fieldName":"WRS_PATH",
                  "dictionaryLink":"",
                  "value":31
               },
               {
                  "id":"5e83d11953cebdfe",
                  "fieldName":"Landsat Scene Identifier",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#landsat_scene_id",
                  "value":"LT50310342012125EDC00"
               },
               {
                  "id":"5e83d1194a136c3d",
                  "fieldName":"Landsat Scene Identifier",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#landsat_scene_id",
                  "value":"LT50310342012125EDC00"
               },
               {
                  "id":"5e83d1192f6378c5",
                  "fieldName":"Path",
                  "dictionaryLink":"",
                  "value":" 031"
               },
               {
                  "id":"5e83d119f944a59c",
                  "fieldName":"Date Acquired",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#acquisition_date",
                  "value":"2012/05/04"
               },
               {
                  "id":"5e83d1194a5390ef",
                  "fieldName":"Date Acquired",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#acquisition_date",
                  "value":"2012/05/04"
               },
               {
                  "id":"5e83d11918829eab",
                  "fieldName":"Row",
                  "dictionaryLink":"",
                  "value":" 034"
               },
               {
                  "id":"5e83d1193fd26358",
                  "fieldName":"WRS_ROW",
                  "dictionaryLink":"",
                  "value":34
               },
               {
                  "id":"5e83d119c421c04",
                  "fieldName":"UL_LAT",
                  "dictionaryLink":"",
                  "value":"38.43249"
               },
               {
                  "id":"5e83d1197bb52c7b",
                  "fieldName":"Collection Category",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#collection_category",
                  "value":"T1"
               },
               {
                  "id":"5e83d119d3605acf",
                  "fieldName":"Collection Category",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#collection_category",
                  "value":"T1"
               },
               {
                  "id":"5e83d1193fcff364",
                  "fieldName":"Collection Number",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#collection_number",
                  "value":2
               },
               {
                  "id":"5e83d119d14a7b88",
                  "fieldName":"Collection Number",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#collection_number",
                  "value":2
               },
               {
                  "id":"5e83d11938eb6b8b",
                  "fieldName":"UL_LON",
                  "dictionaryLink":"",
                  "value":"-102.90121"
               },
               {
                  "id":"5e83d11959773042",
                  "fieldName":"UR_LAT",
                  "dictionaryLink":"",
                  "value":"36.55550"
               },
               {
                  "id":"5e83d1191e09dd34",
                  "fieldName":"Sensor Mode",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sensor_mode",
                  "value":"BUMPER"
               },
               {
                  "id":"5e83d119f3f29818",
                  "fieldName":"Sensor Mode",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sensor_mode",
                  "value":"BUMPER"
               },
               {
                  "id":"5e83d11967e7be31",
                  "fieldName":"WRS Path",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#wrs_path",
                  "value":" 031"
               },
               {
                  "id":"5e83d11934daa06d",
                  "fieldName":"WRS Path",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#wrs_path",
                  "value":" 031"
               },
               {
                  "id":"5e83d119dfa54533",
                  "fieldName":"UR_LON",
                  "dictionaryLink":"",
                  "value":"-100.82208"
               },
               {
                  "id":"5e83d119d19269d9",
                  "fieldName":"LR_LAT",
                  "dictionaryLink":"",
                  "value":"36.55550"
               },
               {
                  "id":"5e83d119d93462c",
                  "fieldName":"WRS Row",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#wrs_row",
                  "value":" 034"
               },
               {
                  "id":"5e83d119175a9f85",
                  "fieldName":"WRS Row",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#wrs_row",
                  "value":" 034"
               },
               {
                  "id":"5e83d119a2f770bb",
                  "fieldName":"LR_LON",
                  "dictionaryLink":"",
                  "value":"-101.31485"
               },
               {
                  "id":"5e83d1193eb1eda1",
                  "fieldName":"Full Partial Scene",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#full_partial_scene",
                  "value":"FULL"
               },
               {
                  "id":"5e83d119b5031133",
                  "fieldName":"Full Partial Scene",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#full_partial_scene",
                  "value":"FULL"
               },
               {
                  "id":"5e83d1199d010929",
                  "fieldName":"Date Product Generated L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#date_l2_generated",
                  "value":"2020/08/20"
               },
               {
                  "id":"5e83d11996b0840e",
                  "fieldName":"Date Product Generated L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#date_l2_generated",
                  "value":"2020/08/20"
               },
               {
                  "id":"5e83d119524c32e",
                  "fieldName":"LL_LAT",
                  "dictionaryLink":"",
                  "value":"36.85209"
               },
               {
                  "id":"5e83d119aa12a17b",
                  "fieldName":"LL_LON",
                  "dictionaryLink":"",
                  "value":"-103.35231"
               },
               {
                  "id":"5e83d119c4c93633",
                  "fieldName":"Date Product Generated L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#date_l1_generated",
                  "value":"2020/08/20"
               },
               {
                  "id":"5e83d11925f4d7b1",
                  "fieldName":"Date Product Generated L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#date_l1_generated",
                  "value":"2020/08/20"
               },
               {
                  "id":"5e83d1198198937",
                  "fieldName":"Start Time",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#start_time",
                  "value":"2012-05-04 17:09:28.88181"
               },
               {
                  "id":"5e83d119799cfc69",
                  "fieldName":"Start Time",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#start_time",
                  "value":"2012-05-04 17:09:28.88181"
               },
               {
                  "id":"5e83d119c231dfdb",
                  "fieldName":"Stop Time",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#stop_time",
                  "value":"2012-05-04 17:09:55.49463"
               },
               {
                  "id":"5e83d1197f85f30",
                  "fieldName":"Stop Time",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#stop_time",
                  "value":"2012-05-04 17:09:55.49463"
               },
               {
                  "id":"5e83d1192d8d752e",
                  "fieldName":"Station Identifier",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#distribution_site",
                  "value":"EDC"
               },
               {
                  "id":"5e83d119e814b3d5",
                  "fieldName":"Station Identifier",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#distribution_site",
                  "value":"EDC"
               },
               {
                  "id":"5e83d119a6b422f1",
                  "fieldName":"Day/Night Indicator",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#day_or_night",
                  "value":"DAY"
               },
               {
                  "id":"5e83d119d580c676",
                  "fieldName":"Day/Night Indicator",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#day_or_night",
                  "value":"DAY"
               },
               {
                  "id":"5e83d1191dbc4bb2",
                  "fieldName":"Land Cloud Cover",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#cloud_cover_land",
                  "value":"   0.00"
               },
               {
                  "id":"5e83d119241ce3ad",
                  "fieldName":"Land Cloud Cover",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#cloud_cover_land",
                  "value":"   0.00"
               },
               {
                  "id":"5e83d1192b06440a",
                  "fieldName":"Scene Cloud Cover L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#cloud_cover",
                  "value":"   0.00"
               },
               {
                  "id":"5e83d119c9649ed5",
                  "fieldName":"Scene Cloud Cover L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#cloud_cover",
                  "value":"   0.00"
               },
               {
                  "id":"5e83d1193835ad13",
                  "fieldName":"Ground Control Points Model",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ground_control_points_model",
                  "value":334
               },
               {
                  "id":"5e83d119f145b805",
                  "fieldName":"Ground Control Points Model",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ground_control_points_model",
                  "value":334
               },
               {
                  "id":"5e83d119c72d835f",
                  "fieldName":"Ground Control Points Version",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ground_control_points_version",
                  "value":5
               },
               {
                  "id":"5e83d1195d9e6973",
                  "fieldName":"Ground Control Points Version",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ground_control_points_version",
                  "value":5
               },
               {
                  "id":"5e83d119369526c2",
                  "fieldName":"Geometric RMSE Model",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#geometric_rmse_model",
                  "value":"    4.528"
               },
               {
                  "id":"5e83d1199b894cc6",
                  "fieldName":"Geometric RMSE Model",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#geometric_rmse_model",
                  "value":"    4.528"
               },
               {
                  "id":"5e83d119eb4a52b2",
                  "fieldName":"Geometric RMSE Model X",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#geometric_rmse_model_x",
                  "value":"    3.194"
               },
               {
                  "id":"5e83d119e8b2a364",
                  "fieldName":"Geometric RMSE Model X",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#geometric_rmse_model_x",
                  "value":"    3.194"
               },
               {
                  "id":"5e83d119b3959f54",
                  "fieldName":"Geometric RMSE Model Y",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#geometric_rmse_model_y   ",
                  "value":"    3.210"
               },
               {
                  "id":"5e83d119545197e5",
                  "fieldName":"Geometric RMSE Model Y",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#geometric_rmse_model_y",
                  "value":"    3.210"
               },
               {
                  "id":"5e83d119c6488aea",
                  "fieldName":"Sensor Anomalies",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sensor_anomalies",
                  "value":"NONE"
               },
               {
                  "id":"5e83d11915d08447",
                  "fieldName":"Sensor Anomalies",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sensor_anomalies",
                  "value":"NONE"
               },
               {
                  "id":"5e83d119a837407a",
                  "fieldName":"Data Anomaly",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#data_anomaly",
                  "value":"None"
               },
               {
                  "id":"5e83d119907f9709",
                  "fieldName":"Data Anomaly",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#data_anomaly",
                  "value":"None"
               },
               {
                  "id":"5e83d119da3199a5",
                  "fieldName":"Image Quality",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#image_quality_f1",
                  "value":9
               },
               {
                  "id":"5e83d119c9703e76",
                  "fieldName":"Image Quality",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#image_quality_f1",
                  "value":9
               },
               {
                  "id":"5e83d119ed82d34a",
                  "fieldName":"Gain Change",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#gain_change_band_1",
                  "value":"N"
               },
               {
                  "id":"5e83d119160507a9",
                  "fieldName":"Gain Change",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#gain_change_band_1",
                  "value":"N"
               },
               {
                  "id":"5e83d11961e2e91f",
                  "fieldName":"Processing Software Version",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#processing_software_version",
                  "value":"LPGS_15.3.1c"
               },
               {
                  "id":"5e83d119e682a749",
                  "fieldName":"Processing Software Version",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#processing_software_version",
                  "value":"LPGS_15.3.1c"
               },
               {
                  "id":"5e83d119d68c3c56",
                  "fieldName":"Sun Elevation L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sun_elevation",
                  "value":" 60.08771824"
               },
               {
                  "id":"5e83d119b4f2499a",
                  "fieldName":"Sun Elevation L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sun_elevation",
                  "value":" 60.08771824"
               },
               {
                  "id":"5e83d11974719513",
                  "fieldName":"Sun Azimuth L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sun_azimuth",
                  "value":" 128.84772414"
               },
               {
                  "id":"5e83d1196bd1a17e",
                  "fieldName":"Sun Azimuth L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sun_azimuth",
                  "value":" 128.84772414"
               },
               {
                  "id":"5e83d119bd6f280e",
                  "fieldName":"Data Type L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#data_type_21",
                  "value":"TM_L2SP"
               },
               {
                  "id":"5e83d119e43f7db9",
                  "fieldName":"Data Type L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#data_type_21",
                  "value":"TM_L2SP"
               },
               {
                  "id":"5e83d119709b05d5",
                  "fieldName":"Sensor Identifier",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sensor_id",
                  "value":"TM"
               },
               {
                  "id":"5e83d11922c49edf",
                  "fieldName":"Satellite",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#satellite",
                  "value":"LANDSAT_5"
               },
               {
                  "id":"5e83d11943388776",
                  "fieldName":"Sensor Identifier",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sensor_id",
                  "value":"TM"
               },
               {
                  "id":"5e83d1194ce8c25",
                  "fieldName":"Ephemeris Type",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ephemeris_type",
                  "value":"DEFINITIVE"
               },
               {
                  "id":"5e83d119620279fa",
                  "fieldName":"Satellite",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#satellite",
                  "value":5
               },
               {
                  "id":"5e83d119c93da2ae",
                  "fieldName":"Product Map Projection L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#map_projection_l2",
                  "value":"UTM"
               },
               {
                  "id":"5e83d11994227806",
                  "fieldName":"Ephemeris Type",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ephemeris_type",
                  "value":"DEFINITIVE"
               },
               {
                  "id":"5e83d11920d22995",
                  "fieldName":"UTM Zone",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#utm_zone",
                  "value":13
               },
               {
                  "id":"5e83d119dfc32fd1",
                  "fieldName":"Datum",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#datum",
                  "value":"WGS84"
               },
               {
                  "id":"5e83d119889cd954",
                  "fieldName":"Ellipsoid",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ellipsoid",
                  "value":"WGS84"
               },
               {
                  "id":"5e83d119629796c8",
                  "fieldName":"Product Map Projection L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#map_projection_l2",
                  "value":"UTM"
               },
               {
                  "id":"5e83d119fadd1071",
                  "fieldName":"UTM Zone",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#utm_zone",
                  "value":13
               },
               {
                  "id":"5e83d119388164c",
                  "fieldName":"Datum",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#datum",
                  "value":"WGS84"
               },
               {
                  "id":"5e83d119e4ff9290",
                  "fieldName":"Ellipsoid",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ellipsoid",
                  "value":"WGS84"
               },
               {
                  "id":"5e83d1193b2598ef",
                  "fieldName":"Scene Center Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"37&deg;29\\'49.67\"N"
               },
               {
                  "id":"5e83d119cf9cdcd0",
                  "fieldName":"Scene Center Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"102&deg;05\\'50.86\"W"
               },
               {
                  "id":"5e83d1197f20b27e",
                  "fieldName":"Corner Upper Left Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"38&deg;25\\'56.96\"N"
               },
               {
                  "id":"5e83d11936b6b01f",
                  "fieldName":"Corner Upper Left Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"102&deg;54\\'04.36\"W"
               },
               {
                  "id":"5e83d11961cccb4e",
                  "fieldName":"Corner Upper Right Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"38&deg;07\\'47.06\"N"
               },
               {
                  "id":"5e83d119914affb9",
                  "fieldName":"Corner Upper Right Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"100&deg;49\\'19.49\"W"
               },
               {
                  "id":"5e83d11933e075b3",
                  "fieldName":"Scene Center Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"37&deg;29\\'49.67\"N"
               },
               {
                  "id":"5e83d11929b7c342",
                  "fieldName":"Corner Lower Left Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"36&deg;51\\'07.52\"N"
               },
               {
                  "id":"5e83d11915ccea1b",
                  "fieldName":"Originator",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d119d03bef11",
                  "fieldName":"Scene Center Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"102&deg;05\\'50.86\"W"
               },
               {
                  "id":"5e83d119b27f36cd",
                  "fieldName":"Corner Upper Left Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"38&deg;25\\'56.96\"N"
               },
               {
                  "id":"5e83d119e5a5bc5a",
                  "fieldName":"Corner Lower Left Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"103&deg;21\\'08.32\"W"
               },
               {
                  "id":"5e83d119ae3d36ca",
                  "fieldName":"Corner Lower Right Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"36&deg;33\\'19.80\"N"
               },
               {
                  "id":"5e83d1197695fcdf",
                  "fieldName":"Corner Upper Left Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"102&deg;54\\'04.36\"W"
               },
               {
                  "id":"5e83d119ea4acbd4",
                  "fieldName":"Corner Upper Right Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"38&deg;07\\'47.06\"N"
               },
               {
                  "id":"5e83d119adad6c55",
                  "fieldName":"Corner Lower Right Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"101&deg;18\\'53.46\"W"
               },
               {
                  "id":"5e83d119f6d182f7",
                  "fieldName":"Corner Upper Right Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"100&deg;49\\'19.49\"W"
               },
               {
                  "id":"5e83d1197cfbeb7a",
                  "fieldName":"Scene Center Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 37.49713"
               },
               {
                  "id":"5e83d119dc6d299d",
                  "fieldName":"Scene Center Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-102.09746"
               },
               {
                  "id":"5e83d11932520ee0",
                  "fieldName":"Corner Lower Left Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"36&deg;51\\'07.52\"N"
               },
               {
                  "id":"5e83d1199113b057",
                  "fieldName":"Corner Lower Left Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"103&deg;21\\'08.32\"W"
               },
               {
                  "id":"5e83d11960e9d77a",
                  "fieldName":"Corner Upper Left Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 38.43249"
               },
               {
                  "id":"5e83d119595283a3",
                  "fieldName":"Corner Upper Left Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-102.90121"
               },
               {
                  "id":"5e83d1198025a16",
                  "fieldName":"Corner Lower Right Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"36&deg;33\\'19.80\"N"
               },
               {
                  "id":"5e83d119bac5c53b",
                  "fieldName":"Corner Lower Right Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"101&deg;18\\'53.46\"W"
               },
               {
                  "id":"5e83d119539410e8",
                  "fieldName":"Corner Upper Right Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 38.12974"
               },
               {
                  "id":"5e83d119189a44a8",
                  "fieldName":"Corner Upper Right Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-100.82208"
               },
               {
                  "id":"5e83d119e58ff41",
                  "fieldName":"Scene Center Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 37.49713"
               },
               {
                  "id":"5e83d119ac572a41",
                  "fieldName":"Publication_Date",
                  "dictionaryLink":"",
                  "value":"20050831"
               },
               {
                  "id":"5e83d119b97778e3",
                  "fieldName":"Corner Lower Left Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 36.85209"
               },
               {
                  "id":"5e83d119fc691855",
                  "fieldName":"Scene Center Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-102.09746"
               },
               {
                  "id":"5e83d119d033de1",
                  "fieldName":"Corner Lower Left Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-103.35231"
               },
               {
                  "id":"5e83d119b84d4d52",
                  "fieldName":"Corner Upper Left Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 38.43249"
               },
               {
                  "id":"5e83d119aef3d62e",
                  "fieldName":"Corner Lower Right Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 36.55550"
               },
               {
                  "id":"5e83d1196f42a17a",
                  "fieldName":"Corner Upper Left Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-102.90121"
               },
               {
                  "id":"5e83d119a7e81d55",
                  "fieldName":"Corner Upper Right Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 38.12974"
               },
               {
                  "id":"5e83d11967541ce6",
                  "fieldName":"Corner Lower Right Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-101.31485"
               },
               {
                  "id":"5e83d119a9ebd21b",
                  "fieldName":"Corner Upper Right Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-100.82208"
               },
               {
                  "id":"5e83d1199d5244f6",
                  "fieldName":"Corner Lower Left Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 36.85209"
               },
               {
                  "id":"5e83d119799824a8",
                  "fieldName":"Corner Lower Left Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-103.35231"
               },
               {
                  "id":"5e83d11917097daf",
                  "fieldName":"Corner Lower Right Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 36.55550"
               },
               {
                  "id":"5e83d119d6ce5eaf",
                  "fieldName":"Corner Lower Right Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-101.31485"
               },
               {
                  "id":"5e83d11913a82746",
                  "fieldName":"Title",
                  "dictionaryLink":"",
                  "value":"LANDSAT TM Collection 2 - Path:  31 Row:  34 for Scene:  LT05_L2SP_031034_20120504_20200820_02_T1"
               },
               {
                  "id":"5e83d11972d6e9f7",
                  "fieldName":"Geospatial_Data_Presentation_Form",
                  "dictionaryLink":"",
                  "value":"Remote-Sensing Image"
               },
               {
                  "id":"5e83d119920813bc",
                  "fieldName":"Publication_Place",
                  "dictionaryLink":"",
                  "value":"Sioux Falls, SD USA"
               },
               {
                  "id":"5e83d119df3fe2e7",
                  "fieldName":"Publisher",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d119d2a6e1c7",
                  "fieldName":"Online_Linkage",
                  "dictionaryLink":"",
                  "value":"https://doi.org/10.5066/P9IAXOVV"
               },
               {
                  "id":"5e83d1196aefe139",
                  "fieldName":"Abstract",
                  "dictionaryLink":"",
                  "value":"The USGS Earth Resources Observation and Science (EROS) Center archive holds data collected by the Landsat suite of satellites, beginning with Landsat 1 in 1972 through the latest Landsat Mission."
               },
               {
                  "id":"5e83d119684bd37c",
                  "fieldName":"Purpose",
                  "dictionaryLink":"",
                  "value":"The Landsat archive provides a rich collection of information about the Earth's land surface.  Major characteristics of changes to the surface of the planet can be detected, measured, and analyzed using Landsat data.  The information obtainable from the historical and current Landsat data play a key role in studying surface changes through time.  Collection 2 provides a quality-controlled selection of radiometrically calibrated and consistently geolocated data; particular geodetic accuracy and error tolerances are specified to align pixels for time series analysis through the full Landsat record."
               },
               {
                  "id":"5e83d11945f920c",
                  "fieldName":"Supplemental_Information",
                  "dictionaryLink":"",
                  "value":"Landsat represents the world's longest continuously acquired collection of space-based moderate-resolution land remote sensing data.  As a joint initiative between the U.S. Geological Survey (USGS) and the National Aeronautical Space Administration (NASA), the Landsat Project and the data it collects support government, commercial, industrial, civilian, military, and educational communities throughout the United States and world.  Landsat 8 Operational Land Imager (OLI)/Thermal Infrared Sensor (TIRS) Collection 2 provides a quality-controlled selection of radiometrically calibrated and consistently geolocated data; particular geodetic accuracy and error tolerances are specified to align pixels for time series analysis through the full Landsat record. (https://www.usgs.gov/core-science-systems/nli/landsat/landsat-collections)"
               },
               {
                  "id":"5e83d1197cc754b4",
                  "fieldName":"Calendar_Date",
                  "dictionaryLink":"",
                  "value":"19920704"
               },
               {
                  "id":"5e83d11966963850",
                  "fieldName":"Currentness_Reference",
                  "dictionaryLink":"",
                  "value":"Ground Condition"
               },
               {
                  "id":"5e83d119fc177449",
                  "fieldName":"Progress",
                  "dictionaryLink":"",
                  "value":"In Work"
               },
               {
                  "id":"5e83d11960ba536d",
                  "fieldName":"Maintenance_and_Update_Frequency",
                  "dictionaryLink":"",
                  "value":"Continually"
               },
               {
                  "id":"5e83d119586f0830",
                  "fieldName":"G-Ring_Latitude",
                  "dictionaryLink":"",
                  "value":"36.85209"
               },
               {
                  "id":"5e83d1191be10c7c",
                  "fieldName":"G-Ring_Longitude",
                  "dictionaryLink":"",
                  "value":"-103.35231"
               },
               {
                  "id":"5e83d119281579e6",
                  "fieldName":"Theme_Keyword_Thesaurus",
                  "dictionaryLink":"",
                  "value":"ISO 19115 Topic Category"
               },
               {
                  "id":"5e83d1194abc14dd",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"ImageryBaseMapsEarthCover"
               },
               {
                  "id":"60470441398df5b",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Remote Sensing"
               },
               {
                  "id":"604704413623c358",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Collection 2"
               },
               {
                  "id":"60470441c22543ba",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Infrared Imagery"
               },
               {
                  "id":"604704413ae1d393",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Imagery"
               },
               {
                  "id":"60470441a5f6eec9",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Visible Imagery"
               },
               {
                  "id":"6047044176fd6e3d",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Visible Wavelengths"
               },
               {
                  "id":"604704417339664e",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Radiance"
               },
               {
                  "id":"60470441fb324cd2",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Landsat"
               },
               {
                  "id":"60470441ef8940aa",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Satellite"
               },
               {
                  "id":"60470441fd132ec0",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) "
               },
               {
                  "id":"6047044187bc92ee",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Thematic Mapper (TM)"
               },
               {
                  "id":"6047044159f1e540",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"60470441627f211c",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Infrared Wavelengths"
               },
               {
                  "id":"5e83d119796d51f5",
                  "fieldName":"Place_Keyword_Thesaurus",
                  "dictionaryLink":"",
                  "value":"U.S. Department of Commerce, 1995, Countries, dependencies, areas of special sovereignty, and their principal administrative divisions, Federal Information Processing Standard 10-4, Washington, D.C., National Institute of Standards and Technology"
               },
               {
                  "id":"5e83d119abd5525a",
                  "fieldName":"Place_Keyword",
                  "dictionaryLink":"",
                  "value":"Land Surface"
               },
               {
                  "id":"5e83d119ba008a5",
                  "fieldName":"Access_Constraints",
                  "dictionaryLink":"",
                  "value":"Downloading and using these data signifies that the user comprehends and will comply with the USGS Privacy Policy.  Reading and understanding all portions of the metadata will protect the interests of both the user and the USGS. Please refer to https://www.usgs.gov/privacy.html."
               },
               {
                  "id":"5e83d11a5fe32ed8",
                  "fieldName":"Use_Constraints",
                  "dictionaryLink":"",
                  "value":"There is no guarantee of warranty concerning the accuracy of these data. Users should not use these data for critical applications without a full awareness of their limitations. Temporal changes may have occurred since the data were collected and some parts of these data may no longer represent actual surface conditions.  There are no restrictions regarding the use of Landsat data distributed by the U.S. Geological Survey (USGS), unless expressly identified prior to or at the time of receipt. For Landsat, we request the following statement be used: Data available from the U.S. Geological Survey. Please refer to https://www.usgs.gov/centers/eros/data-citation for more information on data citation."
               },
               {
                  "id":"5e83d11ab9c2ebac",
                  "fieldName":"Contact_Person",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11ab32a4f35",
                  "fieldName":"Contact_Organization",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d11ab61696bc",
                  "fieldName":"Contact_Position",
                  "dictionaryLink":"",
                  "value":"Customer Service Representative"
               },
               {
                  "id":"5e83d11ad3056599",
                  "fieldName":"Address_Type",
                  "dictionaryLink":"",
                  "value":"Mailing and Physical Address"
               },
               {
                  "id":"5e83d11a257fe85d",
                  "fieldName":"Address",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center 47914 252nd Street"
               },
               {
                  "id":"5e83d11ab3eacc95",
                  "fieldName":"City",
                  "dictionaryLink":"",
                  "value":"Sioux Falls"
               },
               {
                  "id":"5e83d11aa1694de",
                  "fieldName":"State_or_Province",
                  "dictionaryLink":"",
                  "value":"SD"
               },
               {
                  "id":"5e83d11ad2c2fb60",
                  "fieldName":"Postal_Code",
                  "dictionaryLink":"",
                  "value":"57198-0001"
               },
               {
                  "id":"5e83d11ae22dc5e3",
                  "fieldName":"Country",
                  "dictionaryLink":"",
                  "value":"USA"
               },
               {
                  "id":"5e83d11aa9f27cf6",
                  "fieldName":"Contact_Electronic_Mail_Address",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a540dcfe9",
                  "fieldName":"Hours_of_Service",
                  "dictionaryLink":"",
                  "value":"0800 - 1600 CT, M-F, 1400 - 2200 h GMT"
               },
               {
                  "id":"5e83d11ac32f724c",
                  "fieldName":"Contact_Instructions",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a5c48c7bc",
                  "fieldName":"Browse_Graphic_File_Name",
                  "dictionaryLink":"",
                  "value":"Not Available"
               },
               {
                  "id":"5e83d11ad8675c1d",
                  "fieldName":"Browse_Graphic_File_Description",
                  "dictionaryLink":"",
                  "value":"This is a compressed and resampled version of the Landsat image to help validate area of interest, data quality, and the impact of cloud cover, if it exists."
               },
               {
                  "id":"5e83d11ae71a3055",
                  "fieldName":"Browse_Graphic_File_Type",
                  "dictionaryLink":"",
                  "value":"JPEG"
               },
               {
                  "id":"5e83d11a9000b8ec",
                  "fieldName":"Security_Classification_System",
                  "dictionaryLink":"",
                  "value":"None"
               },
               {
                  "id":"5e83d11aa39ac7de",
                  "fieldName":"Security_Classification",
                  "dictionaryLink":"",
                  "value":"Unclassified"
               },
               {
                  "id":"5e83d11a1883ab2b",
                  "fieldName":"Security_Handling_Description",
                  "dictionaryLink":"",
                  "value":"None"
               },
               {
                  "id":"5e83d11a7a5e7378",
                  "fieldName":"Native_Data_Set_Environment",
                  "dictionaryLink":"",
                  "value":"Oracle"
               },
               {
                  "id":"5e83d11a53641db4",
                  "fieldName":"Originator",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d11a23ff726c",
                  "fieldName":"Publication_Date",
                  "dictionaryLink":"",
                  "value":"19720723"
               },
               {
                  "id":"5e83d11a3ae40d19",
                  "fieldName":"Title",
                  "dictionaryLink":"",
                  "value":"Landsat Thematic Mapper Collection 2"
               },
               {
                  "id":"5e83d11aa866af3f",
                  "fieldName":"Geospatial_Data_Presentation_Form",
                  "dictionaryLink":"",
                  "value":"Remote-Sensing Image"
               },
               {
                  "id":"5e83d11a3ff08366",
                  "fieldName":"Publication_Place",
                  "dictionaryLink":"",
                  "value":"Sioux Falls,  SD USA"
               },
               {
                  "id":"5e83d11ac764e8b1",
                  "fieldName":"Publisher",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d11af27a7036",
                  "fieldName":"Online_Linkage",
                  "dictionaryLink":"",
                  "value":"https://doi.org/10.5066/P9IAXOVV"
               },
               {
                  "id":"5e83d11a393fe841",
                  "fieldName":"Attribute_Accuracy_Report",
                  "dictionaryLink":"",
                  "value":"The identification of features is provided by the spectral signature which is the distinct electromagnetic energy it emits, reflects, or otherwise transmits.  Other signatures such as tone (lightness or darkness), texture (surface roughness or smoothness), pattern, shadow, shape, and size are just as important.  Thus, through the use of Landsat data, such elements as water features, soils, and vegetation can be identified and distinguished from each other."
               },
               {
                  "id":"5e83d11abfffd008",
                  "fieldName":"Logical_Consistency_Report",
                  "dictionaryLink":"",
                  "value":"N/A"
               },
               {
                  "id":"5e83d11a3a7b431a",
                  "fieldName":"Completeness_Report",
                  "dictionaryLink":"",
                  "value":"The Landsat satellites circle the Earth on a sun-synchronous, near-polar orbit imaging the same 185 km (115 miles) ground swath every 16 days."
               },
               {
                  "id":"5e83d11adfb52c64",
                  "fieldName":"Process_Description",
                  "dictionaryLink":"",
                  "value":"Landsat Collection 2 Level-2 surface reflectance and surface temperature science products are generated from Collection 2 Level-1 inputs that meet the less than 76 degrees Solar Zenith Angle constraint and include the required auxiliary data inputs to generate a scientifically viable product. For more information visit https://www.usgs.gov/core-science-systems/nli/landsat/landsat-collection-2-level-2-science-products."
               },
               {
                  "id":"5e83d11a886ef0cd",
                  "fieldName":"Process_Date",
                  "dictionaryLink":"",
                  "value":"19920704"
               },
               {
                  "id":"5e83d11a11da2ce6",
                  "fieldName":"Contact_Person",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11ab064168",
                  "fieldName":"Contact_Organization",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d11a2304f959",
                  "fieldName":"Contact_Organization",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d11a3bbcd0f2",
                  "fieldName":"Contact_Person",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a9e6d5dd6",
                  "fieldName":"Contact_Position",
                  "dictionaryLink":"",
                  "value":"Customer Service Representative"
               },
               {
                  "id":"5e83d11adff5e984",
                  "fieldName":"Address_Type",
                  "dictionaryLink":"",
                  "value":"Mailing and Physical Address"
               },
               {
                  "id":"5e83d11af63fd80e",
                  "fieldName":"Address",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center 47914 252nd Street"
               },
               {
                  "id":"5e83d11afb799f4a",
                  "fieldName":"City",
                  "dictionaryLink":"",
                  "value":"Sioux Falls"
               },
               {
                  "id":"5e83d11a44b6f7fa",
                  "fieldName":"State_or_Province",
                  "dictionaryLink":"",
                  "value":"SD"
               },
               {
                  "id":"5e83d11a7fc5e25c",
                  "fieldName":"Postal_Code",
                  "dictionaryLink":"",
                  "value":"57198-0001"
               },
               {
                  "id":"5e83d11a85d3d795",
                  "fieldName":"Country",
                  "dictionaryLink":"",
                  "value":"USA"
               },
               {
                  "id":"5e83d11ae7dc9222",
                  "fieldName":"Contact_Electronic_Mail_Address",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11aeb5a94ea",
                  "fieldName":"Hours_of_Service",
                  "dictionaryLink":"",
                  "value":"0800 - 1600 CT, M-F, 1400 - 2200 h GMT"
               },
               {
                  "id":"5e83d11ac055787a",
                  "fieldName":"Contact_Instructions",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a858097e6",
                  "fieldName":"Cloud_Cover",
                  "dictionaryLink":"",
                  "value":"0.000 percent"
               },
               {
                  "id":"5e83d11af1f2843b",
                  "fieldName":"Indirect_Spatial_Reference",
                  "dictionaryLink":"",
                  "value":"Landsats 4 and 5 collected data at 30-meter ground resolution. Band 6 was acquired at 120-meter resolution, but products are resampled to 30-meter pixels."
               },
               {
                  "id":"5e83d11a6b630fc9",
                  "fieldName":"Direct_Spatial_Reference_Method",
                  "dictionaryLink":"",
                  "value":"Raster"
               },
               {
                  "id":"5e83d11a19dda88e",
                  "fieldName":"Raster_Object_Type",
                  "dictionaryLink":"",
                  "value":"Pixel"
               },
               {
                  "id":"5e83d11ac4295b28",
                  "fieldName":"Contact_Person",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a883a82be",
                  "fieldName":"Contact_Organization",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d11aa774b26b",
                  "fieldName":"Contact_Position",
                  "dictionaryLink":"",
                  "value":"Customer Service Representative"
               },
               {
                  "id":"5e83d11ad2958365",
                  "fieldName":"Address_Type",
                  "dictionaryLink":"",
                  "value":"Mailing and Physical Address"
               },
               {
                  "id":"5e83d11af5fa6ab8",
                  "fieldName":"Address",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center 47914 252nd Street"
               },
               {
                  "id":"5e83d11a60757afb",
                  "fieldName":"City",
                  "dictionaryLink":"",
                  "value":"Sioux Falls"
               },
               {
                  "id":"5e83d11a7d7342b1",
                  "fieldName":"State_or_Province",
                  "dictionaryLink":"",
                  "value":"SD"
               },
               {
                  "id":"5e83d11a7c7a009c",
                  "fieldName":"Postal_Code",
                  "dictionaryLink":"",
                  "value":"57198-0001"
               },
               {
                  "id":"5e83d11a6085f9c4",
                  "fieldName":"Country",
                  "dictionaryLink":"",
                  "value":"USA"
               },
               {
                  "id":"5e83d11a8414a2f7",
                  "fieldName":"Contact_Electronic_Mail_Address",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11afd59eb5f",
                  "fieldName":"Hours_of_Service",
                  "dictionaryLink":"",
                  "value":"0800 - 1600 CT, M-F, 1400 - 2200 h GMT"
               },
               {
                  "id":"5e83d11a95f467a9",
                  "fieldName":"Contact_Instructions",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11abaff6573",
                  "fieldName":"Resource_Description",
                  "dictionaryLink":"",
                  "value":"LANDSAT TM Collection 2 Entity ID: LT50310342012125EDC00"
               },
               {
                  "id":"5e83d11a92c457e3",
                  "fieldName":"Distribution_Liability",
                  "dictionaryLink":"",
                  "value":"Although these data have been processed successfully on a computer system at the USGS, no warranty expressed or implied is made by the USGS regarding the use of the data on any other system, nor does the act of distribution constitute any such warranty."
               },
               {
                  "id":"5e83d11af5c0878d",
                  "fieldName":"Format_Name",
                  "dictionaryLink":"",
                  "value":"Geographcal-Tagged Image File Format (GeoTIFF), or Joint Photographic Experts Group (JPEG)"
               },
               {
                  "id":"5e83d11ae0bb7b8f",
                  "fieldName":"Network_Resource_Name",
                  "dictionaryLink":"",
                  "value":"https://earthexplorer.usgs.gov"
               },
               {
                  "id":"5e83d11afb83c4cd",
                  "fieldName":"Fees",
                  "dictionaryLink":"",
                  "value":"Fees:  All Landsat data held in the USGS EROS archive are downloadable at no cost to all users."
               },
               {
                  "id":"5e83d11abe18accc",
                  "fieldName":"Technical_Prerequisites",
                  "dictionaryLink":"",
                  "value":"Adequate computer capability is the only technical prerequisite for viewing data in digital form.\r\n\r\n"
               },
               {
                  "id":"5e83d11a1a5caead",
                  "fieldName":"Beginning_Date",
                  "dictionaryLink":"",
                  "value":"19821113"
               },
               {
                  "id":"5e83d11a7ab01984",
                  "fieldName":"Ending_Date",
                  "dictionaryLink":"",
                  "value":"20011001"
               },
               {
                  "id":"5e83d11a5fe31dbe",
                  "fieldName":"Metadata_Date",
                  "dictionaryLink":"",
                  "value":"20160422"
               },
               {
                  "id":"5e83d11ace8c4be6",
                  "fieldName":"Metadata_Review_Date",
                  "dictionaryLink":"",
                  "value":"As Needed"
               },
               {
                  "id":"5e83d11aaf7b1a8",
                  "fieldName":"Contact_Person",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a6fe629a0",
                  "fieldName":"Contact_Organization",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d11a612f9dca",
                  "fieldName":"Contact_Position",
                  "dictionaryLink":"",
                  "value":"Customer Service Representative"
               },
               {
                  "id":"5e83d11ac64398d4",
                  "fieldName":"Address_Type",
                  "dictionaryLink":"",
                  "value":"Mailing and Physical Address"
               },
               {
                  "id":"5e83d11a2ec3653a",
                  "fieldName":"Address",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center 47914 252nd Street"
               },
               {
                  "id":"5e83d11a9dc347a2",
                  "fieldName":"City",
                  "dictionaryLink":"",
                  "value":"Sioux Falls"
               },
               {
                  "id":"5e83d11a557d59a2",
                  "fieldName":"State_or_Province",
                  "dictionaryLink":"",
                  "value":"SD"
               },
               {
                  "id":"5e83d11ab5408135",
                  "fieldName":"Postal_Code",
                  "dictionaryLink":"",
                  "value":"57198-0001"
               },
               {
                  "id":"5e83d11a12675aa1",
                  "fieldName":"Country",
                  "dictionaryLink":"",
                  "value":"USA"
               },
               {
                  "id":"5e83d11a44f7d71b",
                  "fieldName":"Contact_Electronic_Mail_Address",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a72d65a0f",
                  "fieldName":"Hours_of_Service",
                  "dictionaryLink":"",
                  "value":"0800 - 1600 CT, M-F, 1400 - 2200 h GMT"
               },
               {
                  "id":"5e83d11ac62ff739",
                  "fieldName":"Contact_Instructions",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a2b75b6e1",
                  "fieldName":"Metadata_Standard_Name",
                  "dictionaryLink":"",
                  "value":"Content Standard for Digital Geospatial Metadata"
               },
               {
                  "id":"5e83d11a7c7a0451",
                  "fieldName":"Metadata_Standard_Version",
                  "dictionaryLink":"",
                  "value":"FGDC-STD-001-1998, Version 2"
               },
               {
                  "id":"5e83d11a1ab1600f",
                  "fieldName":"Metadata_Time_Convention",
                  "dictionaryLink":"",
                  "value":"Local Time"
               },
               {
                  "id":"5e83d11a6418b4e",
                  "fieldName":"Metadata_Access_Constraints",
                  "dictionaryLink":"",
                  "value":"None"
               },
               {
                  "id":"5e83d11a37751741",
                  "fieldName":"Metadata_Use_Constraints",
                  "dictionaryLink":"",
                  "value":"None"
               },
               {
                  "id":"5e83d11a6948e4a6",
                  "fieldName":"Metadata_Security_Classification_System",
                  "dictionaryLink":"",
                  "value":"None"
               },
               {
                  "id":"5e83d11a96bf92d9",
                  "fieldName":"Metadata_Security_Classification",
                  "dictionaryLink":"",
                  "value":"Unclassified"
               },
               {
                  "id":"5e83d11ae14acc9d",
                  "fieldName":"Metadata_Security_Handling_Description",
                  "dictionaryLink":"",
                  "value":"None"
               }
            ],
            "hasCustomizedMetadata":"None",
            "options":{
               "bulk":True,
               "download":True,
               "order":False,
               "secondary":False
            },
            "selected":{
               "bulk":False,
               "compare":False,
               "order":False
            },
            "spatialBounds":{
               "type":"Polygon",
               "coordinates":[
                  [
                     [
                        -103.35231,
                        36.5555
                     ],
                     [
                        -103.35231,
                        38.43249
                     ],
                     [
                        -100.82208,
                        38.43249
                     ],
                     [
                        -100.82208,
                        36.5555
                     ],
                     [
                        -103.35231,
                        36.5555
                     ]
                  ]
               ]
            },
            "spatialCoverage":{
               "type":"Polygon",
               "coordinates":[
                  [
                     [
                        -103.35231,
                        36.85209
                     ],
                     [
                        -101.31485,
                        36.5555
                     ],
                     [
                        -100.82208,
                        38.12974
                     ],
                     [
                        -102.90121,
                        38.43249
                     ],
                     [
                        -103.35231,
                        36.85209
                     ]
                  ]
               ]
            },
            "temporalCoverage":{
               "endDate":"2012-05-04 00:00:00",
               "startDate":"2012-05-04 00:00:00"
            },
            "publishDate":"2022-06-22 19:23:13-05"
         },
         {
            "browse":[
               {
                  "id":"5fb4b9d0e5094d61",
                  "browseRotationEnabled":"None",
                  "browseName":"Level-1 Reflective Browse",
                  "browsePath":"https://landsatlook.usgs.gov/gen-browse?size=rrb&type=refl&product_id=LT05_L1TP_031035_20120504_20200820_02_T1",
                  "overlayPath":"https://landsatlook.usgs.gov/dynamic-tiler/scenes/LT05_L1TP_031035_20120504_20200820_02_T1/tiles/{z}/{x}/{y}.png?layer=natural_color",
                  "overlayType":"ls_chs",
                  "thumbnailPath":"https://landsatlook.usgs.gov/gen-browse?size=thumb&type=refl&product_id=LT05_L1TP_031035_20120504_20200820_02_T1"
               }
            ],
            "cloudCover":0,
            "entityId":"LT50310352012125EDC00",
            "displayId":"LT05_L2SP_031035_20120504_20200820_02_T1",
            "orderingId":"None",
            "metadata":[
               {
                  "id":"5e83d1199c3a007a",
                  "fieldName":"ID",
                  "dictionaryLink":"",
                  "value":"LT05_L2SP_031035_20120504_20200820_02_T1"
               },
               {
                  "id":"5e83d1197bb3b471",
                  "fieldName":"Landsat Product Identifier L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#landsat_product_id",
                  "value":"LT05_L2SP_031035_20120504_20200820_02_T1"
               },
               {
                  "id":"5e83d1192913040c",
                  "fieldName":"Landsat Product Identifier L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#landsat_product_id",
                  "value":"LT05_L2SP_031035_20120504_20200820_02_T1"
               },
               {
                  "id":"5e83d11959960e26",
                  "fieldName":"Landsat Product Identifier L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#landsat_product_id",
                  "value":"LT05_L1TP_031035_20120504_20200820_02_T1"
               },
               {
                  "id":"5e83d119d77c851b",
                  "fieldName":"Landsat Product Identifier L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#landsat_product_id",
                  "value":"LT05_L1TP_031035_20120504_20200820_02_T1"
               },
               {
                  "id":"5e83d119e1b717c2",
                  "fieldName":"ACQ_DATE",
                  "dictionaryLink":"",
                  "value":"2012/05/04"
               },
               {
                  "id":"5e83d11991cd15e9",
                  "fieldName":"Date Acquired",
                  "dictionaryLink":"",
                  "value":"2012/05/04"
               },
               {
                  "id":"5e83d119cd314e81",
                  "fieldName":"WRS_PATH",
                  "dictionaryLink":"",
                  "value":31
               },
               {
                  "id":"5e83d11953cebdfe",
                  "fieldName":"Landsat Scene Identifier",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#landsat_scene_id",
                  "value":"LT50310352012125EDC00"
               },
               {
                  "id":"5e83d1194a136c3d",
                  "fieldName":"Landsat Scene Identifier",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#landsat_scene_id",
                  "value":"LT50310352012125EDC00"
               },
               {
                  "id":"5e83d1192f6378c5",
                  "fieldName":"Path",
                  "dictionaryLink":"",
                  "value":" 031"
               },
               {
                  "id":"5e83d119f944a59c",
                  "fieldName":"Date Acquired",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#acquisition_date",
                  "value":"2012/05/04"
               },
               {
                  "id":"5e83d1194a5390ef",
                  "fieldName":"Date Acquired",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#acquisition_date",
                  "value":"2012/05/04"
               },
               {
                  "id":"5e83d11918829eab",
                  "fieldName":"Row",
                  "dictionaryLink":"",
                  "value":" 035"
               },
               {
                  "id":"5e83d1193fd26358",
                  "fieldName":"WRS_ROW",
                  "dictionaryLink":"",
                  "value":35
               },
               {
                  "id":"5e83d119c421c04",
                  "fieldName":"UL_LAT",
                  "dictionaryLink":"",
                  "value":"36.99374"
               },
               {
                  "id":"5e83d1197bb52c7b",
                  "fieldName":"Collection Category",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#collection_category",
                  "value":"T1"
               },
               {
                  "id":"5e83d119d3605acf",
                  "fieldName":"Collection Category",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#collection_category",
                  "value":"T1"
               },
               {
                  "id":"5e83d1193fcff364",
                  "fieldName":"Collection Number",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#collection_number",
                  "value":2
               },
               {
                  "id":"5e83d119d14a7b88",
                  "fieldName":"Collection Number",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#collection_number",
                  "value":2
               },
               {
                  "id":"5e83d11938eb6b8b",
                  "fieldName":"UL_LON",
                  "dictionaryLink":"",
                  "value":"-103.30912"
               },
               {
                  "id":"5e83d11959773042",
                  "fieldName":"UR_LAT",
                  "dictionaryLink":"",
                  "value":"35.12093"
               },
               {
                  "id":"5e83d1191e09dd34",
                  "fieldName":"Sensor Mode",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sensor_mode",
                  "value":"BUMPER"
               },
               {
                  "id":"5e83d119f3f29818",
                  "fieldName":"Sensor Mode",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sensor_mode",
                  "value":"BUMPER"
               },
               {
                  "id":"5e83d11967e7be31",
                  "fieldName":"WRS Path",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#wrs_path",
                  "value":" 031"
               },
               {
                  "id":"5e83d11934daa06d",
                  "fieldName":"WRS Path",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#wrs_path",
                  "value":" 031"
               },
               {
                  "id":"5e83d119dfa54533",
                  "fieldName":"UR_LON",
                  "dictionaryLink":"",
                  "value":"-101.26822"
               },
               {
                  "id":"5e83d119d19269d9",
                  "fieldName":"LR_LAT",
                  "dictionaryLink":"",
                  "value":"35.12093"
               },
               {
                  "id":"5e83d119d93462c",
                  "fieldName":"WRS Row",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#wrs_row",
                  "value":" 035"
               },
               {
                  "id":"5e83d119175a9f85",
                  "fieldName":"WRS Row",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#wrs_row",
                  "value":" 035"
               },
               {
                  "id":"5e83d119a2f770bb",
                  "fieldName":"LR_LON",
                  "dictionaryLink":"",
                  "value":"-101.74635"
               },
               {
                  "id":"5e83d1193eb1eda1",
                  "fieldName":"Full Partial Scene",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#full_partial_scene",
                  "value":"FULL"
               },
               {
                  "id":"5e83d119b5031133",
                  "fieldName":"Full Partial Scene",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#full_partial_scene",
                  "value":"FULL"
               },
               {
                  "id":"5e83d1199d010929",
                  "fieldName":"Date Product Generated L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#date_l2_generated",
                  "value":"2020/08/20"
               },
               {
                  "id":"5e83d11996b0840e",
                  "fieldName":"Date Product Generated L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#date_l2_generated",
                  "value":"2020/08/20"
               },
               {
                  "id":"5e83d119524c32e",
                  "fieldName":"LL_LAT",
                  "dictionaryLink":"",
                  "value":"35.41228"
               },
               {
                  "id":"5e83d119aa12a17b",
                  "fieldName":"LL_LON",
                  "dictionaryLink":"",
                  "value":"-103.74835"
               },
               {
                  "id":"5e83d119c4c93633",
                  "fieldName":"Date Product Generated L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#date_l1_generated",
                  "value":"2020/08/20"
               },
               {
                  "id":"5e83d11925f4d7b1",
                  "fieldName":"Date Product Generated L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#date_l1_generated",
                  "value":"2020/08/20"
               },
               {
                  "id":"5e83d1198198937",
                  "fieldName":"Start Time",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#start_time",
                  "value":"2012-05-04 17:09:52.85475"
               },
               {
                  "id":"5e83d119799cfc69",
                  "fieldName":"Start Time",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#start_time",
                  "value":"2012-05-04 17:09:52.85475"
               },
               {
                  "id":"5e83d119c231dfdb",
                  "fieldName":"Stop Time",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#stop_time",
                  "value":"2012-05-04 17:10:19.4675"
               },
               {
                  "id":"5e83d1197f85f30",
                  "fieldName":"Stop Time",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#stop_time",
                  "value":"2012-05-04 17:10:19.4675"
               },
               {
                  "id":"5e83d1192d8d752e",
                  "fieldName":"Station Identifier",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#distribution_site",
                  "value":"EDC"
               },
               {
                  "id":"5e83d119e814b3d5",
                  "fieldName":"Station Identifier",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#distribution_site",
                  "value":"EDC"
               },
               {
                  "id":"5e83d119a6b422f1",
                  "fieldName":"Day/Night Indicator",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#day_or_night",
                  "value":"DAY"
               },
               {
                  "id":"5e83d119d580c676",
                  "fieldName":"Day/Night Indicator",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#day_or_night",
                  "value":"DAY"
               },
               {
                  "id":"5e83d1191dbc4bb2",
                  "fieldName":"Land Cloud Cover",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#cloud_cover_land",
                  "value":"   0.00"
               },
               {
                  "id":"5e83d119241ce3ad",
                  "fieldName":"Land Cloud Cover",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#cloud_cover_land",
                  "value":"   0.00"
               },
               {
                  "id":"5e83d1192b06440a",
                  "fieldName":"Scene Cloud Cover L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#cloud_cover",
                  "value":"   0.00"
               },
               {
                  "id":"5e83d119c9649ed5",
                  "fieldName":"Scene Cloud Cover L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#cloud_cover",
                  "value":"   0.00"
               },
               {
                  "id":"5e83d1193835ad13",
                  "fieldName":"Ground Control Points Model",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ground_control_points_model",
                  "value":643
               },
               {
                  "id":"5e83d119f145b805",
                  "fieldName":"Ground Control Points Model",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ground_control_points_model",
                  "value":643
               },
               {
                  "id":"5e83d119c72d835f",
                  "fieldName":"Ground Control Points Version",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ground_control_points_version",
                  "value":5
               },
               {
                  "id":"5e83d1195d9e6973",
                  "fieldName":"Ground Control Points Version",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ground_control_points_version",
                  "value":5
               },
               {
                  "id":"5e83d119369526c2",
                  "fieldName":"Geometric RMSE Model",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#geometric_rmse_model",
                  "value":"    4.575"
               },
               {
                  "id":"5e83d1199b894cc6",
                  "fieldName":"Geometric RMSE Model",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#geometric_rmse_model",
                  "value":"    4.575"
               },
               {
                  "id":"5e83d119eb4a52b2",
                  "fieldName":"Geometric RMSE Model X",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#geometric_rmse_model_x",
                  "value":"    3.288"
               },
               {
                  "id":"5e83d119e8b2a364",
                  "fieldName":"Geometric RMSE Model X",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#geometric_rmse_model_x",
                  "value":"    3.288"
               },
               {
                  "id":"5e83d119b3959f54",
                  "fieldName":"Geometric RMSE Model Y",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#geometric_rmse_model_y   ",
                  "value":"    3.181"
               },
               {
                  "id":"5e83d119545197e5",
                  "fieldName":"Geometric RMSE Model Y",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#geometric_rmse_model_y",
                  "value":"    3.181"
               },
               {
                  "id":"5e83d119c6488aea",
                  "fieldName":"Sensor Anomalies",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sensor_anomalies",
                  "value":"NONE"
               },
               {
                  "id":"5e83d11915d08447",
                  "fieldName":"Sensor Anomalies",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sensor_anomalies",
                  "value":"NONE"
               },
               {
                  "id":"5e83d119a837407a",
                  "fieldName":"Data Anomaly",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#data_anomaly",
                  "value":"None"
               },
               {
                  "id":"5e83d119907f9709",
                  "fieldName":"Data Anomaly",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#data_anomaly",
                  "value":"None"
               },
               {
                  "id":"5e83d119da3199a5",
                  "fieldName":"Image Quality",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#image_quality_f1",
                  "value":9
               },
               {
                  "id":"5e83d119c9703e76",
                  "fieldName":"Image Quality",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#image_quality_f1",
                  "value":9
               },
               {
                  "id":"5e83d119ed82d34a",
                  "fieldName":"Gain Change",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#gain_change_band_1",
                  "value":"N"
               },
               {
                  "id":"5e83d119160507a9",
                  "fieldName":"Gain Change",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#gain_change_band_1",
                  "value":"N"
               },
               {
                  "id":"5e83d11961e2e91f",
                  "fieldName":"Processing Software Version",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#processing_software_version",
                  "value":"LPGS_15.3.1c"
               },
               {
                  "id":"5e83d119e682a749",
                  "fieldName":"Processing Software Version",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#processing_software_version",
                  "value":"LPGS_15.3.1c"
               },
               {
                  "id":"5e83d119d68c3c56",
                  "fieldName":"Sun Elevation L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sun_elevation",
                  "value":" 60.76170522"
               },
               {
                  "id":"5e83d119b4f2499a",
                  "fieldName":"Sun Elevation L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sun_elevation",
                  "value":" 60.76170522"
               },
               {
                  "id":"5e83d11974719513",
                  "fieldName":"Sun Azimuth L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sun_azimuth",
                  "value":" 126.37480780"
               },
               {
                  "id":"5e83d1196bd1a17e",
                  "fieldName":"Sun Azimuth L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sun_azimuth",
                  "value":" 126.37480780"
               },
               {
                  "id":"5e83d119bd6f280e",
                  "fieldName":"Data Type L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#data_type_21",
                  "value":"TM_L2SP"
               },
               {
                  "id":"5e83d119e43f7db9",
                  "fieldName":"Data Type L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#data_type_21",
                  "value":"TM_L2SP"
               },
               {
                  "id":"5e83d119709b05d5",
                  "fieldName":"Sensor Identifier",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sensor_id",
                  "value":"TM"
               },
               {
                  "id":"5e83d11922c49edf",
                  "fieldName":"Satellite",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#satellite",
                  "value":"LANDSAT_5"
               },
               {
                  "id":"5e83d11943388776",
                  "fieldName":"Sensor Identifier",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sensor_id",
                  "value":"TM"
               },
               {
                  "id":"5e83d1194ce8c25",
                  "fieldName":"Ephemeris Type",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ephemeris_type",
                  "value":"DEFINITIVE"
               },
               {
                  "id":"5e83d119620279fa",
                  "fieldName":"Satellite",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#satellite",
                  "value":5
               },
               {
                  "id":"5e83d119c93da2ae",
                  "fieldName":"Product Map Projection L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#map_projection_l2",
                  "value":"UTM"
               },
               {
                  "id":"5e83d11994227806",
                  "fieldName":"Ephemeris Type",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ephemeris_type",
                  "value":"DEFINITIVE"
               },
               {
                  "id":"5e83d11920d22995",
                  "fieldName":"UTM Zone",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#utm_zone",
                  "value":13
               },
               {
                  "id":"5e83d119dfc32fd1",
                  "fieldName":"Datum",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#datum",
                  "value":"WGS84"
               },
               {
                  "id":"5e83d119889cd954",
                  "fieldName":"Ellipsoid",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ellipsoid",
                  "value":"WGS84"
               },
               {
                  "id":"5e83d119629796c8",
                  "fieldName":"Product Map Projection L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#map_projection_l2",
                  "value":"UTM"
               },
               {
                  "id":"5e83d119fadd1071",
                  "fieldName":"UTM Zone",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#utm_zone",
                  "value":13
               },
               {
                  "id":"5e83d119388164c",
                  "fieldName":"Datum",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#datum",
                  "value":"WGS84"
               },
               {
                  "id":"5e83d119e4ff9290",
                  "fieldName":"Ellipsoid",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ellipsoid",
                  "value":"WGS84"
               },
               {
                  "id":"5e83d1193b2598ef",
                  "fieldName":"Scene Center Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"36&deg;03\\'37.22\"N"
               },
               {
                  "id":"5e83d119cf9cdcd0",
                  "fieldName":"Scene Center Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"102&deg;31\\'04.33\"W"
               },
               {
                  "id":"5e83d1197f20b27e",
                  "fieldName":"Corner Upper Left Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"36&deg;59\\'37.46\"N"
               },
               {
                  "id":"5e83d11936b6b01f",
                  "fieldName":"Corner Upper Left Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"103&deg;18\\'32.83\"W"
               },
               {
                  "id":"5e83d11961cccb4e",
                  "fieldName":"Corner Upper Right Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"36&deg;41\\'47.90\"N"
               },
               {
                  "id":"5e83d119914affb9",
                  "fieldName":"Corner Upper Right Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"101&deg;16\\'05.59\"W"
               },
               {
                  "id":"5e83d11933e075b3",
                  "fieldName":"Scene Center Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"36&deg;03\\'37.22\"N"
               },
               {
                  "id":"5e83d11929b7c342",
                  "fieldName":"Corner Lower Left Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"35&deg;24\\'44.21\"N"
               },
               {
                  "id":"5e83d11915ccea1b",
                  "fieldName":"Originator",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d119d03bef11",
                  "fieldName":"Scene Center Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"102&deg;31\\'04.33\"W"
               },
               {
                  "id":"5e83d119b27f36cd",
                  "fieldName":"Corner Upper Left Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"36&deg;59\\'37.46\"N"
               },
               {
                  "id":"5e83d119e5a5bc5a",
                  "fieldName":"Corner Lower Left Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"103&deg;44\\'54.06\"W"
               },
               {
                  "id":"5e83d119ae3d36ca",
                  "fieldName":"Corner Lower Right Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"35&deg;07\\'15.35\"N"
               },
               {
                  "id":"5e83d1197695fcdf",
                  "fieldName":"Corner Upper Left Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"103&deg;18\\'32.83\"W"
               },
               {
                  "id":"5e83d119ea4acbd4",
                  "fieldName":"Corner Upper Right Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"36&deg;41\\'47.90\"N"
               },
               {
                  "id":"5e83d119adad6c55",
                  "fieldName":"Corner Lower Right Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"101&deg;44\\'46.86\"W"
               },
               {
                  "id":"5e83d119f6d182f7",
                  "fieldName":"Corner Upper Right Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"101&deg;16\\'05.59\"W"
               },
               {
                  "id":"5e83d1197cfbeb7a",
                  "fieldName":"Scene Center Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 36.06034"
               },
               {
                  "id":"5e83d119dc6d299d",
                  "fieldName":"Scene Center Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-102.51787"
               },
               {
                  "id":"5e83d11932520ee0",
                  "fieldName":"Corner Lower Left Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"35&deg;24\\'44.21\"N"
               },
               {
                  "id":"5e83d1199113b057",
                  "fieldName":"Corner Lower Left Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"103&deg;44\\'54.06\"W"
               },
               {
                  "id":"5e83d11960e9d77a",
                  "fieldName":"Corner Upper Left Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 36.99374"
               },
               {
                  "id":"5e83d119595283a3",
                  "fieldName":"Corner Upper Left Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-103.30912"
               },
               {
                  "id":"5e83d1198025a16",
                  "fieldName":"Corner Lower Right Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"35&deg;07\\'15.35\"N"
               },
               {
                  "id":"5e83d119bac5c53b",
                  "fieldName":"Corner Lower Right Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"101&deg;44\\'46.86\"W"
               },
               {
                  "id":"5e83d119539410e8",
                  "fieldName":"Corner Upper Right Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 36.69664"
               },
               {
                  "id":"5e83d119189a44a8",
                  "fieldName":"Corner Upper Right Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-101.26822"
               },
               {
                  "id":"5e83d119e58ff41",
                  "fieldName":"Scene Center Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 36.06034"
               },
               {
                  "id":"5e83d119ac572a41",
                  "fieldName":"Publication_Date",
                  "dictionaryLink":"",
                  "value":"20050831"
               },
               {
                  "id":"5e83d119b97778e3",
                  "fieldName":"Corner Lower Left Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 35.41228"
               },
               {
                  "id":"5e83d119fc691855",
                  "fieldName":"Scene Center Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-102.51787"
               },
               {
                  "id":"5e83d119d033de1",
                  "fieldName":"Corner Lower Left Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-103.74835"
               },
               {
                  "id":"5e83d119b84d4d52",
                  "fieldName":"Corner Upper Left Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 36.99374"
               },
               {
                  "id":"5e83d119aef3d62e",
                  "fieldName":"Corner Lower Right Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 35.12093"
               },
               {
                  "id":"5e83d1196f42a17a",
                  "fieldName":"Corner Upper Left Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-103.30912"
               },
               {
                  "id":"5e83d119a7e81d55",
                  "fieldName":"Corner Upper Right Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 36.69664"
               },
               {
                  "id":"5e83d11967541ce6",
                  "fieldName":"Corner Lower Right Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-101.74635"
               },
               {
                  "id":"5e83d119a9ebd21b",
                  "fieldName":"Corner Upper Right Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-101.26822"
               },
               {
                  "id":"5e83d1199d5244f6",
                  "fieldName":"Corner Lower Left Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 35.41228"
               },
               {
                  "id":"5e83d119799824a8",
                  "fieldName":"Corner Lower Left Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-103.74835"
               },
               {
                  "id":"5e83d11917097daf",
                  "fieldName":"Corner Lower Right Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 35.12093"
               },
               {
                  "id":"5e83d119d6ce5eaf",
                  "fieldName":"Corner Lower Right Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-101.74635"
               },
               {
                  "id":"5e83d11913a82746",
                  "fieldName":"Title",
                  "dictionaryLink":"",
                  "value":"LANDSAT TM Collection 2 - Path:  31 Row:  35 for Scene:  LT05_L2SP_031035_20120504_20200820_02_T1"
               },
               {
                  "id":"5e83d11972d6e9f7",
                  "fieldName":"Geospatial_Data_Presentation_Form",
                  "dictionaryLink":"",
                  "value":"Remote-Sensing Image"
               },
               {
                  "id":"5e83d119920813bc",
                  "fieldName":"Publication_Place",
                  "dictionaryLink":"",
                  "value":"Sioux Falls, SD USA"
               },
               {
                  "id":"5e83d119df3fe2e7",
                  "fieldName":"Publisher",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d119d2a6e1c7",
                  "fieldName":"Online_Linkage",
                  "dictionaryLink":"",
                  "value":"https://doi.org/10.5066/P9IAXOVV"
               },
               {
                  "id":"5e83d1196aefe139",
                  "fieldName":"Abstract",
                  "dictionaryLink":"",
                  "value":"The USGS Earth Resources Observation and Science (EROS) Center archive holds data collected by the Landsat suite of satellites, beginning with Landsat 1 in 1972 through the latest Landsat Mission."
               },
               {
                  "id":"5e83d119684bd37c",
                  "fieldName":"Purpose",
                  "dictionaryLink":"",
                  "value":"The Landsat archive provides a rich collection of information about the Earth's land surface.  Major characteristics of changes to the surface of the planet can be detected, measured, and analyzed using Landsat data.  The information obtainable from the historical and current Landsat data play a key role in studying surface changes through time.  Collection 2 provides a quality-controlled selection of radiometrically calibrated and consistently geolocated data; particular geodetic accuracy and error tolerances are specified to align pixels for time series analysis through the full Landsat record."
               },
               {
                  "id":"5e83d11945f920c",
                  "fieldName":"Supplemental_Information",
                  "dictionaryLink":"",
                  "value":"Landsat represents the world's longest continuously acquired collection of space-based moderate-resolution land remote sensing data.  As a joint initiative between the U.S. Geological Survey (USGS) and the National Aeronautical Space Administration (NASA), the Landsat Project and the data it collects support government, commercial, industrial, civilian, military, and educational communities throughout the United States and world.  Landsat 8 Operational Land Imager (OLI)/Thermal Infrared Sensor (TIRS) Collection 2 provides a quality-controlled selection of radiometrically calibrated and consistently geolocated data; particular geodetic accuracy and error tolerances are specified to align pixels for time series analysis through the full Landsat record. (https://www.usgs.gov/core-science-systems/nli/landsat/landsat-collections)"
               },
               {
                  "id":"5e83d1197cc754b4",
                  "fieldName":"Calendar_Date",
                  "dictionaryLink":"",
                  "value":"19920704"
               },
               {
                  "id":"5e83d11966963850",
                  "fieldName":"Currentness_Reference",
                  "dictionaryLink":"",
                  "value":"Ground Condition"
               },
               {
                  "id":"5e83d119fc177449",
                  "fieldName":"Progress",
                  "dictionaryLink":"",
                  "value":"In Work"
               },
               {
                  "id":"5e83d11960ba536d",
                  "fieldName":"Maintenance_and_Update_Frequency",
                  "dictionaryLink":"",
                  "value":"Continually"
               },
               {
                  "id":"5e83d119586f0830",
                  "fieldName":"G-Ring_Latitude",
                  "dictionaryLink":"",
                  "value":"35.41228"
               },
               {
                  "id":"5e83d1191be10c7c",
                  "fieldName":"G-Ring_Longitude",
                  "dictionaryLink":"",
                  "value":"-103.74835"
               },
               {
                  "id":"5e83d119281579e6",
                  "fieldName":"Theme_Keyword_Thesaurus",
                  "dictionaryLink":"",
                  "value":"ISO 19115 Topic Category"
               },
               {
                  "id":"5e83d1194abc14dd",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"ImageryBaseMapsEarthCover"
               },
               {
                  "id":"60470441398df5b",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Remote Sensing"
               },
               {
                  "id":"604704413623c358",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Collection 2"
               },
               {
                  "id":"60470441c22543ba",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Infrared Imagery"
               },
               {
                  "id":"604704413ae1d393",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Imagery"
               },
               {
                  "id":"60470441a5f6eec9",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Visible Imagery"
               },
               {
                  "id":"6047044176fd6e3d",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Visible Wavelengths"
               },
               {
                  "id":"604704417339664e",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Radiance"
               },
               {
                  "id":"60470441fb324cd2",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Landsat"
               },
               {
                  "id":"60470441ef8940aa",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Satellite"
               },
               {
                  "id":"60470441fd132ec0",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) "
               },
               {
                  "id":"6047044187bc92ee",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Thematic Mapper (TM)"
               },
               {
                  "id":"6047044159f1e540",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"60470441627f211c",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Infrared Wavelengths"
               },
               {
                  "id":"5e83d119796d51f5",
                  "fieldName":"Place_Keyword_Thesaurus",
                  "dictionaryLink":"",
                  "value":"U.S. Department of Commerce, 1995, Countries, dependencies, areas of special sovereignty, and their principal administrative divisions, Federal Information Processing Standard 10-4, Washington, D.C., National Institute of Standards and Technology"
               },
               {
                  "id":"5e83d119abd5525a",
                  "fieldName":"Place_Keyword",
                  "dictionaryLink":"",
                  "value":"Land Surface"
               },
               {
                  "id":"5e83d119ba008a5",
                  "fieldName":"Access_Constraints",
                  "dictionaryLink":"",
                  "value":"Downloading and using these data signifies that the user comprehends and will comply with the USGS Privacy Policy.  Reading and understanding all portions of the metadata will protect the interests of both the user and the USGS. Please refer to https://www.usgs.gov/privacy.html."
               },
               {
                  "id":"5e83d11a5fe32ed8",
                  "fieldName":"Use_Constraints",
                  "dictionaryLink":"",
                  "value":"There is no guarantee of warranty concerning the accuracy of these data. Users should not use these data for critical applications without a full awareness of their limitations. Temporal changes may have occurred since the data were collected and some parts of these data may no longer represent actual surface conditions.  There are no restrictions regarding the use of Landsat data distributed by the U.S. Geological Survey (USGS), unless expressly identified prior to or at the time of receipt. For Landsat, we request the following statement be used: Data available from the U.S. Geological Survey. Please refer to https://www.usgs.gov/centers/eros/data-citation for more information on data citation."
               },
               {
                  "id":"5e83d11ab9c2ebac",
                  "fieldName":"Contact_Person",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11ab32a4f35",
                  "fieldName":"Contact_Organization",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d11ab61696bc",
                  "fieldName":"Contact_Position",
                  "dictionaryLink":"",
                  "value":"Customer Service Representative"
               },
               {
                  "id":"5e83d11ad3056599",
                  "fieldName":"Address_Type",
                  "dictionaryLink":"",
                  "value":"Mailing and Physical Address"
               },
               {
                  "id":"5e83d11a257fe85d",
                  "fieldName":"Address",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center 47914 252nd Street"
               },
               {
                  "id":"5e83d11ab3eacc95",
                  "fieldName":"City",
                  "dictionaryLink":"",
                  "value":"Sioux Falls"
               },
               {
                  "id":"5e83d11aa1694de",
                  "fieldName":"State_or_Province",
                  "dictionaryLink":"",
                  "value":"SD"
               },
               {
                  "id":"5e83d11ad2c2fb60",
                  "fieldName":"Postal_Code",
                  "dictionaryLink":"",
                  "value":"57198-0001"
               },
               {
                  "id":"5e83d11ae22dc5e3",
                  "fieldName":"Country",
                  "dictionaryLink":"",
                  "value":"USA"
               },
               {
                  "id":"5e83d11aa9f27cf6",
                  "fieldName":"Contact_Electronic_Mail_Address",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a540dcfe9",
                  "fieldName":"Hours_of_Service",
                  "dictionaryLink":"",
                  "value":"0800 - 1600 CT, M-F, 1400 - 2200 h GMT"
               },
               {
                  "id":"5e83d11ac32f724c",
                  "fieldName":"Contact_Instructions",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a5c48c7bc",
                  "fieldName":"Browse_Graphic_File_Name",
                  "dictionaryLink":"",
                  "value":"Not Available"
               },
               {
                  "id":"5e83d11ad8675c1d",
                  "fieldName":"Browse_Graphic_File_Description",
                  "dictionaryLink":"",
                  "value":"This is a compressed and resampled version of the Landsat image to help validate area of interest, data quality, and the impact of cloud cover, if it exists."
               },
               {
                  "id":"5e83d11ae71a3055",
                  "fieldName":"Browse_Graphic_File_Type",
                  "dictionaryLink":"",
                  "value":"JPEG"
               },
               {
                  "id":"5e83d11a9000b8ec",
                  "fieldName":"Security_Classification_System",
                  "dictionaryLink":"",
                  "value":"None"
               },
               {
                  "id":"5e83d11aa39ac7de",
                  "fieldName":"Security_Classification",
                  "dictionaryLink":"",
                  "value":"Unclassified"
               },
               {
                  "id":"5e83d11a1883ab2b",
                  "fieldName":"Security_Handling_Description",
                  "dictionaryLink":"",
                  "value":"None"
               },
               {
                  "id":"5e83d11a7a5e7378",
                  "fieldName":"Native_Data_Set_Environment",
                  "dictionaryLink":"",
                  "value":"Oracle"
               },
               {
                  "id":"5e83d11a53641db4",
                  "fieldName":"Originator",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d11a23ff726c",
                  "fieldName":"Publication_Date",
                  "dictionaryLink":"",
                  "value":"19720723"
               },
               {
                  "id":"5e83d11a3ae40d19",
                  "fieldName":"Title",
                  "dictionaryLink":"",
                  "value":"Landsat Thematic Mapper Collection 2"
               },
               {
                  "id":"5e83d11aa866af3f",
                  "fieldName":"Geospatial_Data_Presentation_Form",
                  "dictionaryLink":"",
                  "value":"Remote-Sensing Image"
               },
               {
                  "id":"5e83d11a3ff08366",
                  "fieldName":"Publication_Place",
                  "dictionaryLink":"",
                  "value":"Sioux Falls,  SD USA"
               },
               {
                  "id":"5e83d11ac764e8b1",
                  "fieldName":"Publisher",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d11af27a7036",
                  "fieldName":"Online_Linkage",
                  "dictionaryLink":"",
                  "value":"https://doi.org/10.5066/P9IAXOVV"
               },
               {
                  "id":"5e83d11a393fe841",
                  "fieldName":"Attribute_Accuracy_Report",
                  "dictionaryLink":"",
                  "value":"The identification of features is provided by the spectral signature which is the distinct electromagnetic energy it emits, reflects, or otherwise transmits.  Other signatures such as tone (lightness or darkness), texture (surface roughness or smoothness), pattern, shadow, shape, and size are just as important.  Thus, through the use of Landsat data, such elements as water features, soils, and vegetation can be identified and distinguished from each other."
               },
               {
                  "id":"5e83d11abfffd008",
                  "fieldName":"Logical_Consistency_Report",
                  "dictionaryLink":"",
                  "value":"N/A"
               },
               {
                  "id":"5e83d11a3a7b431a",
                  "fieldName":"Completeness_Report",
                  "dictionaryLink":"",
                  "value":"The Landsat satellites circle the Earth on a sun-synchronous, near-polar orbit imaging the same 185 km (115 miles) ground swath every 16 days."
               },
               {
                  "id":"5e83d11adfb52c64",
                  "fieldName":"Process_Description",
                  "dictionaryLink":"",
                  "value":"Landsat Collection 2 Level-2 surface reflectance and surface temperature science products are generated from Collection 2 Level-1 inputs that meet the less than 76 degrees Solar Zenith Angle constraint and include the required auxiliary data inputs to generate a scientifically viable product. For more information visit https://www.usgs.gov/core-science-systems/nli/landsat/landsat-collection-2-level-2-science-products."
               },
               {
                  "id":"5e83d11a886ef0cd",
                  "fieldName":"Process_Date",
                  "dictionaryLink":"",
                  "value":"19920704"
               },
               {
                  "id":"5e83d11a11da2ce6",
                  "fieldName":"Contact_Person",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11ab064168",
                  "fieldName":"Contact_Organization",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d11a2304f959",
                  "fieldName":"Contact_Organization",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d11a3bbcd0f2",
                  "fieldName":"Contact_Person",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a9e6d5dd6",
                  "fieldName":"Contact_Position",
                  "dictionaryLink":"",
                  "value":"Customer Service Representative"
               },
               {
                  "id":"5e83d11adff5e984",
                  "fieldName":"Address_Type",
                  "dictionaryLink":"",
                  "value":"Mailing and Physical Address"
               },
               {
                  "id":"5e83d11af63fd80e",
                  "fieldName":"Address",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center 47914 252nd Street"
               },
               {
                  "id":"5e83d11afb799f4a",
                  "fieldName":"City",
                  "dictionaryLink":"",
                  "value":"Sioux Falls"
               },
               {
                  "id":"5e83d11a44b6f7fa",
                  "fieldName":"State_or_Province",
                  "dictionaryLink":"",
                  "value":"SD"
               },
               {
                  "id":"5e83d11a7fc5e25c",
                  "fieldName":"Postal_Code",
                  "dictionaryLink":"",
                  "value":"57198-0001"
               },
               {
                  "id":"5e83d11a85d3d795",
                  "fieldName":"Country",
                  "dictionaryLink":"",
                  "value":"USA"
               },
               {
                  "id":"5e83d11ae7dc9222",
                  "fieldName":"Contact_Electronic_Mail_Address",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11aeb5a94ea",
                  "fieldName":"Hours_of_Service",
                  "dictionaryLink":"",
                  "value":"0800 - 1600 CT, M-F, 1400 - 2200 h GMT"
               },
               {
                  "id":"5e83d11ac055787a",
                  "fieldName":"Contact_Instructions",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a858097e6",
                  "fieldName":"Cloud_Cover",
                  "dictionaryLink":"",
                  "value":"0.000 percent"
               },
               {
                  "id":"5e83d11af1f2843b",
                  "fieldName":"Indirect_Spatial_Reference",
                  "dictionaryLink":"",
                  "value":"Landsats 4 and 5 collected data at 30-meter ground resolution. Band 6 was acquired at 120-meter resolution, but products are resampled to 30-meter pixels."
               },
               {
                  "id":"5e83d11a6b630fc9",
                  "fieldName":"Direct_Spatial_Reference_Method",
                  "dictionaryLink":"",
                  "value":"Raster"
               },
               {
                  "id":"5e83d11a19dda88e",
                  "fieldName":"Raster_Object_Type",
                  "dictionaryLink":"",
                  "value":"Pixel"
               },
               {
                  "id":"5e83d11ac4295b28",
                  "fieldName":"Contact_Person",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a883a82be",
                  "fieldName":"Contact_Organization",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d11aa774b26b",
                  "fieldName":"Contact_Position",
                  "dictionaryLink":"",
                  "value":"Customer Service Representative"
               },
               {
                  "id":"5e83d11ad2958365",
                  "fieldName":"Address_Type",
                  "dictionaryLink":"",
                  "value":"Mailing and Physical Address"
               },
               {
                  "id":"5e83d11af5fa6ab8",
                  "fieldName":"Address",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center 47914 252nd Street"
               },
               {
                  "id":"5e83d11a60757afb",
                  "fieldName":"City",
                  "dictionaryLink":"",
                  "value":"Sioux Falls"
               },
               {
                  "id":"5e83d11a7d7342b1",
                  "fieldName":"State_or_Province",
                  "dictionaryLink":"",
                  "value":"SD"
               },
               {
                  "id":"5e83d11a7c7a009c",
                  "fieldName":"Postal_Code",
                  "dictionaryLink":"",
                  "value":"57198-0001"
               },
               {
                  "id":"5e83d11a6085f9c4",
                  "fieldName":"Country",
                  "dictionaryLink":"",
                  "value":"USA"
               },
               {
                  "id":"5e83d11a8414a2f7",
                  "fieldName":"Contact_Electronic_Mail_Address",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11afd59eb5f",
                  "fieldName":"Hours_of_Service",
                  "dictionaryLink":"",
                  "value":"0800 - 1600 CT, M-F, 1400 - 2200 h GMT"
               },
               {
                  "id":"5e83d11a95f467a9",
                  "fieldName":"Contact_Instructions",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11abaff6573",
                  "fieldName":"Resource_Description",
                  "dictionaryLink":"",
                  "value":"LANDSAT TM Collection 2 Entity ID: LT50310352012125EDC00"
               },
               {
                  "id":"5e83d11a92c457e3",
                  "fieldName":"Distribution_Liability",
                  "dictionaryLink":"",
                  "value":"Although these data have been processed successfully on a computer system at the USGS, no warranty expressed or implied is made by the USGS regarding the use of the data on any other system, nor does the act of distribution constitute any such warranty."
               },
               {
                  "id":"5e83d11af5c0878d",
                  "fieldName":"Format_Name",
                  "dictionaryLink":"",
                  "value":"Geographcal-Tagged Image File Format (GeoTIFF), or Joint Photographic Experts Group (JPEG)"
               },
               {
                  "id":"5e83d11ae0bb7b8f",
                  "fieldName":"Network_Resource_Name",
                  "dictionaryLink":"",
                  "value":"https://earthexplorer.usgs.gov"
               },
               {
                  "id":"5e83d11afb83c4cd",
                  "fieldName":"Fees",
                  "dictionaryLink":"",
                  "value":"Fees:  All Landsat data held in the USGS EROS archive are downloadable at no cost to all users."
               },
               {
                  "id":"5e83d11abe18accc",
                  "fieldName":"Technical_Prerequisites",
                  "dictionaryLink":"",
                  "value":"Adequate computer capability is the only technical prerequisite for viewing data in digital form.\r\n\r\n"
               },
               {
                  "id":"5e83d11a1a5caead",
                  "fieldName":"Beginning_Date",
                  "dictionaryLink":"",
                  "value":"19821113"
               },
               {
                  "id":"5e83d11a7ab01984",
                  "fieldName":"Ending_Date",
                  "dictionaryLink":"",
                  "value":"20011001"
               },
               {
                  "id":"5e83d11a5fe31dbe",
                  "fieldName":"Metadata_Date",
                  "dictionaryLink":"",
                  "value":"20160422"
               },
               {
                  "id":"5e83d11ace8c4be6",
                  "fieldName":"Metadata_Review_Date",
                  "dictionaryLink":"",
                  "value":"As Needed"
               },
               {
                  "id":"5e83d11aaf7b1a8",
                  "fieldName":"Contact_Person",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a6fe629a0",
                  "fieldName":"Contact_Organization",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d11a612f9dca",
                  "fieldName":"Contact_Position",
                  "dictionaryLink":"",
                  "value":"Customer Service Representative"
               },
               {
                  "id":"5e83d11ac64398d4",
                  "fieldName":"Address_Type",
                  "dictionaryLink":"",
                  "value":"Mailing and Physical Address"
               },
               {
                  "id":"5e83d11a2ec3653a",
                  "fieldName":"Address",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center 47914 252nd Street"
               },
               {
                  "id":"5e83d11a9dc347a2",
                  "fieldName":"City",
                  "dictionaryLink":"",
                  "value":"Sioux Falls"
               },
               {
                  "id":"5e83d11a557d59a2",
                  "fieldName":"State_or_Province",
                  "dictionaryLink":"",
                  "value":"SD"
               },
               {
                  "id":"5e83d11ab5408135",
                  "fieldName":"Postal_Code",
                  "dictionaryLink":"",
                  "value":"57198-0001"
               },
               {
                  "id":"5e83d11a12675aa1",
                  "fieldName":"Country",
                  "dictionaryLink":"",
                  "value":"USA"
               },
               {
                  "id":"5e83d11a44f7d71b",
                  "fieldName":"Contact_Electronic_Mail_Address",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a72d65a0f",
                  "fieldName":"Hours_of_Service",
                  "dictionaryLink":"",
                  "value":"0800 - 1600 CT, M-F, 1400 - 2200 h GMT"
               },
               {
                  "id":"5e83d11ac62ff739",
                  "fieldName":"Contact_Instructions",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a2b75b6e1",
                  "fieldName":"Metadata_Standard_Name",
                  "dictionaryLink":"",
                  "value":"Content Standard for Digital Geospatial Metadata"
               },
               {
                  "id":"5e83d11a7c7a0451",
                  "fieldName":"Metadata_Standard_Version",
                  "dictionaryLink":"",
                  "value":"FGDC-STD-001-1998, Version 2"
               },
               {
                  "id":"5e83d11a1ab1600f",
                  "fieldName":"Metadata_Time_Convention",
                  "dictionaryLink":"",
                  "value":"Local Time"
               },
               {
                  "id":"5e83d11a6418b4e",
                  "fieldName":"Metadata_Access_Constraints",
                  "dictionaryLink":"",
                  "value":"None"
               },
               {
                  "id":"5e83d11a37751741",
                  "fieldName":"Metadata_Use_Constraints",
                  "dictionaryLink":"",
                  "value":"None"
               },
               {
                  "id":"5e83d11a6948e4a6",
                  "fieldName":"Metadata_Security_Classification_System",
                  "dictionaryLink":"",
                  "value":"None"
               },
               {
                  "id":"5e83d11a96bf92d9",
                  "fieldName":"Metadata_Security_Classification",
                  "dictionaryLink":"",
                  "value":"Unclassified"
               },
               {
                  "id":"5e83d11ae14acc9d",
                  "fieldName":"Metadata_Security_Handling_Description",
                  "dictionaryLink":"",
                  "value":"None"
               }
            ],
            "hasCustomizedMetadata":"None",
            "options":{
               "bulk":True,
               "download":True,
               "order":False,
               "secondary":False
            },
            "selected":{
               "bulk":False,
               "compare":False,
               "order":False
            },
            "spatialBounds":{
               "type":"Polygon",
               "coordinates":[
                  [
                     [
                        -103.74835,
                        35.12093
                     ],
                     [
                        -103.74835,
                        36.99374
                     ],
                     [
                        -101.26822,
                        36.99374
                     ],
                     [
                        -101.26822,
                        35.12093
                     ],
                     [
                        -103.74835,
                        35.12093
                     ]
                  ]
               ]
            },
            "spatialCoverage":{
               "type":"Polygon",
               "coordinates":[
                  [
                     [
                        -103.74835,
                        35.41228
                     ],
                     [
                        -101.74635,
                        35.12093
                     ],
                     [
                        -101.26822,
                        36.69664
                     ],
                     [
                        -103.30912,
                        36.99374
                     ],
                     [
                        -103.74835,
                        35.41228
                     ]
                  ]
               ]
            },
            "temporalCoverage":{
               "endDate":"2012-05-04 00:00:00",
               "startDate":"2012-05-04 00:00:00"
            },
            "publishDate":"2022-06-22 19:23:13-05"
         },
         {
            "browse":[
               {
                  "id":"5fb4b9d0e5094d61",
                  "browseRotationEnabled":"None",
                  "browseName":"Level-1 Reflective Browse",
                  "browsePath":"https://landsatlook.usgs.gov/gen-browse?size=rrb&type=refl&product_id=LT05_L1TP_031036_20120504_20200820_02_T1",
                  "overlayPath":"https://landsatlook.usgs.gov/dynamic-tiler/scenes/LT05_L1TP_031036_20120504_20200820_02_T1/tiles/{z}/{x}/{y}.png?layer=natural_color",
                  "overlayType":"ls_chs",
                  "thumbnailPath":"https://landsatlook.usgs.gov/gen-browse?size=thumb&type=refl&product_id=LT05_L1TP_031036_20120504_20200820_02_T1"
               }
            ],
            "cloudCover":0,
            "entityId":"LT50310362012125EDC00",
            "displayId":"LT05_L2SP_031036_20120504_20200820_02_T1",
            "orderingId":"None",
            "metadata":[
               {
                  "id":"5e83d1199c3a007a",
                  "fieldName":"ID",
                  "dictionaryLink":"",
                  "value":"LT05_L2SP_031036_20120504_20200820_02_T1"
               },
               {
                  "id":"5e83d1197bb3b471",
                  "fieldName":"Landsat Product Identifier L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#landsat_product_id",
                  "value":"LT05_L2SP_031036_20120504_20200820_02_T1"
               },
               {
                  "id":"5e83d1192913040c",
                  "fieldName":"Landsat Product Identifier L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#landsat_product_id",
                  "value":"LT05_L2SP_031036_20120504_20200820_02_T1"
               },
               {
                  "id":"5e83d11959960e26",
                  "fieldName":"Landsat Product Identifier L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#landsat_product_id",
                  "value":"LT05_L1TP_031036_20120504_20200820_02_T1"
               },
               {
                  "id":"5e83d119d77c851b",
                  "fieldName":"Landsat Product Identifier L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#landsat_product_id",
                  "value":"LT05_L1TP_031036_20120504_20200820_02_T1"
               },
               {
                  "id":"5e83d119e1b717c2",
                  "fieldName":"ACQ_DATE",
                  "dictionaryLink":"",
                  "value":"2012/05/04"
               },
               {
                  "id":"5e83d11991cd15e9",
                  "fieldName":"Date Acquired",
                  "dictionaryLink":"",
                  "value":"2012/05/04"
               },
               {
                  "id":"5e83d119cd314e81",
                  "fieldName":"WRS_PATH",
                  "dictionaryLink":"",
                  "value":31
               },
               {
                  "id":"5e83d11953cebdfe",
                  "fieldName":"Landsat Scene Identifier",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#landsat_scene_id",
                  "value":"LT50310362012125EDC00"
               },
               {
                  "id":"5e83d1194a136c3d",
                  "fieldName":"Landsat Scene Identifier",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#landsat_scene_id",
                  "value":"LT50310362012125EDC00"
               },
               {
                  "id":"5e83d1192f6378c5",
                  "fieldName":"Path",
                  "dictionaryLink":"",
                  "value":" 031"
               },
               {
                  "id":"5e83d119f944a59c",
                  "fieldName":"Date Acquired",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#acquisition_date",
                  "value":"2012/05/04"
               },
               {
                  "id":"5e83d1194a5390ef",
                  "fieldName":"Date Acquired",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#acquisition_date",
                  "value":"2012/05/04"
               },
               {
                  "id":"5e83d11918829eab",
                  "fieldName":"Row",
                  "dictionaryLink":"",
                  "value":" 036"
               },
               {
                  "id":"5e83d1193fd26358",
                  "fieldName":"WRS_ROW",
                  "dictionaryLink":"",
                  "value":36
               },
               {
                  "id":"5e83d119c421c04",
                  "fieldName":"UL_LAT",
                  "dictionaryLink":"",
                  "value":"35.56248"
               },
               {
                  "id":"5e83d1197bb52c7b",
                  "fieldName":"Collection Category",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#collection_category",
                  "value":"T1"
               },
               {
                  "id":"5e83d119d3605acf",
                  "fieldName":"Collection Category",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#collection_category",
                  "value":"T1"
               },
               {
                  "id":"5e83d1193fcff364",
                  "fieldName":"Collection Number",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#collection_number",
                  "value":2
               },
               {
                  "id":"5e83d119d14a7b88",
                  "fieldName":"Collection Number",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#collection_number",
                  "value":2
               },
               {
                  "id":"5e83d11938eb6b8b",
                  "fieldName":"UL_LON",
                  "dictionaryLink":"",
                  "value":"-103.70399"
               },
               {
                  "id":"5e83d11959773042",
                  "fieldName":"UR_LAT",
                  "dictionaryLink":"",
                  "value":"33.69355"
               },
               {
                  "id":"5e83d1191e09dd34",
                  "fieldName":"Sensor Mode",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sensor_mode",
                  "value":"BUMPER"
               },
               {
                  "id":"5e83d119f3f29818",
                  "fieldName":"Sensor Mode",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sensor_mode",
                  "value":"BUMPER"
               },
               {
                  "id":"5e83d11967e7be31",
                  "fieldName":"WRS Path",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#wrs_path",
                  "value":" 031"
               },
               {
                  "id":"5e83d11934daa06d",
                  "fieldName":"WRS Path",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#wrs_path",
                  "value":" 031"
               },
               {
                  "id":"5e83d119dfa54533",
                  "fieldName":"UR_LON",
                  "dictionaryLink":"",
                  "value":"-101.69857"
               },
               {
                  "id":"5e83d119d19269d9",
                  "fieldName":"LR_LAT",
                  "dictionaryLink":"",
                  "value":"33.69355"
               },
               {
                  "id":"5e83d119d93462c",
                  "fieldName":"WRS Row",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#wrs_row",
                  "value":" 036"
               },
               {
                  "id":"5e83d119175a9f85",
                  "fieldName":"WRS Row",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#wrs_row",
                  "value":" 036"
               },
               {
                  "id":"5e83d119a2f770bb",
                  "fieldName":"LR_LON",
                  "dictionaryLink":"",
                  "value":"-102.16336"
               },
               {
                  "id":"5e83d1193eb1eda1",
                  "fieldName":"Full Partial Scene",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#full_partial_scene",
                  "value":"FULL"
               },
               {
                  "id":"5e83d119b5031133",
                  "fieldName":"Full Partial Scene",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#full_partial_scene",
                  "value":"FULL"
               },
               {
                  "id":"5e83d1199d010929",
                  "fieldName":"Date Product Generated L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#date_l2_generated",
                  "value":"2020/08/20"
               },
               {
                  "id":"5e83d11996b0840e",
                  "fieldName":"Date Product Generated L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#date_l2_generated",
                  "value":"2020/08/20"
               },
               {
                  "id":"5e83d119524c32e",
                  "fieldName":"LL_LAT",
                  "dictionaryLink":"",
                  "value":"33.98005"
               },
               {
                  "id":"5e83d119aa12a17b",
                  "fieldName":"LL_LON",
                  "dictionaryLink":"",
                  "value":"-104.13244"
               },
               {
                  "id":"5e83d119c4c93633",
                  "fieldName":"Date Product Generated L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#date_l1_generated",
                  "value":"2020/08/20"
               },
               {
                  "id":"5e83d11925f4d7b1",
                  "fieldName":"Date Product Generated L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#date_l1_generated",
                  "value":"2020/08/20"
               },
               {
                  "id":"5e83d1198198937",
                  "fieldName":"Start Time",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#start_time",
                  "value":"2012-05-04 17:10:16.68494"
               },
               {
                  "id":"5e83d119799cfc69",
                  "fieldName":"Start Time",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#start_time",
                  "value":"2012-05-04 17:10:16.68494"
               },
               {
                  "id":"5e83d119c231dfdb",
                  "fieldName":"Stop Time",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#stop_time",
                  "value":"2012-05-04 17:10:43.29769"
               },
               {
                  "id":"5e83d1197f85f30",
                  "fieldName":"Stop Time",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#stop_time",
                  "value":"2012-05-04 17:10:43.29769"
               },
               {
                  "id":"5e83d1192d8d752e",
                  "fieldName":"Station Identifier",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#distribution_site",
                  "value":"EDC"
               },
               {
                  "id":"5e83d119e814b3d5",
                  "fieldName":"Station Identifier",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#distribution_site",
                  "value":"EDC"
               },
               {
                  "id":"5e83d119a6b422f1",
                  "fieldName":"Day/Night Indicator",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#day_or_night",
                  "value":"DAY"
               },
               {
                  "id":"5e83d119d580c676",
                  "fieldName":"Day/Night Indicator",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#day_or_night",
                  "value":"DAY"
               },
               {
                  "id":"5e83d1191dbc4bb2",
                  "fieldName":"Land Cloud Cover",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#cloud_cover_land",
                  "value":"   0.00"
               },
               {
                  "id":"5e83d119241ce3ad",
                  "fieldName":"Land Cloud Cover",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#cloud_cover_land",
                  "value":"   0.00"
               },
               {
                  "id":"5e83d1192b06440a",
                  "fieldName":"Scene Cloud Cover L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#cloud_cover",
                  "value":"   0.00"
               },
               {
                  "id":"5e83d119c9649ed5",
                  "fieldName":"Scene Cloud Cover L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#cloud_cover",
                  "value":"   0.00"
               },
               {
                  "id":"5e83d1193835ad13",
                  "fieldName":"Ground Control Points Model",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ground_control_points_model",
                  "value":658
               },
               {
                  "id":"5e83d119f145b805",
                  "fieldName":"Ground Control Points Model",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ground_control_points_model",
                  "value":658
               },
               {
                  "id":"5e83d119c72d835f",
                  "fieldName":"Ground Control Points Version",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ground_control_points_version",
                  "value":5
               },
               {
                  "id":"5e83d1195d9e6973",
                  "fieldName":"Ground Control Points Version",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ground_control_points_version",
                  "value":5
               },
               {
                  "id":"5e83d119369526c2",
                  "fieldName":"Geometric RMSE Model",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#geometric_rmse_model",
                  "value":"    4.464"
               },
               {
                  "id":"5e83d1199b894cc6",
                  "fieldName":"Geometric RMSE Model",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#geometric_rmse_model",
                  "value":"    4.464"
               },
               {
                  "id":"5e83d119eb4a52b2",
                  "fieldName":"Geometric RMSE Model X",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#geometric_rmse_model_x",
                  "value":"    3.098"
               },
               {
                  "id":"5e83d119e8b2a364",
                  "fieldName":"Geometric RMSE Model X",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#geometric_rmse_model_x",
                  "value":"    3.098"
               },
               {
                  "id":"5e83d119b3959f54",
                  "fieldName":"Geometric RMSE Model Y",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#geometric_rmse_model_y   ",
                  "value":"    3.214"
               },
               {
                  "id":"5e83d119545197e5",
                  "fieldName":"Geometric RMSE Model Y",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#geometric_rmse_model_y",
                  "value":"    3.214"
               },
               {
                  "id":"5e83d119c6488aea",
                  "fieldName":"Sensor Anomalies",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sensor_anomalies",
                  "value":"NONE"
               },
               {
                  "id":"5e83d11915d08447",
                  "fieldName":"Sensor Anomalies",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sensor_anomalies",
                  "value":"NONE"
               },
               {
                  "id":"5e83d119a837407a",
                  "fieldName":"Data Anomaly",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#data_anomaly",
                  "value":"None"
               },
               {
                  "id":"5e83d119907f9709",
                  "fieldName":"Data Anomaly",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#data_anomaly",
                  "value":"None"
               },
               {
                  "id":"5e83d119da3199a5",
                  "fieldName":"Image Quality",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#image_quality_f1",
                  "value":9
               },
               {
                  "id":"5e83d119c9703e76",
                  "fieldName":"Image Quality",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#image_quality_f1",
                  "value":9
               },
               {
                  "id":"5e83d119ed82d34a",
                  "fieldName":"Gain Change",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#gain_change_band_1",
                  "value":"N"
               },
               {
                  "id":"5e83d119160507a9",
                  "fieldName":"Gain Change",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#gain_change_band_1",
                  "value":"N"
               },
               {
                  "id":"5e83d11961e2e91f",
                  "fieldName":"Processing Software Version",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#processing_software_version",
                  "value":"LPGS_15.3.1c"
               },
               {
                  "id":"5e83d119e682a749",
                  "fieldName":"Processing Software Version",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#processing_software_version",
                  "value":"LPGS_15.3.1c"
               },
               {
                  "id":"5e83d119d68c3c56",
                  "fieldName":"Sun Elevation L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sun_elevation",
                  "value":" 61.37639468"
               },
               {
                  "id":"5e83d119b4f2499a",
                  "fieldName":"Sun Elevation L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sun_elevation",
                  "value":" 61.37639468"
               },
               {
                  "id":"5e83d11974719513",
                  "fieldName":"Sun Azimuth L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sun_azimuth",
                  "value":" 123.81509762"
               },
               {
                  "id":"5e83d1196bd1a17e",
                  "fieldName":"Sun Azimuth L1",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sun_azimuth",
                  "value":" 123.81509762"
               },
               {
                  "id":"5e83d119bd6f280e",
                  "fieldName":"Data Type L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#data_type_21",
                  "value":"TM_L2SP"
               },
               {
                  "id":"5e83d119e43f7db9",
                  "fieldName":"Data Type L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#data_type_21",
                  "value":"TM_L2SP"
               },
               {
                  "id":"5e83d119709b05d5",
                  "fieldName":"Sensor Identifier",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sensor_id",
                  "value":"TM"
               },
               {
                  "id":"5e83d11922c49edf",
                  "fieldName":"Satellite",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#satellite",
                  "value":"LANDSAT_5"
               },
               {
                  "id":"5e83d11943388776",
                  "fieldName":"Sensor Identifier",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#sensor_id",
                  "value":"TM"
               },
               {
                  "id":"5e83d1194ce8c25",
                  "fieldName":"Ephemeris Type",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ephemeris_type",
                  "value":"DEFINITIVE"
               },
               {
                  "id":"5e83d119620279fa",
                  "fieldName":"Satellite",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#satellite",
                  "value":5
               },
               {
                  "id":"5e83d119c93da2ae",
                  "fieldName":"Product Map Projection L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#map_projection_l2",
                  "value":"UTM"
               },
               {
                  "id":"5e83d11994227806",
                  "fieldName":"Ephemeris Type",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ephemeris_type",
                  "value":"DEFINITIVE"
               },
               {
                  "id":"5e83d11920d22995",
                  "fieldName":"UTM Zone",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#utm_zone",
                  "value":13
               },
               {
                  "id":"5e83d119dfc32fd1",
                  "fieldName":"Datum",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#datum",
                  "value":"WGS84"
               },
               {
                  "id":"5e83d119889cd954",
                  "fieldName":"Ellipsoid",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ellipsoid",
                  "value":"WGS84"
               },
               {
                  "id":"5e83d119629796c8",
                  "fieldName":"Product Map Projection L2",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#map_projection_l2",
                  "value":"UTM"
               },
               {
                  "id":"5e83d119fadd1071",
                  "fieldName":"UTM Zone",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#utm_zone",
                  "value":13
               },
               {
                  "id":"5e83d119388164c",
                  "fieldName":"Datum",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#datum",
                  "value":"WGS84"
               },
               {
                  "id":"5e83d119e4ff9290",
                  "fieldName":"Ellipsoid",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#ellipsoid",
                  "value":"WGS84"
               },
               {
                  "id":"5e83d1193b2598ef",
                  "fieldName":"Scene Center Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"34&deg;37\\'51.20\"N"
               },
               {
                  "id":"5e83d119cf9cdcd0",
                  "fieldName":"Scene Center Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"102&deg;55\\'28.06\"W"
               },
               {
                  "id":"5e83d1197f20b27e",
                  "fieldName":"Corner Upper Left Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"35&deg;33\\'44.93\"N"
               },
               {
                  "id":"5e83d11936b6b01f",
                  "fieldName":"Corner Upper Left Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"103&deg;42\\'14.36\"W"
               },
               {
                  "id":"5e83d11961cccb4e",
                  "fieldName":"Corner Upper Right Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"35&deg;16\\'14.27\"N"
               },
               {
                  "id":"5e83d119914affb9",
                  "fieldName":"Corner Upper Right Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"101&deg;41\\'54.85\"W"
               },
               {
                  "id":"5e83d11933e075b3",
                  "fieldName":"Scene Center Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"34&deg;37\\'51.20\"N"
               },
               {
                  "id":"5e83d11929b7c342",
                  "fieldName":"Corner Lower Left Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"33&deg;58\\'48.18\"N"
               },
               {
                  "id":"5e83d11915ccea1b",
                  "fieldName":"Originator",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d119d03bef11",
                  "fieldName":"Scene Center Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"102&deg;55\\'28.06\"W"
               },
               {
                  "id":"5e83d119b27f36cd",
                  "fieldName":"Corner Upper Left Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"35&deg;33\\'44.93\"N"
               },
               {
                  "id":"5e83d119e5a5bc5a",
                  "fieldName":"Corner Lower Left Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"104&deg;07\\'56.78\"W"
               },
               {
                  "id":"5e83d119ae3d36ca",
                  "fieldName":"Corner Lower Right Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"33&deg;41\\'36.78\"N"
               },
               {
                  "id":"5e83d1197695fcdf",
                  "fieldName":"Corner Upper Left Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"103&deg;42\\'14.36\"W"
               },
               {
                  "id":"5e83d119ea4acbd4",
                  "fieldName":"Corner Upper Right Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"35&deg;16\\'14.27\"N"
               },
               {
                  "id":"5e83d119adad6c55",
                  "fieldName":"Corner Lower Right Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"102&deg;09\\'48.10\"W"
               },
               {
                  "id":"5e83d119f6d182f7",
                  "fieldName":"Corner Upper Right Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"101&deg;41\\'54.85\"W"
               },
               {
                  "id":"5e83d1197cfbeb7a",
                  "fieldName":"Scene Center Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 34.63089"
               },
               {
                  "id":"5e83d119dc6d299d",
                  "fieldName":"Scene Center Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-102.92446"
               },
               {
                  "id":"5e83d11932520ee0",
                  "fieldName":"Corner Lower Left Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"33&deg;58\\'48.18\"N"
               },
               {
                  "id":"5e83d1199113b057",
                  "fieldName":"Corner Lower Left Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"104&deg;07\\'56.78\"W"
               },
               {
                  "id":"5e83d11960e9d77a",
                  "fieldName":"Corner Upper Left Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 35.56248"
               },
               {
                  "id":"5e83d119595283a3",
                  "fieldName":"Corner Upper Left Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-103.70399"
               },
               {
                  "id":"5e83d1198025a16",
                  "fieldName":"Corner Lower Right Lat DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"33&deg;41\\'36.78\"N"
               },
               {
                  "id":"5e83d119bac5c53b",
                  "fieldName":"Corner Lower Right Long DMS",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_degrees",
                  "value":"102&deg;09\\'48.10\"W"
               },
               {
                  "id":"5e83d119539410e8",
                  "fieldName":"Corner Upper Right Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 35.27063"
               },
               {
                  "id":"5e83d119189a44a8",
                  "fieldName":"Corner Upper Right Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-101.69857"
               },
               {
                  "id":"5e83d119e58ff41",
                  "fieldName":"Scene Center Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 34.63089"
               },
               {
                  "id":"5e83d119ac572a41",
                  "fieldName":"Publication_Date",
                  "dictionaryLink":"",
                  "value":"20050831"
               },
               {
                  "id":"5e83d119b97778e3",
                  "fieldName":"Corner Lower Left Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 33.98005"
               },
               {
                  "id":"5e83d119fc691855",
                  "fieldName":"Scene Center Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-102.92446"
               },
               {
                  "id":"5e83d119d033de1",
                  "fieldName":"Corner Lower Left Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-104.13244"
               },
               {
                  "id":"5e83d119b84d4d52",
                  "fieldName":"Corner Upper Left Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 35.56248"
               },
               {
                  "id":"5e83d119aef3d62e",
                  "fieldName":"Corner Lower Right Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 33.69355"
               },
               {
                  "id":"5e83d1196f42a17a",
                  "fieldName":"Corner Upper Left Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-103.70399"
               },
               {
                  "id":"5e83d119a7e81d55",
                  "fieldName":"Corner Upper Right Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 35.27063"
               },
               {
                  "id":"5e83d11967541ce6",
                  "fieldName":"Corner Lower Right Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-102.16336"
               },
               {
                  "id":"5e83d119a9ebd21b",
                  "fieldName":"Corner Upper Right Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-101.69857"
               },
               {
                  "id":"5e83d1199d5244f6",
                  "fieldName":"Corner Lower Left Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 33.98005"
               },
               {
                  "id":"5e83d119799824a8",
                  "fieldName":"Corner Lower Left Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-104.13244"
               },
               {
                  "id":"5e83d11917097daf",
                  "fieldName":"Corner Lower Right Latitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":" 33.69355"
               },
               {
                  "id":"5e83d119d6ce5eaf",
                  "fieldName":"Corner Lower Right Longitude",
                  "dictionaryLink":"https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#coordinates_decimal",
                  "value":"-102.16336"
               },
               {
                  "id":"5e83d11913a82746",
                  "fieldName":"Title",
                  "dictionaryLink":"",
                  "value":"LANDSAT TM Collection 2 - Path:  31 Row:  36 for Scene:  LT05_L2SP_031036_20120504_20200820_02_T1"
               },
               {
                  "id":"5e83d11972d6e9f7",
                  "fieldName":"Geospatial_Data_Presentation_Form",
                  "dictionaryLink":"",
                  "value":"Remote-Sensing Image"
               },
               {
                  "id":"5e83d119920813bc",
                  "fieldName":"Publication_Place",
                  "dictionaryLink":"",
                  "value":"Sioux Falls, SD USA"
               },
               {
                  "id":"5e83d119df3fe2e7",
                  "fieldName":"Publisher",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d119d2a6e1c7",
                  "fieldName":"Online_Linkage",
                  "dictionaryLink":"",
                  "value":"https://doi.org/10.5066/P9IAXOVV"
               },
               {
                  "id":"5e83d1196aefe139",
                  "fieldName":"Abstract",
                  "dictionaryLink":"",
                  "value":"The USGS Earth Resources Observation and Science (EROS) Center archive holds data collected by the Landsat suite of satellites, beginning with Landsat 1 in 1972 through the latest Landsat Mission."
               },
               {
                  "id":"5e83d119684bd37c",
                  "fieldName":"Purpose",
                  "dictionaryLink":"",
                  "value":"The Landsat archive provides a rich collection of information about the Earth's land surface.  Major characteristics of changes to the surface of the planet can be detected, measured, and analyzed using Landsat data.  The information obtainable from the historical and current Landsat data play a key role in studying surface changes through time.  Collection 2 provides a quality-controlled selection of radiometrically calibrated and consistently geolocated data; particular geodetic accuracy and error tolerances are specified to align pixels for time series analysis through the full Landsat record."
               },
               {
                  "id":"5e83d11945f920c",
                  "fieldName":"Supplemental_Information",
                  "dictionaryLink":"",
                  "value":"Landsat represents the world's longest continuously acquired collection of space-based moderate-resolution land remote sensing data.  As a joint initiative between the U.S. Geological Survey (USGS) and the National Aeronautical Space Administration (NASA), the Landsat Project and the data it collects support government, commercial, industrial, civilian, military, and educational communities throughout the United States and world.  Landsat 8 Operational Land Imager (OLI)/Thermal Infrared Sensor (TIRS) Collection 2 provides a quality-controlled selection of radiometrically calibrated and consistently geolocated data; particular geodetic accuracy and error tolerances are specified to align pixels for time series analysis through the full Landsat record. (https://www.usgs.gov/core-science-systems/nli/landsat/landsat-collections)"
               },
               {
                  "id":"5e83d1197cc754b4",
                  "fieldName":"Calendar_Date",
                  "dictionaryLink":"",
                  "value":"19920704"
               },
               {
                  "id":"5e83d11966963850",
                  "fieldName":"Currentness_Reference",
                  "dictionaryLink":"",
                  "value":"Ground Condition"
               },
               {
                  "id":"5e83d119fc177449",
                  "fieldName":"Progress",
                  "dictionaryLink":"",
                  "value":"In Work"
               },
               {
                  "id":"5e83d11960ba536d",
                  "fieldName":"Maintenance_and_Update_Frequency",
                  "dictionaryLink":"",
                  "value":"Continually"
               },
               {
                  "id":"5e83d119586f0830",
                  "fieldName":"G-Ring_Latitude",
                  "dictionaryLink":"",
                  "value":"33.98005"
               },
               {
                  "id":"5e83d1191be10c7c",
                  "fieldName":"G-Ring_Longitude",
                  "dictionaryLink":"",
                  "value":"-104.13244"
               },
               {
                  "id":"5e83d119281579e6",
                  "fieldName":"Theme_Keyword_Thesaurus",
                  "dictionaryLink":"",
                  "value":"ISO 19115 Topic Category"
               },
               {
                  "id":"5e83d1194abc14dd",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"ImageryBaseMapsEarthCover"
               },
               {
                  "id":"60470441398df5b",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Remote Sensing"
               },
               {
                  "id":"604704413623c358",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Collection 2"
               },
               {
                  "id":"60470441c22543ba",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Infrared Imagery"
               },
               {
                  "id":"604704413ae1d393",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Imagery"
               },
               {
                  "id":"60470441a5f6eec9",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Visible Imagery"
               },
               {
                  "id":"6047044176fd6e3d",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Visible Wavelengths"
               },
               {
                  "id":"604704417339664e",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Radiance"
               },
               {
                  "id":"60470441fb324cd2",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Landsat"
               },
               {
                  "id":"60470441ef8940aa",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Satellite"
               },
               {
                  "id":"60470441fd132ec0",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) "
               },
               {
                  "id":"6047044187bc92ee",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Thematic Mapper (TM)"
               },
               {
                  "id":"6047044159f1e540",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"60470441627f211c",
                  "fieldName":"Theme_Keyword",
                  "dictionaryLink":"",
                  "value":"Infrared Wavelengths"
               },
               {
                  "id":"5e83d119796d51f5",
                  "fieldName":"Place_Keyword_Thesaurus",
                  "dictionaryLink":"",
                  "value":"U.S. Department of Commerce, 1995, Countries, dependencies, areas of special sovereignty, and their principal administrative divisions, Federal Information Processing Standard 10-4, Washington, D.C., National Institute of Standards and Technology"
               },
               {
                  "id":"5e83d119abd5525a",
                  "fieldName":"Place_Keyword",
                  "dictionaryLink":"",
                  "value":"Land Surface"
               },
               {
                  "id":"5e83d119ba008a5",
                  "fieldName":"Access_Constraints",
                  "dictionaryLink":"",
                  "value":"Downloading and using these data signifies that the user comprehends and will comply with the USGS Privacy Policy.  Reading and understanding all portions of the metadata will protect the interests of both the user and the USGS. Please refer to https://www.usgs.gov/privacy.html."
               },
               {
                  "id":"5e83d11a5fe32ed8",
                  "fieldName":"Use_Constraints",
                  "dictionaryLink":"",
                  "value":"There is no guarantee of warranty concerning the accuracy of these data. Users should not use these data for critical applications without a full awareness of their limitations. Temporal changes may have occurred since the data were collected and some parts of these data may no longer represent actual surface conditions.  There are no restrictions regarding the use of Landsat data distributed by the U.S. Geological Survey (USGS), unless expressly identified prior to or at the time of receipt. For Landsat, we request the following statement be used: Data available from the U.S. Geological Survey. Please refer to https://www.usgs.gov/centers/eros/data-citation for more information on data citation."
               },
               {
                  "id":"5e83d11ab9c2ebac",
                  "fieldName":"Contact_Person",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11ab32a4f35",
                  "fieldName":"Contact_Organization",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d11ab61696bc",
                  "fieldName":"Contact_Position",
                  "dictionaryLink":"",
                  "value":"Customer Service Representative"
               },
               {
                  "id":"5e83d11ad3056599",
                  "fieldName":"Address_Type",
                  "dictionaryLink":"",
                  "value":"Mailing and Physical Address"
               },
               {
                  "id":"5e83d11a257fe85d",
                  "fieldName":"Address",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center 47914 252nd Street"
               },
               {
                  "id":"5e83d11ab3eacc95",
                  "fieldName":"City",
                  "dictionaryLink":"",
                  "value":"Sioux Falls"
               },
               {
                  "id":"5e83d11aa1694de",
                  "fieldName":"State_or_Province",
                  "dictionaryLink":"",
                  "value":"SD"
               },
               {
                  "id":"5e83d11ad2c2fb60",
                  "fieldName":"Postal_Code",
                  "dictionaryLink":"",
                  "value":"57198-0001"
               },
               {
                  "id":"5e83d11ae22dc5e3",
                  "fieldName":"Country",
                  "dictionaryLink":"",
                  "value":"USA"
               },
               {
                  "id":"5e83d11aa9f27cf6",
                  "fieldName":"Contact_Electronic_Mail_Address",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a540dcfe9",
                  "fieldName":"Hours_of_Service",
                  "dictionaryLink":"",
                  "value":"0800 - 1600 CT, M-F, 1400 - 2200 h GMT"
               },
               {
                  "id":"5e83d11ac32f724c",
                  "fieldName":"Contact_Instructions",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a5c48c7bc",
                  "fieldName":"Browse_Graphic_File_Name",
                  "dictionaryLink":"",
                  "value":"Not Available"
               },
               {
                  "id":"5e83d11ad8675c1d",
                  "fieldName":"Browse_Graphic_File_Description",
                  "dictionaryLink":"",
                  "value":"This is a compressed and resampled version of the Landsat image to help validate area of interest, data quality, and the impact of cloud cover, if it exists."
               },
               {
                  "id":"5e83d11ae71a3055",
                  "fieldName":"Browse_Graphic_File_Type",
                  "dictionaryLink":"",
                  "value":"JPEG"
               },
               {
                  "id":"5e83d11a9000b8ec",
                  "fieldName":"Security_Classification_System",
                  "dictionaryLink":"",
                  "value":"None"
               },
               {
                  "id":"5e83d11aa39ac7de",
                  "fieldName":"Security_Classification",
                  "dictionaryLink":"",
                  "value":"Unclassified"
               },
               {
                  "id":"5e83d11a1883ab2b",
                  "fieldName":"Security_Handling_Description",
                  "dictionaryLink":"",
                  "value":"None"
               },
               {
                  "id":"5e83d11a7a5e7378",
                  "fieldName":"Native_Data_Set_Environment",
                  "dictionaryLink":"",
                  "value":"Oracle"
               },
               {
                  "id":"5e83d11a53641db4",
                  "fieldName":"Originator",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d11a23ff726c",
                  "fieldName":"Publication_Date",
                  "dictionaryLink":"",
                  "value":"19720723"
               },
               {
                  "id":"5e83d11a3ae40d19",
                  "fieldName":"Title",
                  "dictionaryLink":"",
                  "value":"Landsat Thematic Mapper Collection 2"
               },
               {
                  "id":"5e83d11aa866af3f",
                  "fieldName":"Geospatial_Data_Presentation_Form",
                  "dictionaryLink":"",
                  "value":"Remote-Sensing Image"
               },
               {
                  "id":"5e83d11a3ff08366",
                  "fieldName":"Publication_Place",
                  "dictionaryLink":"",
                  "value":"Sioux Falls,  SD USA"
               },
               {
                  "id":"5e83d11ac764e8b1",
                  "fieldName":"Publisher",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d11af27a7036",
                  "fieldName":"Online_Linkage",
                  "dictionaryLink":"",
                  "value":"https://doi.org/10.5066/P9IAXOVV"
               },
               {
                  "id":"5e83d11a393fe841",
                  "fieldName":"Attribute_Accuracy_Report",
                  "dictionaryLink":"",
                  "value":"The identification of features is provided by the spectral signature which is the distinct electromagnetic energy it emits, reflects, or otherwise transmits.  Other signatures such as tone (lightness or darkness), texture (surface roughness or smoothness), pattern, shadow, shape, and size are just as important.  Thus, through the use of Landsat data, such elements as water features, soils, and vegetation can be identified and distinguished from each other."
               },
               {
                  "id":"5e83d11abfffd008",
                  "fieldName":"Logical_Consistency_Report",
                  "dictionaryLink":"",
                  "value":"N/A"
               },
               {
                  "id":"5e83d11a3a7b431a",
                  "fieldName":"Completeness_Report",
                  "dictionaryLink":"",
                  "value":"The Landsat satellites circle the Earth on a sun-synchronous, near-polar orbit imaging the same 185 km (115 miles) ground swath every 16 days."
               },
               {
                  "id":"5e83d11adfb52c64",
                  "fieldName":"Process_Description",
                  "dictionaryLink":"",
                  "value":"Landsat Collection 2 Level-2 surface reflectance and surface temperature science products are generated from Collection 2 Level-1 inputs that meet the less than 76 degrees Solar Zenith Angle constraint and include the required auxiliary data inputs to generate a scientifically viable product. For more information visit https://www.usgs.gov/core-science-systems/nli/landsat/landsat-collection-2-level-2-science-products."
               },
               {
                  "id":"5e83d11a886ef0cd",
                  "fieldName":"Process_Date",
                  "dictionaryLink":"",
                  "value":"19920704"
               },
               {
                  "id":"5e83d11a11da2ce6",
                  "fieldName":"Contact_Person",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11ab064168",
                  "fieldName":"Contact_Organization",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d11a2304f959",
                  "fieldName":"Contact_Organization",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d11a3bbcd0f2",
                  "fieldName":"Contact_Person",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a9e6d5dd6",
                  "fieldName":"Contact_Position",
                  "dictionaryLink":"",
                  "value":"Customer Service Representative"
               },
               {
                  "id":"5e83d11adff5e984",
                  "fieldName":"Address_Type",
                  "dictionaryLink":"",
                  "value":"Mailing and Physical Address"
               },
               {
                  "id":"5e83d11af63fd80e",
                  "fieldName":"Address",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center 47914 252nd Street"
               },
               {
                  "id":"5e83d11afb799f4a",
                  "fieldName":"City",
                  "dictionaryLink":"",
                  "value":"Sioux Falls"
               },
               {
                  "id":"5e83d11a44b6f7fa",
                  "fieldName":"State_or_Province",
                  "dictionaryLink":"",
                  "value":"SD"
               },
               {
                  "id":"5e83d11a7fc5e25c",
                  "fieldName":"Postal_Code",
                  "dictionaryLink":"",
                  "value":"57198-0001"
               },
               {
                  "id":"5e83d11a85d3d795",
                  "fieldName":"Country",
                  "dictionaryLink":"",
                  "value":"USA"
               },
               {
                  "id":"5e83d11ae7dc9222",
                  "fieldName":"Contact_Electronic_Mail_Address",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11aeb5a94ea",
                  "fieldName":"Hours_of_Service",
                  "dictionaryLink":"",
                  "value":"0800 - 1600 CT, M-F, 1400 - 2200 h GMT"
               },
               {
                  "id":"5e83d11ac055787a",
                  "fieldName":"Contact_Instructions",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a858097e6",
                  "fieldName":"Cloud_Cover",
                  "dictionaryLink":"",
                  "value":"0.000 percent"
               },
               {
                  "id":"5e83d11af1f2843b",
                  "fieldName":"Indirect_Spatial_Reference",
                  "dictionaryLink":"",
                  "value":"Landsats 4 and 5 collected data at 30-meter ground resolution. Band 6 was acquired at 120-meter resolution, but products are resampled to 30-meter pixels."
               },
               {
                  "id":"5e83d11a6b630fc9",
                  "fieldName":"Direct_Spatial_Reference_Method",
                  "dictionaryLink":"",
                  "value":"Raster"
               },
               {
                  "id":"5e83d11a19dda88e",
                  "fieldName":"Raster_Object_Type",
                  "dictionaryLink":"",
                  "value":"Pixel"
               },
               {
                  "id":"5e83d11ac4295b28",
                  "fieldName":"Contact_Person",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a883a82be",
                  "fieldName":"Contact_Organization",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d11aa774b26b",
                  "fieldName":"Contact_Position",
                  "dictionaryLink":"",
                  "value":"Customer Service Representative"
               },
               {
                  "id":"5e83d11ad2958365",
                  "fieldName":"Address_Type",
                  "dictionaryLink":"",
                  "value":"Mailing and Physical Address"
               },
               {
                  "id":"5e83d11af5fa6ab8",
                  "fieldName":"Address",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center 47914 252nd Street"
               },
               {
                  "id":"5e83d11a60757afb",
                  "fieldName":"City",
                  "dictionaryLink":"",
                  "value":"Sioux Falls"
               },
               {
                  "id":"5e83d11a7d7342b1",
                  "fieldName":"State_or_Province",
                  "dictionaryLink":"",
                  "value":"SD"
               },
               {
                  "id":"5e83d11a7c7a009c",
                  "fieldName":"Postal_Code",
                  "dictionaryLink":"",
                  "value":"57198-0001"
               },
               {
                  "id":"5e83d11a6085f9c4",
                  "fieldName":"Country",
                  "dictionaryLink":"",
                  "value":"USA"
               },
               {
                  "id":"5e83d11a8414a2f7",
                  "fieldName":"Contact_Electronic_Mail_Address",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11afd59eb5f",
                  "fieldName":"Hours_of_Service",
                  "dictionaryLink":"",
                  "value":"0800 - 1600 CT, M-F, 1400 - 2200 h GMT"
               },
               {
                  "id":"5e83d11a95f467a9",
                  "fieldName":"Contact_Instructions",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11abaff6573",
                  "fieldName":"Resource_Description",
                  "dictionaryLink":"",
                  "value":"LANDSAT TM Collection 2 Entity ID: LT50310362012125EDC00"
               },
               {
                  "id":"5e83d11a92c457e3",
                  "fieldName":"Distribution_Liability",
                  "dictionaryLink":"",
                  "value":"Although these data have been processed successfully on a computer system at the USGS, no warranty expressed or implied is made by the USGS regarding the use of the data on any other system, nor does the act of distribution constitute any such warranty."
               },
               {
                  "id":"5e83d11af5c0878d",
                  "fieldName":"Format_Name",
                  "dictionaryLink":"",
                  "value":"Geographcal-Tagged Image File Format (GeoTIFF), or Joint Photographic Experts Group (JPEG)"
               },
               {
                  "id":"5e83d11ae0bb7b8f",
                  "fieldName":"Network_Resource_Name",
                  "dictionaryLink":"",
                  "value":"https://earthexplorer.usgs.gov"
               },
               {
                  "id":"5e83d11afb83c4cd",
                  "fieldName":"Fees",
                  "dictionaryLink":"",
                  "value":"Fees:  All Landsat data held in the USGS EROS archive are downloadable at no cost to all users."
               },
               {
                  "id":"5e83d11abe18accc",
                  "fieldName":"Technical_Prerequisites",
                  "dictionaryLink":"",
                  "value":"Adequate computer capability is the only technical prerequisite for viewing data in digital form.\r\n\r\n"
               },
               {
                  "id":"5e83d11a1a5caead",
                  "fieldName":"Beginning_Date",
                  "dictionaryLink":"",
                  "value":"19821113"
               },
               {
                  "id":"5e83d11a7ab01984",
                  "fieldName":"Ending_Date",
                  "dictionaryLink":"",
                  "value":"20011001"
               },
               {
                  "id":"5e83d11a5fe31dbe",
                  "fieldName":"Metadata_Date",
                  "dictionaryLink":"",
                  "value":"20160422"
               },
               {
                  "id":"5e83d11ace8c4be6",
                  "fieldName":"Metadata_Review_Date",
                  "dictionaryLink":"",
                  "value":"As Needed"
               },
               {
                  "id":"5e83d11aaf7b1a8",
                  "fieldName":"Contact_Person",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a6fe629a0",
                  "fieldName":"Contact_Organization",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center"
               },
               {
                  "id":"5e83d11a612f9dca",
                  "fieldName":"Contact_Position",
                  "dictionaryLink":"",
                  "value":"Customer Service Representative"
               },
               {
                  "id":"5e83d11ac64398d4",
                  "fieldName":"Address_Type",
                  "dictionaryLink":"",
                  "value":"Mailing and Physical Address"
               },
               {
                  "id":"5e83d11a2ec3653a",
                  "fieldName":"Address",
                  "dictionaryLink":"",
                  "value":"U.S. Geological Survey (USGS) Earth Resources Observation and Science (EROS) Center 47914 252nd Street"
               },
               {
                  "id":"5e83d11a9dc347a2",
                  "fieldName":"City",
                  "dictionaryLink":"",
                  "value":"Sioux Falls"
               },
               {
                  "id":"5e83d11a557d59a2",
                  "fieldName":"State_or_Province",
                  "dictionaryLink":"",
                  "value":"SD"
               },
               {
                  "id":"5e83d11ab5408135",
                  "fieldName":"Postal_Code",
                  "dictionaryLink":"",
                  "value":"57198-0001"
               },
               {
                  "id":"5e83d11a12675aa1",
                  "fieldName":"Country",
                  "dictionaryLink":"",
                  "value":"USA"
               },
               {
                  "id":"5e83d11a44f7d71b",
                  "fieldName":"Contact_Electronic_Mail_Address",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a72d65a0f",
                  "fieldName":"Hours_of_Service",
                  "dictionaryLink":"",
                  "value":"0800 - 1600 CT, M-F, 1400 - 2200 h GMT"
               },
               {
                  "id":"5e83d11ac62ff739",
                  "fieldName":"Contact_Instructions",
                  "dictionaryLink":"",
                  "value":"custserv@usgs.gov"
               },
               {
                  "id":"5e83d11a2b75b6e1",
                  "fieldName":"Metadata_Standard_Name",
                  "dictionaryLink":"",
                  "value":"Content Standard for Digital Geospatial Metadata"
               },
               {
                  "id":"5e83d11a7c7a0451",
                  "fieldName":"Metadata_Standard_Version",
                  "dictionaryLink":"",
                  "value":"FGDC-STD-001-1998, Version 2"
               },
               {
                  "id":"5e83d11a1ab1600f",
                  "fieldName":"Metadata_Time_Convention",
                  "dictionaryLink":"",
                  "value":"Local Time"
               },
               {
                  "id":"5e83d11a6418b4e",
                  "fieldName":"Metadata_Access_Constraints",
                  "dictionaryLink":"",
                  "value":"None"
               },
               {
                  "id":"5e83d11a37751741",
                  "fieldName":"Metadata_Use_Constraints",
                  "dictionaryLink":"",
                  "value":"None"
               },
               {
                  "id":"5e83d11a6948e4a6",
                  "fieldName":"Metadata_Security_Classification_System",
                  "dictionaryLink":"",
                  "value":"None"
               },
               {
                  "id":"5e83d11a96bf92d9",
                  "fieldName":"Metadata_Security_Classification",
                  "dictionaryLink":"",
                  "value":"Unclassified"
               },
               {
                  "id":"5e83d11ae14acc9d",
                  "fieldName":"Metadata_Security_Handling_Description",
                  "dictionaryLink":"",
                  "value":"None"
               }
            ],
            "hasCustomizedMetadata":"None",
            "options":{
               "bulk":True,
               "download":True,
               "order":False,
               "secondary":False
            },
            "selected":{
               "bulk":False,
               "compare":False,
               "order":False
            },
            "spatialBounds":{
               "type":"Polygon",
               "coordinates":[
                  [
                     [
                        -104.13244,
                        33.69355
                     ],
                     [
                        -104.13244,
                        35.56248
                     ],
                     [
                        -101.69857,
                        35.56248
                     ],
                     [
                        -101.69857,
                        33.69355
                     ],
                     [
                        -104.13244,
                        33.69355
                     ]
                  ]
               ]
            },
            "spatialCoverage":{
               "type":"Polygon",
               "coordinates":[
                  [
                     [
                        -104.13244,
                        33.98005
                     ],
                     [
                        -102.16336,
                        33.69355
                     ],
                     [
                        -101.69857,
                        35.27063
                     ],
                     [
                        -103.70399,
                        35.56248
                     ],
                     [
                        -104.13244,
                        33.98005
                     ]
                  ]
               ]
            },
            "temporalCoverage":{
               "endDate":"2012-05-04 00:00:00",
               "startDate":"2012-05-04 00:00:00"
            },
            "publishDate":"2022-06-22 19:23:13-05"
         }
      ],
      "recordsReturned":5,
      "totalHits":2891701,
      "totalHitsAccuracy":"approximate",
      "isCustomized":False,
      "numExcluded":0,
      "startingNumber":1,
      "nextRecord":6
   },
   "errorCode":"None",
   "errorMessage":"None"
}

print(file['errorCode'])

None
